# cache

Commands and population data for importing in to a database.

Even M$ visual studio products have to cache symbols somewhere.

I suppose this notebook should offline documentation too.

Rules:

- Cache functions and caches must be idempotent, calling the function multiple times with the same or different parameters will produce enjoyable results.
- Calling functions without any parameters should result in the default cache updating procedure, run for some time, and then exit.
- Caches stored in this notebook should be small, less than 1 MB each.
- Caches should be uniquely indexed and sorted so it is easy to display when something changes.



## notebook cache

Information about the current library cached.


### keyword index

A cache of question marked sentences to use as keywords in looking up functions.

Long term: keywords and context can be derived from code and ```code``` in markdown cells.

TODO: look at media server caching techniques.


#### the cache

cell cache?


In [ ]:

// cell cache automatically replaced
var cellCache = [
    [
        1745121387270,
        "/Users/briancullinan/jupyter_ops/Algorithms/audio.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[10]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[11]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[12]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[13]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[14]",
        {
            "from": 14,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[15]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[16]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[17]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[18]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[19]",
        {
            "from": 19,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[2]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[3]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[4]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[5]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[6]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[7]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[8]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121384789,
        "/Users/briancullinan/jupyter_ops/Algorithms/ffmpeg commands.ipynb[9]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Algorithms/graph algorithms.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "reduce noise from images after resizing",
                "reduce noise from images after resizing",
                "source: https://www.imagemagick.org/discourse-server/viewtopic.phpt=15855"
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[10]",
        {
            "from": 14,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[11]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[12]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[13]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[14]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[15]",
        {
            "from": 19,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[2]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[4]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[5]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[6]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[7]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[8]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121530255,
        "/Users/briancullinan/jupyter_ops/Algorithms/image magik commands.ipynb[9]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "llama vision",
                "llama vision"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "analyze image with llm",
                "analyze image with llm"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[10]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "mask image",
                "mask image",
                "remove background image"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[11]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                "inpaint mask",
                "inpaint mask"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[12]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                "image 2 image",
                "image 2 image"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[13]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                "whisk images",
                "whisk images"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[14]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                "local lumina",
                "local lumina"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[15]",
        {
            "from": 33,
            "to": 34,
            "questions": [
                "stability 3.5",
                "stability 3.5"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[16]",
        {
            "from": 35,
            "to": 36,
            "questions": [
                "login huggingface",
                "login huggingface"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "llm deceive",
                "llm deceive"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "llm voice",
                "llm voice"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[6]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[7]",
        {
            "from": 16,
            "to": 18,
            "questions": [
                "ollama vision request",
                "ollama vision request",
                "request ollama vision"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "start a bunch of llm rpc services",
                "start a bunch of llm rpc services"
            ]
        }
    ],
    [
        1744667039123,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama vision.ipynb[9]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "stable diffusion request",
                "stable diffusion request"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "search llama",
                "search llama",
                "ask llm matching function",
                "ask llm which function matches",
                "search jupyter notebooks using llama-gpt"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "ask llm about categories",
                "ask llm about categories"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[10]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "create llm session",
                "create llm session"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "ask llm to generalize categories",
                "ask llm to generalize categories"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "ask llm for a shorter list of categories",
                "ask llm for a shorter list of categories"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "ask llm about functions",
                "ask llm about functions"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "ask llm about code",
                "ask llm about code"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "ask llm about notebooks",
                "ask llm about notebooks"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "ask llm to summerize",
                "ask llm to summerize"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "store llama function",
                "store llama function",
                "TODO: store in JSON files in Resources/Projectsmove this code to caches.ipynb"
            ]
        }
    ],
    [
        1745287168515,
        "/Users/briancullinan/jupyter_ops/Algorithms/llama.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "store all notebook llm functions",
                "store all notebook llm functions"
            ]
        }
    ],
    [
        1741116318292,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm blogging.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "blog about code",
                "blog about code"
            ]
        }
    ],
    [
        1741116318292,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm blogging.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "parse patch file",
                "parse patch file"
            ]
        }
    ],
    [
        1741116318292,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm blogging.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "TODO: summerize file changesdownloads try to infer purposeuse file watcher install time machine  i version my entire disk",
                "TODO: summerize file changesdownloads try to infer purposeuse file watcher install time machine  i version my entire disk"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "store llm response",
                "store llm response"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "general chit chat",
                "general chit chat"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "relevant llm history",
                "relevant llm history"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "relevant chat keywords",
                "relevant chat keywords"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "relevant history timestamps",
                "relevant history timestamps"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[5]",
        {
            "from": 13,
            "to": 15,
            "questions": [
                "classify llm prompt",
                "classify llm prompt"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[6]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "handle conversation",
                "handle conversation"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[7]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "add conversation context",
                "add conversation context"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[8]",
        {
            "from": 20,
            "to": 22,
            "questions": [
                "llm load memories",
                "llm load memories"
            ]
        }
    ],
    [
        1744675441375,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm chat.ipynb[9]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "llm save memories",
                "llm save memories"
            ]
        }
    ],
    [
        1736737215091,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm dnd.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "llm scaffold",
                "llm scaffold"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "bash project files",
                "bash project files"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "generate code",
                "generate code"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "make code file with llm",
                "make code file with llm"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "edit code cell in notebook",
                "edit code cell in notebook"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "add llm code cell",
                "add llm code cell"
            ]
        }
    ],
    [
        1744661624427,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm scaffold.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740355915824,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm tools.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "scan chat logs",
                "scan chat logs",
                "scan sqlite tools"
            ]
        }
    ],
    [
        1740355915824,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm tools.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "ask llm about emotions",
                "ask llm about emotions"
            ]
        }
    ],
    [
        1740355915824,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm tools.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "cache chat logs",
                "cache chat logs"
            ]
        }
    ],
    [
        1740355915824,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm tools.ipynb[3]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "ask llm about chat conversations",
                "ask llm about chat conversations"
            ]
        }
    ],
    [
        1740355915824,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm tools.ipynb[4]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "llm respond like a personality",
                "llm respond like a personality"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "business plan llm",
                "business plan llm",
                "write a business plan with llm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "write creatively llm",
                "write creatively llm",
                "write creatively with an llm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "argue with multiple llms",
                "argue with multiple llms"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "research paper llm",
                "research paper llm",
                "write a research paper with llm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[4]",
        {
            "from": 14,
            "to": 17,
            "questions": [
                "select llm",
                "select llm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[5]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "ask llm to write chapter titles and descriptions",
                "ask llm to write chapter titles and descriptions"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[6]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "ask llm to brainstorm",
                "ask llm to brainstorm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[7]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "write a grant proposal with llm",
                "write a grant proposal with llm"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[8]",
        {
            "from": 25,
            "to": 27,
            "questions": [
                "decode xlsx spreadsheet",
                "decode xlsx spreadsheet"
            ]
        }
    ],
    [
        1742093404929,
        "/Users/briancullinan/jupyter_ops/Algorithms/llm writing.ipynb[9]",
        {
            "from": 28,
            "to": 30,
            "questions": [
                "elaborate list with llm",
                "elaborate list with llm"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "detect people",
                "detect people"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "motion detection",
                "motion detection"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[10]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "transition live stream",
                "transition live stream"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "python write video",
                "python write video"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[3]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "python write images to ffmpeg stream",
                "python write images to ffmpeg stream"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[4]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "create a live streaming request",
                "create a live streaming request"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[5]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "authorize youtube in python",
                "authorize youtube in python"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[6]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "list broadcasts",
                "list broadcasts"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[7]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "create live stream",
                "create live stream"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[8]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "list live stream",
                "list live stream"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Algorithms/opencv.ipynb[9]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "bind youtube stream",
                "bind youtube stream"
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Algorithms/search algorithms.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Algorithms/sorting techniques.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1736791179588,
        "/Users/briancullinan/jupyter_ops/Algorithms/stable diffusion.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Algorithms/time complexity.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[10]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[11]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[12]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[13]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[14]",
        {
            "from": 14,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[15]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[16]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[17]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[18]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[19]",
        {
            "from": 19,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[2]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[20]",
        {
            "from": 20,
            "to": 20,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[21]",
        {
            "from": 21,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[22]",
        {
            "from": 22,
            "to": 22,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[23]",
        {
            "from": 23,
            "to": 23,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[24]",
        {
            "from": 24,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[25]",
        {
            "from": 25,
            "to": 25,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[26]",
        {
            "from": 26,
            "to": 26,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[27]",
        {
            "from": 27,
            "to": 27,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[28]",
        {
            "from": 28,
            "to": 28,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[29]",
        {
            "from": 29,
            "to": 29,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[3]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[30]",
        {
            "from": 30,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[31]",
        {
            "from": 31,
            "to": 31,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[4]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[5]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[6]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[7]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[8]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1724633526120,
        "/Users/briancullinan/jupyter_ops/Algorithms/trainmodel.ipynb[9]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513375970000,
        "/Users/briancullinan/jupyter_ops/Analytics/budgeting.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[10]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[11]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[12]",
        {
            "from": 14,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[4]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[5]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[6]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[7]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[8]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745121885396,
        "/Users/briancullinan/jupyter_ops/Analytics/calculations for life.ipynb[9]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Analytics/data cleaning.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "Crime reports",
                "Crime reports",
                "https://www.maricopacountyattorney.org/CivicAlerts.aspxAID=400"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "https://www.amazon.com/gp/yourstore/iyr/ref=pd_ys_iyr_nextie=UTF8&collection=watched&iyrGroup=&maxItem=616&minItem=600",
                "https://www.amazon.com/gp/yourstore/iyr/ref=pd_ys_iyr_nextie=UTF8&collection=watched&iyrGroup=&maxItem=616&minItem=600"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[10]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                "schedule crawl domain",
                "schedule crawl domain"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[11]",
        {
            "from": 29,
            "to": 33,
            "questions": [
                "collect all bookmarks",
                "collect all bookmarks"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[12]",
        {
            "from": 34,
            "to": 37,
            "questions": [
                "search results as json",
                "search results as json"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[2]",
        {
            "from": 7,
            "to": 9,
            "questions": [
                "meta search all",
                "meta search all"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[3]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "schedule search all",
                "schedule search all"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[4]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "tell joke",
                "tell joke"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[5]",
        {
            "from": 14,
            "to": 17,
            "questions": [
                "multi crawl",
                "multi crawl"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[6]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "crawl domain",
                "crawl domain"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[7]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "domain cache tools",
                "domain cache tools"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[8]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "browser crawler tools",
                "browser crawler tools"
            ]
        }
    ],
    [
        1738358792864,
        "/Users/briancullinan/jupyter_ops/Analytics/data collection.ipynb[9]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                "analyze cache file",
                "analyze cache file"
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Analytics/data visualization.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install movie database",
                "install movie database"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Find cross section of actors and movies",
                "Find cross section of actors and movies"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "use elastic search from node",
                "use elastic search from node"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[4]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "create movie database tables",
                "create movie database tables"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[5]",
        {
            "from": 9,
            "to": 11,
            "questions": [
                "express handle movie search",
                "express handle movie search",
                "Or use ZMQ interface like ijupyter"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "express list movies database",
                "express list movies database"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "movie page",
                "movie page"
            ]
        }
    ],
    [
        1744599566227,
        "/Users/briancullinan/jupyter_ops/Analytics/movie database.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "movie tile",
                "movie tile"
            ]
        }
    ],
    [
        1592414050000,
        "/Users/briancullinan/jupyter_ops/Analytics/parsing email addresses.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "search google email format",
                "search google email format"
            ]
        }
    ],
    [
        1592414050000,
        "/Users/briancullinan/jupyter_ops/Analytics/parsing email addresses.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511638019000,
        "/Users/briancullinan/jupyter_ops/Analytics/reconcile timeline data.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "read Google calendar and timeline data",
                "read Google calendar and timeline data"
            ]
        }
    ],
    [
        1557605569000,
        "/Users/briancullinan/jupyter_ops/Analytics/resume.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "course list pie chart",
                "course list pie chart"
            ]
        }
    ],
    [
        1557605569000,
        "/Users/briancullinan/jupyter_ops/Analytics/resume.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test course list pie chart",
                "test course list pie chart"
            ]
        }
    ],
    [
        1557605569000,
        "/Users/briancullinan/jupyter_ops/Analytics/resume.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557605569000,
        "/Users/briancullinan/jupyter_ops/Analytics/resume.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Analytics/time series.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Analytics/user retention.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Art/ai image ethics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Art/concept art workflow.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1733772316956,
        "/Users/briancullinan/jupyter_ops/Art/stable diffusion.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "rename txt2img directories",
                "rename txt2img directories"
            ]
        }
    ],
    [
        1733772316956,
        "/Users/briancullinan/jupyter_ops/Art/stable diffusion.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561646772000,
        "/Users/briancullinan/jupyter_ops/Art/virtual worlds.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559494801000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/aws.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "aws rpc wrapper",
                "aws rpc wrapper"
            ]
        }
    ],
    [
        1559494801000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/aws.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "latest s3 bucket",
                "latest s3 bucket"
            ]
        }
    ],
    [
        1559494801000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/aws.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Cloud Services/cloud security.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738998280247,
        "/Users/briancullinan/jupyter_ops/Cloud Services/cloudflare.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Cloud Services/cost optimization.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516666686000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/deploy host.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "deploy aws",
                "deploy aws"
            ]
        }
    ],
    [
        1516666686000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/deploy host.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516666686000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/deploy host.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516666686000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/deploy host.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Cloud Services/deployment strategies.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "list google bucket url map",
                "list google bucket url map"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test list url map",
                "test list url map"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[10]",
        {
            "from": 20,
            "to": 22,
            "questions": [
                "setup sheet backend",
                "setup sheet backend",
                "sheet backend handler"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "wait for cloud operation",
                "wait for cloud operation",
                "wait google cloud operation"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "add a google bucket web map",
                "add a google bucket web map"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "test global forward",
                "test global forward"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "test bucket web map",
                "test bucket web map"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "check dns",
                "check dns"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "test check dns",
                "test check dns"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "invalidate cdn cache",
                "invalidate cdn cache"
            ]
        }
    ],
    [
        1738639631974,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud api.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "authorize google service",
                "authorize google service",
                "authorize google services"
            ]
        }
    ],
    [
        1556343901000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud commands.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "upload output to google cloud storage",
                "upload output to google cloud storage"
            ]
        }
    ],
    [
        1556343901000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud commands.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "add public permissions to google cloud storage",
                "add public permissions to google cloud storage"
            ]
        }
    ],
    [
        1556343901000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud commands.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "copy html to google cloud storage",
                "copy html to google cloud storage"
            ]
        }
    ],
    [
        1556343901000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud commands.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "test command arguments",
                "test command arguments"
            ]
        }
    ],
    [
        1556343901000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google cloud commands.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "deploy a cloud function",
                "deploy a cloud function"
            ]
        }
    ],
    [
        1603062959000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "firebase rpc wrapper",
                "firebase rpc wrapper"
            ]
        }
    ],
    [
        1603062959000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "google cloud function rpc wrapper",
                "google cloud function rpc wrapper"
            ]
        }
    ],
    [
        1603062959000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test google express rpc handler",
                "test google express rpc handler"
            ]
        }
    ],
    [
        1603062959000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "generic gcloud function handler",
                "generic gcloud function handler"
            ]
        }
    ],
    [
        1603062959000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google rpc.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "create a bucket",
                "create a bucket",
                "create storage bucket"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test create bucket",
                "test create bucket"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test stream to output",
                "test stream to output"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "fetch file or stream",
                "fetch file or stream"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "upload files to google cloud",
                "upload files to google cloud"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "copy a file in storage bucket",
                "copy a file in storage bucket"
            ]
        }
    ],
    [
        1643816558000,
        "/Users/briancullinan/jupyter_ops/Cloud Services/google storage api.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "test upload files to google storage",
                "test upload files to google storage"
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Cloud Services/serverless functions.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745039510234,
        "/Users/briancullinan/jupyter_ops/Core/aspects.ipynb[0]",
        {
            "from": 0,
            "to": 6,
            "questions": [
                "inspect every statement",
                "inspect every statement",
                "- expect - deriving from the context find the variables involved and if they do not meet expectation create a \"soft assertion\""
            ]
        }
    ],
    [
        1745039510234,
        "/Users/briancullinan/jupyter_ops/Core/aspects.ipynb[1]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745039510234,
        "/Users/briancullinan/jupyter_ops/Core/aspects.ipynb[2]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745039510234,
        "/Users/briancullinan/jupyter_ops/Core/aspects.ipynb[3]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "cell cache",
                "cell cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "cache notebook",
                "cache notebook",
                "cache notebook questions in fuse",
                "find functions in jupyter notebooks",
                ".map(r => r.replaceAll(/|\\|+|/ig'').trim())",
                ".map(r => r.replaceAll(/|\\|+|/ig'').trim()))"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[10]",
        {
            "from": 25,
            "to": 27,
            "questions": [
                "update code cell",
                "update code cell"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[11]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "update regular cache",
                "update regular cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[12]",
        {
            "from": 30,
            "to": 31,
            "questions": [
                "memory safe async",
                "memory safe async"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[13]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                "refresh regular cache",
                "refresh regular cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[14]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[15]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[16]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "test cache notebook",
                "test cache notebook"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "cache all",
                "cache all",
                "find all notebooks",
                "interpret a jupyter {directory}",
                "get markdown from all notebooks in {directory}"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[4]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "imports cache",
                "imports cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[5]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "create import cache",
                "create import cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[6]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "test import cache",
                "test import cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[7]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "exports cache",
                "exports cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[8]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "create cell cache",
                "create cell cache",
                "create export cache"
            ]
        }
    ],
    [
        1745430227099,
        "/Users/briancullinan/jupyter_ops/Core/cache.ipynb[9]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "test export cache",
                "test export cache"
            ]
        }
    ],
    [
        1562190188000,
        "/Users/briancullinan/jupyter_ops/Core/compile.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562190188000,
        "/Users/briancullinan/jupyter_ops/Core/compile.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562190188000,
        "/Users/briancullinan/jupyter_ops/Core/compile.ipynb[2]",
        {
            "from": 7,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562190188000,
        "/Users/briancullinan/jupyter_ops/Core/compile.ipynb[3]",
        {
            "from": 10,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Core/debugging tools.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Core/environment setup.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[1]",
        {
            "from": 2,
            "to": 5,
            "questions": [
                "project word-cloud",
                "project word-cloud"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[10]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[11]",
        {
            "from": 33,
            "to": 35,
            "questions": [
                "(=\\{((:[^{}]++|\\{(1)\\})++)\\})",
                "(=\\{((:[^{}]++|\\{(1)\\})++)\\})",
                "Find too long of functions using regexp"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[12]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[13]",
        {
            "from": 38,
            "to": 40,
            "questions": [
                "file tools match filename",
                "file tools match filename"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[14]",
        {
            "from": 41,
            "to": 42,
            "questions": [
                "mkdirp",
                "mkdirp"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[15]",
        {
            "from": 43,
            "to": 44,
            "questions": [
                "changing file names",
                "changing file names"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[16]",
        {
            "from": 45,
            "to": 47,
            "questions": [
                "use memory-fs and rewire to audit cli events",
                "use memory-fs and rewire to audit cli events"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[17]",
        {
            "from": 48,
            "to": 49,
            "questions": [
                "TODO: combine with DOM minimizing service from recording app",
                "TODO: combine with DOM minimizing service from recording app",
                "TODO: use static code analysis to find consecutive words in template files"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[18]",
        {
            "from": 50,
            "to": 51,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[2]",
        {
            "from": 6,
            "to": 8,
            "questions": [
                "project imports d3 tree",
                "project imports d3 tree"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[3]",
        {
            "from": 9,
            "to": 12,
            "questions": [
                "glob files",
                "glob files"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[4]",
        {
            "from": 13,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[5]",
        {
            "from": 16,
            "to": 18,
            "questions": [
                "list project files",
                "list project files",
                "List files in a project excluding some excessive searches locations"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[6]",
        {
            "from": 19,
            "to": 21,
            "questions": [
                "common ignore paths",
                "common ignore paths"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[7]",
        {
            "from": 22,
            "to": 24,
            "questions": [
                "List all projects by names",
                "List all projects by names"
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[8]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744480751345,
        "/Users/briancullinan/jupyter_ops/Core/files.ipynb[9]",
        {
            "from": 27,
            "to": 30,
            "questions": [
                "fix project paths",
                "fix project paths"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "import notebook",
                "import notebook",
                "use jupyter notebooks as services",
                "use inheritence in jupyter notebooks",
                "make a class out of jupter notebooks",
                "execute and import a jupyter notebook"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[10]",
        {
            "from": 33,
            "to": 34,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[11]",
        {
            "from": 35,
            "to": 38,
            "questions": [
                "convert regexp matches to an array",
                "convert regexp matches to an array"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[12]",
        {
            "from": 39,
            "to": 41,
            "questions": [
                "run all promises sequentially",
                "run all promises sequentially"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[13]",
        {
            "from": 42,
            "to": 44,
            "questions": [
                "parse cells from a notebook",
                "parse cells from a notebook"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[14]",
        {
            "from": 45,
            "to": 47,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[2]",
        {
            "from": 7,
            "to": 11,
            "questions": [
                "how does node module require work",
                "how does node module require work",
                "turn jupyter notebooks in to modules",
                "Import arbitrary code in to a new context",
                "Import arbitrary code in to the current context"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[3]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[4]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "compile an es module",
                "compile an es module"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[5]",
        {
            "from": 17,
            "to": 19,
            "questions": [
                "compile a csharp file into a DLL",
                "compile a csharp file into a DLL"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[6]",
        {
            "from": 20,
            "to": 22,
            "questions": [
                "compile a dylib from source cell",
                "compile a dylib from source cell"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[7]",
        {
            "from": 23,
            "to": 25,
            "questions": [
                "display interpreted results in markdown",
                "display interpreted results in markdown"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[8]",
        {
            "from": 26,
            "to": 30,
            "questions": [
                "search jupter notebooks for questions and answers",
                "search jupter notebooks for questions and answers",
                "answer questions using jupyter notebooks and simple search"
            ]
        }
    ],
    [
        1745185185929,
        "/Users/briancullinan/jupyter_ops/Core/import.ipynb[9]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                "search notebook questions",
                "search notebook questions"
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[2]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[3]",
        {
            "from": 9,
            "to": 15,
            "questions": [
                "use intent",
                "use intent"
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[4]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[5]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1651283275000,
        "/Users/briancullinan/jupyter_ops/Core/intent.ipynb[6]",
        {
            "from": 20,
            "to": 29,
            "questions": [
                "to exist right (i.e. monitor program state)",
                "to exist right (i.e. monitor program state)",
                "Just like it saysmonitors process state for changes because that is it's purpose. With logic like thatwhat can go wrong"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[0]",
        {
            "from": 0,
            "to": 5,
            "questions": [
                "get kernel json",
                "get kernel json",
                "notebook kernel json",
                "TODO: submit to this page and exported notebook of the kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[1]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "test notebook kernel json",
                "test notebook kernel json"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[10]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "test process meta kernel",
                "test process meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[11]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "TODO: repl process kernel",
                "TODO: repl process kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[12]",
        {
            "from": 30,
            "to": 31,
            "questions": [
                "test repl process using bash",
                "test repl process using bash"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[13]",
        {
            "from": 32,
            "to": 35,
            "questions": [
                "jupyter wire interface",
                "jupyter wire interface"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[14]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[15]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[16]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                "TODO jupyter wire kernel",
                "TODO jupyter wire kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[17]",
        {
            "from": 42,
            "to": 43,
            "questions": [
                "TODO: wire meta kernel",
                "TODO: wire meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[18]",
        {
            "from": 44,
            "to": 46,
            "questions": [
                "node meta kernel",
                "node meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[19]",
        {
            "from": 47,
            "to": 48,
            "questions": [
                "test node meta kernel",
                "test node meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[2]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "custom kernel json",
                "custom kernel json"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[20]",
        {
            "from": 49,
            "to": 50,
            "questions": [
                "node wire kernel",
                "node wire kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[21]",
        {
            "from": 51,
            "to": 52,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[22]",
        {
            "from": 53,
            "to": 54,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[23]",
        {
            "from": 55,
            "to": 58,
            "questions": [
                "https://pypi.org/search/q=&o=&c=Framework+%3A%3A+IPython",
                "https://pypi.org/search/q=&o=&c=Framework+%3A%3A+IPython",
                "TODO: use magics parser from jupyter-nodejsdoes something with splitting up linesconvert magics to babel and run along side the actual code"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[24]",
        {
            "from": 59,
            "to": 61,
            "questions": [
                "decode encode ipython zmq protocol",
                "decode encode ipython zmq protocol"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[25]",
        {
            "from": 62,
            "to": 63,
            "questions": [
                "bind wire sockets",
                "bind wire sockets"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[26]",
        {
            "from": 64,
            "to": 65,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[27]",
        {
            "from": 66,
            "to": 67,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[3]",
        {
            "from": 10,
            "to": 12,
            "questions": [
                "jupyter meta kernel",
                "jupyter meta kernel",
                "kernel language interface"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[4]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "kernel info interface",
                "kernel info interface"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[5]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "meta kernel interface",
                "meta kernel interface"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[6]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "native meta kernel",
                "native meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[7]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "native meta methods",
                "native meta methods"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[8]",
        {
            "from": 21,
            "to": 23,
            "questions": [
                "socket meta kernel",
                "socket meta kernel"
            ]
        }
    ],
    [
        1744650670982,
        "/Users/briancullinan/jupyter_ops/Core/kernels.ipynb[9]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "process meta kernel",
                "process meta kernel"
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[1]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[10]",
        {
            "from": 24,
            "to": 28,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[11]",
        {
            "from": 29,
            "to": 31,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[12]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[13]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[14]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[15]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                "TODO: import falsey javascript in to this notebook",
                "TODO: import falsey javascript in to this notebook"
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[16]",
        {
            "from": 40,
            "to": 44,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[17]",
        {
            "from": 45,
            "to": 47,
            "questions": [
                "display safe object",
                "display safe object"
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[18]",
        {
            "from": 48,
            "to": 49,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[19]",
        {
            "from": 50,
            "to": 51,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[20]",
        {
            "from": 52,
            "to": 53,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[21]",
        {
            "from": 54,
            "to": 55,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[3]",
        {
            "from": 7,
            "to": 10,
            "questions": [
                "transpile code",
                "transpile code"
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[8]",
        {
            "from": 19,
            "to": 21,
            "questions": [
                "babel transpiler",
                "babel transpiler"
            ]
        }
    ],
    [
        1649950603000,
        "/Users/briancullinan/jupyter_ops/Core/languages.ipynb[9]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "readme.md",
                "readme.md",
                "- every module should have a markdown titleat least one question that the code intends to answer ()includes at least (where name is the name of any function in the code block):"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[1]",
        {
            "from": 4,
            "to": 7,
            "questions": [
                "import notebook import all cell modules",
                "import notebook import all cell modules",
                "import files back in to cells (two-way workflow)"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[10]",
        {
            "from": 26,
            "to": 28,
            "questions": [
                "export notebook",
                "export notebook",
                "TODO: automatically find missing imports from distrib/Github/3rd party sources in any language"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[11]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                "inject cells in to notebooks when built with webpack",
                "inject cells in to notebooks when built with webpack"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[12]",
        {
            "from": 31,
            "to": 33,
            "questions": [
                "get cell extension",
                "get cell extension"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[13]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                "nice name",
                "nice name",
                "rename cell to a nice name",
                "TODO: move this to Core import notebook"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[14]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                "authoring header template",
                "authoring header template"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[15]",
        {
            "from": 38,
            "to": 38,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[2]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[3]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[4]",
        {
            "from": 12,
            "to": 15,
            "questions": [
                "replace core requirement",
                "replace core requirement"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[5]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "replace notebook import",
                "replace notebook import"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[6]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[7]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "add missing imports",
                "add missing imports",
                "automatic jupyter dependency injector"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[8]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "test add missing imports",
                "test add missing imports"
            ]
        }
    ],
    [
        1744567867652,
        "/Users/briancullinan/jupyter_ops/Core/notebook.ipynb[9]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[0]",
        {
            "from": 0,
            "to": 5,
            "questions": [
                "walk tree",
                "walk tree"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[1]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[10]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "test enforcing an interface with prototype",
                "test enforcing an interface with prototype"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[11]",
        {
            "from": 30,
            "to": 32,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[12]",
        {
            "from": 33,
            "to": 35,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "select tree",
                "select tree",
                "Get all elements XPath"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[3]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[4]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[5]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[6]",
        {
            "from": 17,
            "to": 19,
            "questions": [
                "resolve a promise properly",
                "resolve a promise properly"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[7]",
        {
            "from": 20,
            "to": 23,
            "questions": [
                "making a class the normal way",
                "making a class the normal way",
                "this is extremely annoying achieve the same effect with prototype"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[8]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "extend prototype class",
                "extend prototype class"
            ]
        }
    ],
    [
        1745164162645,
        "/Users/briancullinan/jupyter_ops/Core/patterns.ipynb[9]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "enforcing an interface",
                "enforcing an interface"
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "get python params",
                "get python params"
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "questions = [re.sub(r'|\\|+'''qflags=re.IGNORECASE).strip()",
                "questions = [re.sub(r'|\\|+'''qflags=re.IGNORECASE).strip()",
                "questions += [re.sub(r'|\\|+'''qflags=re.IGNORECASE).strip()"
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741033686742,
        "/Users/briancullinan/jupyter_ops/Core/pyimport.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "run python cells",
                "run python cells"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "rpc groups",
                "rpc groups"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "get cell rpc groups",
                "get cell rpc groups"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[10]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "test rpc from spec",
                "test rpc from spec"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[11]",
        {
            "from": 24,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[12]",
        {
            "from": 25,
            "to": 25,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "rpc permissions",
                "rpc permissions"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "test get permissions",
                "test get permissions"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "filter command permission",
                "filter command permission"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "store rpc result",
                "store rpc result"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "rpc result",
                "rpc result"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "get environment",
                "get environment",
                "get environment variables"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "get rpc from spec",
                "get rpc from spec"
            ]
        }
    ],
    [
        1745122590612,
        "/Users/briancullinan/jupyter_ops/Core/rpc.ipynb[9]",
        {
            "from": 21,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "get requires",
                "get requires",
                "get requires from source"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[10]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "test get imports",
                "test get imports"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[11]",
        {
            "from": 28,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[12]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[13]",
        {
            "from": 33,
            "to": 34,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[14]",
        {
            "from": 35,
            "to": 40,
            "questions": [
                "tree to html",
                "tree to html"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[15]",
        {
            "from": 41,
            "to": 45,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[16]",
        {
            "from": 46,
            "to": 48,
            "questions": [
                "html to tree",
                "html to tree"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[17]",
        {
            "from": 49,
            "to": 50,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[18]",
        {
            "from": 51,
            "to": 53,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[19]",
        {
            "from": 54,
            "to": 56,
            "questions": [
                "select ast",
                "select ast",
                "select code tree",
                "select code from tree"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "builtin and local modules",
                "builtin and local modules"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[20]",
        {
            "from": 57,
            "to": 58,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[21]",
        {
            "from": 59,
            "to": 60,
            "questions": [
                "select acorn tree",
                "select acorn tree"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[22]",
        {
            "from": 61,
            "to": 63,
            "questions": [
                "make xpaths",
                "make xpaths"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[23]",
        {
            "from": 64,
            "to": 65,
            "questions": [
                "generate css selectors from code",
                "generate css selectors from code"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[24]",
        {
            "from": 66,
            "to": 67,
            "questions": [
                "minimize xpath",
                "minimize xpath"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[25]",
        {
            "from": 68,
            "to": 69,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[26]",
        {
            "from": 70,
            "to": 72,
            "questions": [
                "select from code",
                "select from code",
                "select expression"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[27]",
        {
            "from": 73,
            "to": 74,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[28]",
        {
            "from": 75,
            "to": 76,
            "questions": [
                "test expression on notebook code",
                "test expression on notebook code"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[29]",
        {
            "from": 77,
            "to": 78,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "core dependencies",
                "core dependencies"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[30]",
        {
            "from": 79,
            "to": 82,
            "questions": [
                "Generate unit test from logic branching",
                "Generate unit test from logic branching"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[31]",
        {
            "from": 83,
            "to": 84,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[32]",
        {
            "from": 85,
            "to": 86,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[33]",
        {
            "from": 87,
            "to": 89,
            "questions": [
                "generate unit test",
                "generate unit test"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[34]",
        {
            "from": 90,
            "to": 91,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[35]",
        {
            "from": 92,
            "to": 93,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[36]",
        {
            "from": 94,
            "to": 95,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[37]",
        {
            "from": 96,
            "to": 97,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "test core dependencies",
                "test core dependencies"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[5]",
        {
            "from": 13,
            "to": 15,
            "questions": [
                "get exports from source",
                "get exports from source",
                "get module exports from source"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[6]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[7]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "get parameter names",
                "get parameter names",
                "function parameters"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[8]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "test parameter names",
                "test parameter names"
            ]
        }
    ],
    [
        1745099782891,
        "/Users/briancullinan/jupyter_ops/Core/syntax.ipynb[9]",
        {
            "from": 23,
            "to": 25,
            "questions": [
                "get imports from source",
                "get imports from source"
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "attribute code",
                "attribute code"
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "attribute notebook",
                "attribute notebook"
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745078767190,
        "/Users/briancullinan/jupyter_ops/Core/voidzero.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Databases/active directory.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745296585626,
        "/Users/briancullinan/jupyter_ops/Databases/caches.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "cache rpc functions with llm descriptions",
                "cache rpc functions with llm descriptions",
                "\"amazing\": \"```javascript\\n/**\\n * Calculates the average heart rate and estimates the total heartbeats since birth.\\n * \\n * @param {number} ageInYears The age in years.\\n * @param {number} avgHeartRate The average heart rate in beats per minute.\\n * @param {boolean} [isVerbose=false] Whether to log the detailed output.\\n */\\nfunction estimateTotalHeartbeats(ageInYearsavgHeartRateisVerbose = false) {\\n    const totalBeats = calculateTotalHeartbeats(ageInYearsavgHeartRate);\\n    const formattedOutput = formatOutput(totalBeatsisVerbose);\\n    return formattedOutput;\\n}\\n\\n/**\\n * Calculates the total heartbeats since birth.\\n * \\n * @param {number} ageInYears The age in years.\\n * @param {number} avgHeartRate The average heart rate in beats per minute.\\n * @returns {number} The total heartbeats since birth.\\n */\\nfunction calculateTotalHeartbeats(ageInYearsavgHeartRate) {\\n    const millisecondsPerYear = 365.25 * 24 * 60 * 60 * 1000;\\n    const millisecondsPerBeat = 60000 / avgHeartRate;\\n    const totalMilliseconds = ageInYears * millisecondsPerYear;\\n    const totalBeats = Math.floor(totalMilliseconds / millisecondsPerBeat);\\n    return totalBeats;\\n}\\n\\n/**\\n * Formats the output of the estimated total heartbeats since birth.\\n * \\n * @param {number} totalBeats The total heartbeats since birth.\\n * @param {boolean} isVerbose Whether to log the detailed output.\\n * @returns {string} The formatted output.\\n */\\nfunction formatOutput(totalBeatsisVerbose) {\\n    const output = isVerbose `Estimated total heartbeats since birth: ${totalBeats.toLocaleString()}` : `Approximate heartbeats since birth: ${totalBeats.toLocaleString()}`;\\n    console.log(output);\\n    return output;\\n}\\n\\n// Example usage\\nconst ageInYears = 30;\\nconst avgHeartRate = 75;\\nconst totalBeats = estimateTotalHeartbeats(ageInYearsavgHeartRatetrue);\\n```\\n\\nNote: This improved code includes clear documentation with function comments and JSDoc-style documentation. Each function is designed to perform a specific taskand the code is organized and readable. The example usage demonstrates  call the `estimateTotalHeartbeats` function and log the output.\",",
                "\"react\": \" Refactored Code in React TypeScript\\n\\n Tool Instructions Interface\\n\\n```typescript\\n// tools-instructions.interface.ts\\ninterface ToolInstructions {\\n  instructions: string;\\n  title: string;\\n  subheading: string;\\n}\\n```\\n\\n Instructions Component\\n\\n```typescript\\n// Instructions.tsx\\nimport React from'react';\\nimport { ToolInstructions } from './tools-instructions.interface';\\n\\nconst Instructions: React.FC<ToolInstructions> = ({\\n  instructions,\\n  title,\\n  subheading,\\n}) => {\\n  return (\\n    <div>\\n      <h2>{title}</h2>\\n      {subheading && <h3>{subheading}</h3>}\\n      <p>{instructions}</p>\\n    </div>\\n  );\\n};\\n\\nexport default Instructions;\\n```\\n\\n ToolInstructions Component\\n\\n```typescript\\n// ToolInstructions.tsx\\nimport React from'react';\\nimport { ToolInstructions } from './tools-instructions.interface';\\nimport Instructions from './Instructions';\\n\\nconst ToolInstructions: React.FC<ToolInstructions> = ({\\n  instructions,\\n  title,\\n  subheading,\\n}) => {\\n  return (\\n    <div>\\n      <Instructions\\n        instructions={instructions}\\n        title={title}\\n        subheading={subheading}\\n      />\\n    </div>\\n  );\\n};\\n\\nexport default ToolInstructions;\\n```\\n\\n Use Case: ToolInstructions Component\\n\\n```typescript\\n// ToolInstructionsExample.tsx\\nimport React from'react';\\nimport { ToolInstructions } from './ToolInstructions';\\n\\nconst ToolInstructionsExample: React.FC = () => {\\n  const toolInstructions: ToolInstructions = {\\n    instructions: `When looking for real-time information use relevant functions if available.`,\\n    title: 'Tool Instructions',\\n    subheading: 'Learn  effectively use this tool',\\n  };\\n\\n  return (\\n    <div>\\n      <ToolInstructions {...toolInstructions} />\\n    </div>\\n  );\\n};\\n\\nexport default ToolInstructionsExample;\\n```\\n\\n Large Language Model Interface\\n\\n```typescript\\n// large-language-model.interface.ts\\ninterface LargeLanguageModel {\\n  think: string;\\n}\\n```\\n\\n Large Language Model Component\\n\\n```typescript\\n// LargeLanguageModel.tsx\\nimport React from'react';\\nimport { LargeLanguageModel } from './large-language-model.interface';\\n\\nconst LargeLanguageModel: React.FC<LargeLanguageModel> = ({ think }) => {\\n  return <p>{think}</p>;\\n};\\n\\nexport default LargeLanguageModel;\\n```\\n\\n Use Case: Large Language Model Component\\n\\n```typescript\\n// LargeLanguageModelExample.tsx\\nimport React from'react';\\nimport { LargeLanguageModel } from './LargeLanguageModel';\\n\\nconst LargeLanguageModelExample: React.FC = () => {\\n  const largeLanguageModel: LargeLanguageModel = {\\n    think: '...',\\n  };\\n\\n  return (\\n    <div>\\n      <LargeLanguageModel {...largeLanguageModel} />\\n    </div>\\n  );\\n};\\n\\nexport default LargeLanguageModelExample;\\n```\"",
                "\"react\": \"**utils.ts**\\n```typescript\\nimport * as crypto from 'crypto';\\nimport * as uuid from 'uuid';\\n\\n// Constants\\nexport const DELIM: string = '<IDS|MSG>';\\n\\n// Utilities\\nexport function parseMessage(...args: any[]): { [key: string]: any } {\\n  // Get the index of the delimiter\\n  const idx = args.findIndex((arg) => arg === DELIM);\\n  if (idx === -1) return {};\\n\\n  // Extract uuidsheadermetadatacontent\\n  const uuids = args.slice(0idx);\\n  const [headerparentmetadatacontent] = args.slice(idx + 2);\\n\\n  // Return the parsed message\\n  return {\\n    [header.msg_type]: {\\n      content: JSON.parse(content),\\n      header: JSON.parse(header),\\n      metadata: JSON.parse(metadata),\\n      parent: JSON.parse(parent),\\n    },\\n  };\\n}\\n\\nexport function hash(string: stringkey: string): string {\\n  const hmac = crypto.createHmac('sha256'key);\\n  hmac.update(string);\\n  return hmac.digest('hex');\\n}\\n\\nexport function json(data: any): string {\\n  return JSON.stringify(data).replace('\\\\ufdd0''\\\\\\\\ufdd0');\\n}\\n\\nexport function collapseMessage(key: stringmessage: any): string[] {\\n  const msgType = Object.keys(message)[0];\\n  const { headerparentmetadatacontent } = message[msgType];\\n  const newHeader = {\\n   ...parent,\\n    msg_id: uuid.v4(),\\n    msg_type: msgType,\\n  };\\n\\n  const toHash = [json(header)json(parent)json(metadata)json(content)];\\n  const hmac = hash(toHash.join('')key);\\n  return [...(parent.uuids || [])DELIMhmac,...toHash];\\n}\\n```\\n\\n**index.ts**\\n```typescript\\nimport { collapseMessageparseMessagehashjson } from './utils';\\n\\n// Export the utilities as a service\\nexport const utilsService = {\\n  parseMessage,\\n  collapseMessage,\\n  hash,\\n  json,\\n};\\n```\\n\\n**Usage**\\n```typescript\\nimport { utilsService } from './index';\\n\\n// Usage example\\nconst result = utilsService.parseMessage(\\n  'header',\\n  'parent',\\n 'metadata',\\n  'content',\\n  'delimiter'\\n);\\nconsole.log(result);\\n\\nconst collapsedMessage = utilsService.collapseMessage('key'{\\n  header: 'header',\\n  parent: 'parent',\\n  metadata:'metadata',\\n  content: 'content',\\n});\\nconsole.log(collapsedMessage);\\n```\\n\\nChanges made:\\n\\n* Converted the code to TypeScript and followed best practices for naming and organization.\\n* Extracted the utilities into separate functions to make the code more modular and reusable.\\n* Removed the `util` and `zmq` dependencies as they were not being used in the code.\\n* Added JSDoc comments to explain the purpose of each function and variable.\\n* Used the `any[]` type for the `args` parameter in the `parseMessage` function to match the original JavaScript code.\\n* Used the `string[]` type for the `toHash` array in the `collapseMessage` function to match the original JavaScript code.\\n* Removed the `module.exports` statement and replaced it with a service-based export using `index.ts`.\\n* Added a usage example to demonstrate  use the utilities service.\"",
                "\"react\": \"Below is the equivalent code in React TypeScript:\\n\\n```typescript\\n// Import required modules\\nimport { URL } from 'url';\\nimport * as fs from 'fs';\\nimport { findCache } from 'domain-crawler-tools';\\n\\n// Define the analyzeCache function\\nconst analyzeCache = async (url: string): Promise<{ result: CacheAnalysisResult; error: string }> => {\\n    // Find the cache file\\n    const cache = await findCache(url);\\n    if (!cache.length) {\\n        return { result: {}error: `No cache file found ${url}` };\\n    }\\n\\n    // Read the cache file\\n    const json = JSON.parse(fs.readFileSync(cache[0]'utf8'));\\n\\n    // Process the cache data\\n    const domains = [...new Set(json.map((s) => new URL(s.url).hostname))];\\n    const largeness = json.sort((ab) => b.content.length - a.content.length).slice(010);\\n    const urls = json.map((s) => s.url);\\n    const repeats = json.filter((siarr) => i > 0 && urls.indexOf(s.url) === i);\\n\\n    // Update the cache file\\n    fs.writeFileSync(cache[0]JSON.stringify(repeatsnull2));\\n\\n    // Return the analysis result\\n    return {\\n        result: {\\n            countPages: json.length,\\n            countCaches: cache.length,\\n            target: json[0].url,\\n            countDomains: domains.length,\\n            domains,\\n            countLargest: largeness.reduce((curl) => cur + l.content.length0),\\n            largest10: largeness.map((l) => l.url),\\n            repeats: json.length - repeats.length,\\n        },\\n    };\\n};\\n\\n// CacheAnalysisResult type definition\\ninterface CacheAnalysisResult {\\n    countPages: number;\\n    countCaches: number;\\n    target: string;\\n    countDomains: number;\\n    domains: string[];\\n    countLargest: number;\\n    largest10: string[];\\n    repeats: number;\\n}\\n\\n// Export the analyzeCache function\\nexport default analyzeCache;\\n\\n// Example usage\\n// import analyzeCache from './analyzeCache';\\n// analyzeCache('https://example.com')\\n//    .then((result) => console.log(result.result))\\n//    .catch((error) => console.error(error.error));\\n```\\n\\nNote that I've made the following changes:\\n\\n1.  Imported the required modules using ES6 import statements.\\n2.  Refactored the code to use async/await for better readability and error handling.\\n3.  Renamed the `analyzeCache` function to be more descriptive and consistent with TypeScript naming conventions.\\n4.  Defined a `CacheAnalysisResult` interface to represent the structure of the analysis result.\\n5.  Imported the necessary types from the `url` module using the `URL` type.\\n6.  Used the `Set` data structure to remove duplicate domains.\\n7.  Used the `utf8` encoding when reading the cache file to ensure correct parsing of the JSON data.\\n8.  Removed the `module.exports` statementas it's not necessary in a React TypeScript environment.\\n9.  Added an example usage section to demonstrate  use the `analyzeCache` function.\\n10. Added TODO comments as necessary to indicate areas where further improvement or refactoring could be beneficial.\"",
                "\"react\": \"**Core Importer Service (typescript-react)**\\n```typescript\\n// CoreImportService.ts\\nimport { Injectable } from '@nestjs/common';\\n\\n@Injectable()\\nexport class CoreImportService {\\n  // Import notebook\\n  async importNotebook(notebookId: string): Promise<any> {\\n    // TODO: Implement notebook import logic\\n    // For demonstration purposesreturn a placeholder value\\n    return { notebookIddata: 'Notebook data' };\\n  }\\n\\n  // Run all promises in an array\\n  private async runAllPromises(promises: Promise<any>[]): Promise<any[]> {\\n    const results = await Promise.allSettled(promises);\\n    return results.map((resultindex) => result.status === 'fulfilled' result.value : null);\\n  }\\n}\\n```\\n\\n**Core Importer Module (typescript-react)**\\n```typescript\\n// CoreImporterModule.ts\\nimport { Module } from '@nestjs/common';\\nimport { CoreImportService } from './CoreImportService';\\n\\n@Module({\\n  providers: [CoreImportService],\\n  exports: [CoreImportService],\\n})\\nexport class CoreImporterModule {}\\n```\\n\\n**Async Helper (typescript-react)**\\n```typescript\\n// AsyncHelper.ts\\nimport { Injectable } from '@nestjs/common';\\n\\n@Injectable()\\nexport class AsyncHelper {\\n  // Run all promises in an array\\n  async runAllPromises(promises: Promise<any>[]): Promise<any[]> {\\n    return promises.reduce(async (acccurrentPromise) => {\\n      const accValue = await acc;\\n      const currentValue = await currentPromise;\\n      return [...accValuecurrentValue];\\n    }[]);\\n  }\\n}\\n```\\n\\n**Component Example (typescript-react)**\\n```typescript\\n// CoreImporterComponent.ts\\nimport { ComponentOnInit } from '@nestjs/common';\\nimport { CoreImportService } from '../CoreImportService';\\nimport { AsyncHelper } from '../AsyncHelper';\\n\\n@Component({\\n  selector: 'app-core-importer',\\n  template: '<p>Importing notebooks...</p>',\\n})\\nexport class CoreImporterComponent implements OnInit {\\n  constructor(private readonly coreImportService: CoreImportServiceprivate readonly asyncHelper: AsyncHelper) {}\\n\\n  ngOnInit(): void {\\n    const notebookIds = ['notebook1''notebook2'];\\n    this.coreImportService.importNotebook(notebookIds[0]).then((result) => console.log(result));\\n    this.coreImportService.importNotebook(notebookIds[1]).then((result) => console.log(result));\\n\\n    // Using AsyncHelper to run all promises in an array\\n    this.asyncHelper.runAllPromises(\\n      notebookIds.map((notebookId) => this.coreImportService.importNotebook(notebookId)),\\n    ).then((results) => console.log(results));\\n  }\\n}\\n```\\n\\nIn this examplewe've created a `CoreImportService` that provides the `importNotebook` method for importing a notebook. We've also created an `AsyncHelper` service that provides a `runAllPromises` method for running all promises in an array.\\n\\nThe `CoreImporterComponent` example demonstrates  use the `CoreImportService` and `AsyncHelper` to import notebooks and run all promises in an array.\\n\\nNote that the `AsyncHelper` is not strictly necessaryas you could use `Promise.allSettled` directly in the component. Howeverit's included here for illustration purposes.\"",
                "\"react\": \"Here's the equivalent code in React TypeScript:\\n\\n Constants and Services\\n\\n```typescript\\n// constants.ts\\nexport const npmCmd = 'npm install --global --production --unsafe-perm nativescript windows-build-tools';\\n\\n// services/exec-service.ts\\nimport { spawn } from 'child_process';\\n\\ninterface ExecOptions {\\n  command: string;\\n  callback: (error: anystdout: stringstderr: string) => void;\\n}\\n\\nclass ExecService {\\n  execute(options: ExecOptions) {\\n    const { commandcallback } = options;\\n    const process = spawn(command{ shell: true });\\n\\n    process.stdout.on('data'(data) => {\\n      callback(nulldata.toString()'');\\n    });\\n\\n    process.stderr.on('data'(data) => {\\n      callback(new Error(data.toString())''data.toString());\\n    });\\n\\n    process.on('close'(code) => {\\n      if (code!== 0) {\\n        callback(new Error(`Command failed with code ${code}`)'''');\\n      }\\n    });\\n  }\\n}\\n\\nexport default ExecService;\\n```\\n\\n Component (for demo purposes)\\n\\n```typescript\\n// components/install-npm-tools.tsx\\nimport React{ useStateuseEffect } from'react';\\nimport ExecService from '../services/exec-service';\\nimport { npmCmd } from '../constants';\\n\\ninterface InstallNpmToolsProps {\\n  onInstallComplete: () => void;\\n}\\n\\nconst InstallNpmTools = ({ onInstallComplete }: InstallNpmToolsProps) => {\\n  const [installingsetInstalling] = useState(false);\\n  const [errorsetError] = useState(null);\\n\\n  useEffect(() => {\\n    const execService = new ExecService();\\n    const command = `powershell -c \\\"${npmCmd} ; ${getNetworkCmd()}\\\"`;\\n    const options: ExecOptions = {\\n      command,\\n      callback: (errorstdoutstderr) => {\\n        if (error) {\\n          setError(error.message);\\n        } else {\\n          onInstallComplete();\\n        }\\n        setInstalling(false);\\n      },\\n    };\\n\\n    execService.execute(options);\\n    setInstalling(true);\\n  }[onInstallComplete]);\\n\\n  return (\\n    <div>\\n      {installing (\\n        <p>Installing npm tools...</p>\\n      ) : error (\\n        <p>Error: {error}</p>\\n      ) : (\\n        <p>Npm tools installed.</p>\\n      )}\\n    </div>\\n  );\\n};\\n\\nexport default InstallNpmTools;\\n```\\n\\n Usage\\n\\n```typescript\\n// components/app.tsx\\nimport React from'react';\\nimport InstallNpmTools from './install-npm-tools';\\n\\nconst App = () => {\\n  const onInstallComplete = () => {\\n    // Handle install complete callback\\n  };\\n\\n  return (\\n    <div>\\n      <InstallNpmTools onInstallComplete={onInstallComplete} />\\n    </div>\\n  );\\n};\\n\\nexport default App;\\n```\\n\\n Notes\\n\\n* The `ExecService` class encapsulates the execution of shell commandsproviding a more structured approach to error handling and callback management.\\n* The `InstallNpmTools` component demonstrates  use the `ExecService` to install npm toolsdisplaying a progress indicator and handling errors.\\n* The `getNetworkCmd` function is assumed to be implemented elsewhereas it was not provided in the original code snippet.\\n* This implementation is designed to be functional and modularwith a clear separation of concerns between services and components.\"",
                "\"react\": \"Here's the refactored code in React TypeScript:\\n\\n```typescript\\nimport * as path from 'path';\\nimport { Core } from '../Core';\\nimport { Interface } from '../Core';\\nimport { IKernelJson } from './IKernelJson';\\n\\nconst jsonInterface: IKernelJson = {\\n  display_name: '',\\n  argv: [] as string[]// not optional\\n  language: ''// not optional\\n  metadata: [null{}] as [null{}]// optional\\n  env: [null{}] as [null{}]// optional\\n  interrupt_mode: [null''] as [nullstring]// optional\\n};\\n\\ninterface KernelJsonProps {\\n  kernel_json: IKernelJson;\\n}\\n\\ninterface WiresJsonProps {\\n  kernel_json: IKernelJson;\\n}\\n\\ninterface PathsJsonProps {\\n  kernel_json: IKernelJson;\\n}\\n\\nfunction wireJson(kernel_json: KernelJsonProps['kernel_json']) {\\n  return createInterface(kernel_json{\\n   ...jsonInterface,\\n    argv: (kernel_json.argv || []).includes('{connection_file}')\\n      (kernel_json.argv || []).concat(['{connection_file}'])\\n      : (kernel_json.argv || []),\\n  });\\n}\\n\\nfunction createInterface(kernel_json: IKernelJsonoverrides: Partial<IKernelJson>) {\\n  return Object.assign(interface(kernel_jsonjsonInterface)overrides);\\n}\\n\\nfunction pathJson(kernel_json: KernelJsonProps['kernel_json']) {\\n  const argv = (kernel_json.path [kernel_json.path] : []).concat(kernel_json.argv || kernel_json.args || []);\\n  return createInterface(\\n    {\\n     ...kernel_json,\\n      argv,\\n    },\\n    {\\n      argv,\\n    },\\n  );\\n}\\n\\ninterface JsonServiceProps {\\n  kernel_json: IKernelJson;\\n}\\n\\nclass JsonService {\\n  constructor(private kernel_json: IKernelJson) {}\\n\\n  getJsonInterface() {\\n    return jsonInterface;\\n  }\\n\\n  wireJson(): IKernelJson {\\n    return wireJson(this.kernel_json as IKernelJson);\\n  }\\n\\n  pathJson(): IKernelJson {\\n    return pathJson(this.kernel_json as IKernelJson);\\n  }\\n}\\n\\nexport { jsonInterfacewireJsonpathJsonJsonService };\\n```\\n\\nHere's what I changed:\\n\\n* I used TypeScript interfaces to define the shape of the `jsonInterface` and other types.\\n* I extracted the creation of the `jsonInterface` into its own variableand used a constant type to ensure it's not changed accidentally.\\n* I created new functions `createInterface` and `wireJson` to make the code more modular and reusable.\\n* I created a `JsonService` class that encapsulates the `jsonInterface` and provides methods to wire and path JSON.\\n* I used the `Partial` type to allow overriding properties of the `jsonInterface`.\\n* I used the spread operator (`{... }`) to create a new object with some properties overwritten.\\n* I removed the `TODO` comments and replaced them with a TODO item in the TypeScript docstring.\\n* I used the `Object.assign` function to create a new object with some properties overwrittenbut I also used the spread operator to avoid using `Object.assign` directly.\\n* I added type annotations to the function parameters and return types to make the code more self-documenting.\\n\\nNote that I didn't use the `yargs` library to parse the stringas it's not clear  use it in this context. If you need to parse the stringyou may need to add additional dependencies or use a different library.\"",
                "\"react\": \"```typescript\\n// Import necessary modules\\nimport * as fs from 'fs';\\nimport * as path from 'path';\\nimport * as json5 from 'json5';\\n\\n// Define a service to extract cells from a Jupyter Notebook\\nclass JupyterNotebookService {\\n  private notebookPath: string;\\n\\n  constructor(notebookPath: string) {\\n    this.notebookPath = notebookPath;\\n  }\\n\\n  // Extract cells from a Jupyter Notebook with additional metadata\\n  public async getCells(types: string[] = ['*''code']): Promise<Cell[]> {\\n    const notebookContent = await this.getNotebookContent();\\n\\n    // Parse the notebook content as JSON\\n    const notebook = json5.parse(notebookContent);\\n\\n    // Get the kernel information\\n    const kernel = notebook.metadata.kernelspec || {};\\n\\n    // Get the cells from the notebook\\n    const cells = notebook.cells.map((cellindex) => {\\n      // Add additional metadata to each cell\\n      return {\\n       ...cell,\\n        language: this.getLanguage(cell.metadatakernelnotebook.metadata),\\n        filename: this.notebookPath,\\n        id: `${path.basename(this.notebookPath)}[${index}]`\\n      };\\n    }).filter((cell) => '*' === types[0] || types.includes(cell.cell_type)) || [];\\n\\n    return cells;\\n  }\\n\\n  // Get the content of the notebook\\n  private async getNotebookContent(): Promise<string> {\\n    return new Promise((resolve) => {\\n      fs.readFile(this.notebookPath'utf8'(errdata) => {\\n        if (err) {\\n          throw err;\\n        }\\n        resolve(data);\\n      });\\n    });\\n  }\\n\\n  // Get the language of a cell\\n  private getLanguage(cellMetadatakernellanguageInfo): string {\\n    return cellMetadata.vscode.languageId || kernel.language || (languageInfo.language_info.name || '');\\n  }\\n}\\n\\ninterface Cell {\\n  cell_type: string;\\n  metadata: {\\n    vscode: {\\n      languageId: string;\\n    };\\n  };\\n  language: string;\\n  filename: string;\\n  id: string;\\n}\\n\\n// Example usage:\\nclass App {\\n  private service: JupyterNotebookService;\\n\\n  constructor(notebookPath: string) {\\n    this.service = new JupyterNotebookService(notebookPath);\\n  }\\n\\n  public async componentDidMount(): Promise<void> {\\n    const cells = await this.service.getCells();\\n    console.log(cells);\\n  }\\n}\\n\\n// Export the service\\nexport { JupyterNotebookService };\\n```\\n\\nI made the following changes:\\n\\n1. Imported the necessary modules.\\n2. Created a service `JupyterNotebookService` with a constructor that takes the notebook path as an argument.\\n3. Defined a method `getCells` that extracts cells from a Jupyter Notebook with additional metadata.\\n4. Used the `fs` module to read the notebook content.\\n5. Used the `json5` module to parse the notebook content as JSON.\\n6. Added additional metadata to each cellincluding the languagefilenameand id.\\n7. Filtered the cells based on the types provided.\\n8. Created a `Cell` interface to represent a cell in the notebook.\\n9. Created an example `App` class that demonstrates  use the service.\\n10. Exported the `JupyterNotebookService` class.\\n\\nNote that this implementation uses TypeScript and imports the necessary modules. I also added comments to explain the purpose of each method and variable.\"",
                "\"react\": \" LlamaFunctionsService.ts\\n\\n```typescript\\nimport { createLlmsession } from './createLlmsession';\\n\\ninterface LlamaFunctionsServiceOptions {\\n  categories: boolean;\\n}\\n\\nclass LlamaFunctionsService {\\n  private llmPrompt: (prompt: string) => Promise<string>;\\n\\n  constructor(llmPrompt: (prompt: string) => Promise<string>) {\\n    this.llmPrompt = llmPrompt;\\n  }\\n\\n  async askLlamaAboutFunctions(\\n    query: string,\\n    functions: string[],\\n    descriptions: string[],\\n    options: LlamaFunctionsServiceOptions = { categories: false }\\n  ): Promise<string> {\\n    const prompt = this.createPrompt(functionsdescriptionsoptions.categories);\\n    console.log(`User: ${prompt}`);\\n    const response = await this.llmPrompt(prompt);\\n    console.log(`AI: ${response}`);\\n    return this.parseResponse(responsefunctions);\\n  }\\n\\n  private createPrompt(functions: string[]descriptions: string[]categories: boolean): string {\\n    let prompt = `Given a list of ${categories 'categories' : 'functions'}:\\\\n`;\\n    for (let i = 0; i < functions.length; i++) {\\n      if (descriptions[i]) {\\n        prompt += `${functions[i]} - ${descriptions[i]}\\\\n`;\\n      } else {\\n        prompt += `${functions[i]}\\\\n`;\\n      }\\n    }\\n    prompt += `which one most closely matches the query \\\"${query}\\\"`;\\n    return prompt;\\n  }\\n\\n  private parseResponse(response: stringfunctions: string[]): string {\\n    const trimmedResponse = response.trim();\\n    const splitResponse = trimmedResponse.split(/[\\\"`*'\\\\n]/gi);\\n    return splitResponse.find((value) => functions.includes(value)) || '';\\n  }\\n}\\n\\nexport { LlamaFunctionsService };\\n```\\n\\n createLlmsession.ts\\n\\n```typescript\\nimport { createRequire } from'module';\\nconst require = createRequire(import.meta.url);\\n\\ninterface CreateLlmsessionOptions {\\n  // options for creating the llm session\\n}\\n\\nclass CreateLlmsession {\\n  public async import(dependency: string): Promise<any> {\\n    return require(dependency);\\n  }\\n}\\n\\nexport { CreateLlmsession };\\n```\\n\\n App.tsx\\n\\n```typescript\\nimport React from'react';\\nimport { LlamaFunctionsService } from './LlamaFunctionsService';\\n\\nconst App = () => {\\n  const llmPrompt = async (prompt: string) => {\\n    // implement your llm prompt logic here\\n    return'some response';\\n  };\\n\\n  const service = new LlamaFunctionsService(llmPrompt);\\n\\n  const query ='some query';\\n  const functions = ['function1''function2''function3'];\\n  const descriptions = ['description1''description2''description3'];\\n\\n  service\\n   .askLlamaAboutFunctions(queryfunctionsdescriptions)\\n   .then((result) => console.log(`Result: ${result}`));\\n  return <div>App</div>;\\n};\\n\\nexport default App;\\n```\\n\\nIn this refactored code:\\n\\n*   We created a new service class `LlamaFunctionsService` that encapsulates the logic for asking Llama about functions.\\n*   The `askLlamaAboutFunctions` method in the service class takes in the queryfunctionsdescriptionsand an optional categories flag.\\n*   We extracted the prompt creation logic into a separate method `createPrompt` to make it reusable.\\n*   We also extracted the response parsing logic into a separate method `parseResponse` to make it reusable.\\n*   We created a `CreateLlmsession` class to manage the creation of the LLM session.\\n*   We implemented the `App` component to demonstrate  use the `LlamaFunctionsService`.\"",
                "\"react\": \"Here's the equivalent React TypeScript code for the given Git commands.\\n\\n Git Log Viewer Component\\n\\n```typescript\\nimport React{ useStateuseEffect } from'react';\\nimport { exec } from 'child_process';\\n\\ninterface GitLogProps {\\n  graph: boolean;\\n  oneline: boolean;\\n  decorate: boolean;\\n  all: boolean;\\n}\\n\\nconst GitLogViewer: React.FC<GitLogProps> = ({\\n  graph = true,\\n  oneline = true,\\n  decorate = true,\\n  all = true,\\n}) => {\\n  const [gitLogsetGitLog] = useState('');\\n  const [errorsetError] = useState('');\\n\\n  useEffect(() => {\\n    const runGitLog = async () => {\\n      try {\\n        const command = `git log --graph --oneline --decorate --all`;\\n        if (!all) {\\n          command = `git log --graph --all`;\\n        }\\n        if (!oneline && all) {\\n          command = `git log --graph --decorate --all`;\\n        }\\n        const output = await exec(command);\\n        setGitLog(output.stdout);\\n      } catch (error) {\\n        setError(`Error running Git command: ${error.message}`);\\n      }\\n    };\\n    runGitLog();\\n  }[alloneline]);\\n\\n  return (\\n    <div>\\n      <h2>Git Log</h2>\\n      {error (\\n        <p style={{ color:'red' }}>{error}</p>\\n      ) : (\\n        <pre>{gitLog}</pre>\\n      )}\\n    </div>\\n  );\\n};\\n\\nexport default GitLogViewer;\\n```\\n\\n Git Log Viewer Service\\n\\n```typescript\\nimport { spawn } from 'child_process';\\n\\ninterface GitLogService {\\n  getGitLog: (graph: booleanoneline: booleandecorate: booleanall: boolean) => Promise<string>;\\n}\\n\\nconst gitLogService: GitLogService = {\\n  async getGitLog(graph = trueoneline = truedecorate = trueall = true) {\\n    try {\\n      const command = `git log --graph --oneline --decorate --all`;\\n      if (!all) {\\n        command = `git log --graph --all`;\\n      }\\n      if (!oneline && all) {\\n        command = `git log --graph --decorate --all`;\\n      }\\n      const childProcess = spawn(command{ shell: true });\\n      const output = await new Promise((resolvereject) => {\\n        let log = '';\\n        childProcess.stdout.on('data'(data) => {\\n          log += data.toString();\\n        });\\n        childProcess.stderr.on('data'(error) => {\\n          reject(error.toString());\\n        });\\n        childProcess.on('close'(code) => {\\n          if (code === 0) {\\n            resolve(log);\\n          } else {\\n            reject('Error running Git command');\\n          }\\n        });\\n      });\\n      return output;\\n    } catch (error) {\\n      throw new Error(`Error running Git command: ${error.message}`);\\n    }\\n  },\\n};\\n\\nexport default gitLogService;\\n```\\n\\n Usage Example\\n\\n```typescript\\nimport React from'react';\\nimport GitLogViewer from './GitLogViewer';\\nimport { gitLogService } from './GitLogService';\\n\\nconst App = () => {\\n  const getGitLog = async () => {\\n    try {\\n      const log = await gitLogService.getGitLog(truetruetruetrue);\\n      console.log(log);\\n    } catch (error) {\\n      console.error(error);\\n    }\\n  };\\n\\n  return (\\n    <div>\\n      <button onClick={getGitLog}>Get Git Log</button>\\n      <GitLogViewer />\\n    </div>\\n  );\\n};\\n\\nexport default App;\\n```\\n\\nIn these examplesI've created a React component `GitLogViewer` and a service `gitLogService` to handle the Git log retrieval. The component uses the `useState` and `useEffect` hooks to fetch the Git log on mount and update the component state accordingly. The service uses the `child_process` module to spawn a new process for each Git command and captures the output and error streams. The usage example demonstrates  use the service to fetch the Git log and display it in the component.\"",
                "\"react\": \"**maricopa-alert.service.ts**\\n```typescript\\nimport { Injectable } from '@nestjs/common';\\nimport * as fs from 'fs';\\nimport * as path from 'path';\\nimport axios from 'axios';\\n\\n@Injectable()\\nexport class MaricopaAlertService {\\n  private PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';\\n  private project = path.join(this.PROFILE_PATH'Collections/crimes');\\n\\n  async scrapeAlert(ID: string): Promise<any> {\\n    try {\\n      const filePath = path.join(this.project,'maricopa-alert-' + ID + '.json');\\n      if (fs.existsSync(filePath)) {\\n        return;\\n      }\\n      const response = await axios.get(`https://www.maricopacountyattorney.org/CivicAlerts.aspxAID=${ID}`);\\n      const $ = await this.getXPath(response.data);\\n      const result = {\\n        time: $.time.text(),\\n        title: $.title.text(),\\n        content: $.content.text(),\\n      };\\n      fs.writeFileSync(filePathJSON.stringify(resultnull4));\\n      return result;\\n    } catch (error) {\\n      console.error(error);\\n      return error;\\n    }\\n  }\\n\\n  private async getXPath(html: string) {\\n    // TO DO: Implement a better way to parse HTML (e.g.using a dedicated HTML parser library)\\n    const $ = cheerio.load(html);\\n    return $;\\n  }\\n}\\n```\\n\\n**multi-crawl.service.ts**\\n```typescript\\nimport { Injectable } from '@nestjs/common';\\nimport { MaricopaAlertService } from './maricopa-alert.service';\\n\\n@Injectable()\\nexport class MultiCrawlService {\\n  constructor(private readonly maricopaAlertService: MaricopaAlertService) {}\\n\\n  async multiCrawl(IDs: string[]name: string): Promise<any> {\\n    const results = await Promise.all(IDs.map((ID) => this.maricopaAlertService.scrapeAlert(ID)));\\n    return results;\\n  }\\n}\\n```\\n\\n**app.module.ts**\\n```typescript\\nimport { Module } from '@nestjs/common';\\nimport { MaricopaAlertService } from './maricopa-alert.service';\\nimport { MultiCrawlService } from './multi-crawl.service';\\n\\n@Module({\\n  providers: [MaricopaAlertServiceMultiCrawlService],\\n})\\nexport class AppModule {}\\n```\\n\\n**async.component.ts**\\n```typescript\\nimport { ComponentOnInit } from '@nestjs/application';\\nimport { MultiCrawlService } from './multi-crawl.service';\\n\\n@Component()\\nexport class AsyncComponent implements OnInit {\\n  constructor(private readonly multiCrawlService: MultiCrawlService) {}\\n\\n  ngOnInit(): void {\\n    const IDs = Array.from(Array(500).keys());\\n    this.multiCrawlService\\n     .multiCrawl(IDs'crime reports')\\n     .then((results) => console.log(results))\\n     .catch((error) => console.error(error));\\n  }\\n}\\n```\\n\\nThis refactored code uses the `@nestjs/core` module to create a service and a component. The `MaricopaAlertService` is responsible for scraping the Maricopa county alert datawhile the `MultiCrawlService` is responsible for running multiple scrapes concurrently. The `AsyncComponent` is an example of  use these services in an Angular component. \\n\\nThe `getXPath` method in the `MaricopaAlertService` is a temporary solution and should be replaced with a more robust HTML parsing library (e.g.Cheerio). \\n\\nNote that this code uses the `axios` library to make HTTP requests and the `fs` library to interact with the file system. You may need to install these dependencies in your project using npm or yarn. \\n\\nAlsoyou may need to adjust the file paths and the `PROFILE_PATH` variable to match your project's structure and environment. \\n\\nFinallyyou can use this code as a starting point and customize it to fit your specific needs. Don't forget to add TODO comments where you need to implement additional functionality or features.\"",
                "\"react\": \" SQLite Database Service in React TypeScript\\n\\nBelow is the refactored code converting the original Node.js code to a React TypeScript service.\\n\\n sqlite.service.ts\\n\\n```typescript\\nimport * as sqlite3 from 'better-sqlite3';\\nimport { readFile } from 'fs/promises';\\nimport { join } from 'path';\\n\\ninterface SQLiteService {\\n  executeSql(sauce: stringcmds: string): Promise<any>;\\n}\\n\\nclass SQLiteService implements SQLiteService {\\n  private db: sqlite3.Database;\\n\\n  executeSql(sauce: stringcmds: string): Promise<any> {\\n    return new Promise((resolvereject) => {\\n      this.db = new sqlite3(sauce{});\\n      this.db.exec(cmdsfunction(errresults) {\\n        if (err) {\\n          reject(err);\\n        } else {\\n          resolve(results);\\n        }\\n      });\\n    });\\n  }\\n\\n  async closeDb(): Promise<void> {\\n    if (this.db) {\\n      await this.db.close();\\n    }\\n  }\\n}\\n\\nexport default new SQLiteService();\\n```\\n\\n sqlite.factory.ts\\n\\n```typescript\\nimport { SQLiteService } from './sqlite.service';\\n\\ninterface DatabaseOptions {\\n  sauce: string;\\n}\\n\\ninterface DatabaseCommands {\\n  cmds: string;\\n}\\n\\nclass SQLiteFactory {\\n  private dbService: SQLiteService;\\n\\n  constructor(dbService: SQLiteService) {\\n    this.dbService = dbService;\\n  }\\n\\n  loadDatabaseConfig(sauce: string): Promise<DatabaseCommands> {\\n    const filePath = join(__dirnamesauce);\\n    return readFile(filePath'utf8')\\n     .then((commands: string) => ({ commands }))\\n     .catch((error: Error) => {\\n        console.error('Error loading database config:'error);\\n        return null;\\n      });\\n  }\\n\\n  async executeSql(sauce: stringcmds: string): Promise<any> {\\n    return this.dbService.executeSql(saucecmds);\\n  }\\n}\\n\\nexport default SQLiteFactory;\\n```\\n\\n app.service.ts\\n\\n```typescript\\nimport { SQLiteFactory } from './sqlite.factory';\\n\\ninterface DatabaseExecution {\\n  sauce: string;\\n  cmds: string;\\n}\\n\\nclass AppService {\\n  private dbFactory: SQLiteFactory;\\n\\n  constructor(dbFactory: SQLiteFactory) {\\n    this.dbFactory = dbFactory;\\n  }\\n\\n  async executeDatabaseCommands(execution: DatabaseExecution): Promise<any> {\\n    const { saucecmds } = execution;\\n    return this.dbFactory.executeSql(saucecmds);\\n  }\\n}\\n\\nexport default new AppService(new SQLiteFactory(new SQLiteService()));\\n```\\n\\n usage.ts\\n\\n```typescript\\nimport { AppService } from './app.service';\\n\\ninterface DatabaseConfig {\\n  sauce: string;\\n}\\n\\nconst config: DatabaseConfig = {\\n  sauce: '/Users/briancullinan/Documents/studysauce3/db/studysauce.db',\\n};\\n\\nconst appService = new AppService();\\nconst dbConfig = {\\n  sauce: '/Users/briancullinan/Documents/studysauce3/db/studysauce.sqlite',\\n};\\n\\nappService.executeDatabaseCommands(dbConfig).then((results) => {\\n  console.log('Database executed successfully:'results);\\n});\\n\\n// You can also load your database config from a file\\n// appService.executeDatabaseCommands(config).then((results) => {\\n//   console.log('Database executed successfully:'results);\\n// });\\n```\\n\\n Note:\\n\\n* The original `fs` module is replaced with `fs/promises` to use promises for file I/O operations.\\n* The `path` module is used for joining file paths.\\n* The `sqlite3` library is used as a dependency in the `sqlite.service.ts` file.\\n* The `SQLiteFactory` and `AppService` classes are introduced to encapsulate the loading of the database config and execution of SQL commands.\\n* The `executeSql` method in the `SQLiteService` class is modified to return a promise.\\n* The `closeDb` method is added to close the database connection.\\n* The `usage.ts` file demonstrates  use the `AppService` to execute database commands.\"",
                "\"react\": \"Below is the equivalent code in React TypeScript:\\n\\n```typescript\\n// Interface for LLMSession\\ninterface LLMSession {\\n  llmPrompt: (prompt: string) => Promise<string>;\\n}\\n\\n// Interface for LLM functions\\ninterface LLMFunction {\\n  (query: stringlanguage: string): Promise<string>;\\n}\\n\\n// Interface for CodeConversionOptions\\ninterface CodeConversionOptions {\\n  language: string;\\n  intention: 'component' |'service';\\n}\\n\\n// Interface for CodeConversionResult\\ninterface CodeConversionResult {\\n  code: string;\\n}\\n\\nclass LLMService {\\n  private llmSession: LLMSession;\\n\\n  constructor(llmSession: LLMSession) {\\n    this.llmSession = llmSession;\\n  }\\n\\n  async askLlamaToSummerize(query: string): Promise<string> {\\n    const prompt = `Summerize this into one or two sentences:\\\\n${query}\\\\nDiscard any pleasantries or explanationsdon't include the words \\\"here is a two-sentence summary\\\"only respond with the summary.`;\\n    const response = await this.llmSession.llmPrompt(prompt);\\n    return response.trim();\\n  }\\n\\n  async askLlamaToGeneralize(query: string): Promise<string> {\\n    const prompt = `How would you categorize this in two or three words:\\\\n${query}\\\\nReturn only the category.`;\\n    const response = await this.llmSession.llmPrompt(prompt);\\n    return response.trim();\\n  }\\n\\n  async askLlamaToImplement(query: stringlanguage: string): Promise<string> {\\n    const prompt = `Improve this code in every way you cankeeping the same basic inputsoutputsand reasoning ${language `in ${language}` : ''}:\\\\n${query}\\\\nAddrefactorremoveimplement TODO comments if you can. Try to make the code as functional as possiblearound 10 to 20 lines of code per function is good. Only return the new beautiful and amazing code and nothing elseno markdown or explanations.`;\\n    const response = await this.llmSession.llmPrompt(prompt);\\n    return response.trim();\\n  }\\n\\n  async askLlamaToReact(query: stringoptions: CodeConversionOptions = { intention: 'component' }): Promise<CodeConversionResult> {\\n    const prompt = `Convert this code in every way you cankeeping the same basic inputsoutputsand reasoning ${options.language `to ${options.language}` : ''}:\\\\n${query}\\\\nAddrefactorremoveimplement TODO comments if you can. Try to make the code as functional as possiblearound 10 to 20 lines of code per function is good. Only return a new ${options.intention} dependending on the intention.`;\\n    const response = await this.llmSession.llmPrompt(prompt);\\n    return {\\n      code: response.trim(),\\n    };\\n  }\\n}\\n\\n// Usage\\nimport { LLMService } from './llm-service';\\nimport { createLLMSession } from 'create-llm-session';\\n\\nconst llmSession = createLLMSession();\\nconst llmService = new LLMService(llmSession);\\n\\nllmService.askLlamaToSummerize('query')\\n .then((response) => console.log(response))\\n .catch((error) => console.error(error));\\n\\nllmService.askLlamaToGeneralize('query')\\n .then((response) => console.log(response))\\n .catch((error) => console.error(error));\\n\\nllmService.askLlamaToImplement('query''typescript')\\n .then((response) => console.log(response))\\n .catch((error) => console.error(error));\\n\\nllmService.askLlamaToReact('query'{ intention:'service' })\\n .then((response) => console.log(response.code))\\n .catch((error) => console.error(error));\\n```\\n\\nNote that I've created a separate LLMService class to encapsulate the LLMSession and the LLM functions. I've also added interfaces for LLMSessionLLM functionsCodeConversionOptionsand CodeConversionResult to define the structure of the data.\\n\\nThe `askLlamaToReact` function now takes an options object with an intention propertywhich determines whether the response should be a new component or service. The response is now returned as a CodeConversionResult object with a code property.\\n\\nThe usage section shows  create an instance of the LLMService class and use the LLM functions to generate responses.\"",
                "\"react\": \"Below is the refactored code in React TypeScript.\\n\\n```typescript\\n// questionExtractor.ts\\nimport { RegExp } from'regex';\\n\\nconst QUESTION_REGEX = new RegExp(r'^.*\\\\.*$'RegExp.IGNORECASE | RegExp.MULTILINE);\\n\\ninterface Question {\\n  question: string;\\n  source: string;\\n}\\n\\nclass QuestionExtractor {\\n  extractQuestions(markdown: stringsource: string): Question[] {\\n    const matchQuestions = QUESTION_REGEX.exec(markdown) || [];\\n    const questionsFromMarkdown = matchQuestions.map((match) =>\\n      match.replace(r'|\\\\|+'''matchRegExp.IGNORECASE).trim()\\n    );\\n\\n    const matchQuestionsFromSource = QUESTION_REGEX.exec(source) || [];\\n    const questionsFromSource = matchQuestionsFromSource\\n     .map((match) =>\\n        match.replace(r'|\\\\|+'''matchRegExp.IGNORECASE).trim()\\n      )\\n     .filter((question) => question.toLowerCase().includes(''));\\n\\n    return [...questionsFromMarkdown,...questionsFromSource].sort((ab) => a.length - b.length);\\n  }\\n}\\n\\nexport const questionExtractor = new QuestionExtractor();\\n```\\n\\n```typescript\\n// markdownAccumulator.ts\\ninterface Cell {\\n  id: string;\\n  code: string;\\n  language: string;\\n  markdown: string;\\n  source: string;\\n}\\n\\nclass MarkdownAccumulator {\\n  accumulateMarkdown(cells: Cell[]): { id: string; language: string; from: number; to: number; markdown: string; code: string }[] {\\n    const codeCells = cells.filter((cell) => cell.code);\\n    return codeCells.reduce((acccodeCellindex) => {\\n      const fromIndex = cells.findIndex((cell) => cell.id === codeCells[index - 1].id) + 1 || 0;\\n      const markdown = cells.slice(fromIndexindex + 1).map((cell) => cell.source).join('\\\\n');\\n      return [...acc{ id: codeCell.idlanguage: codeCell.languagefrom: fromIndexto: index + 1markdowncode: codeCell.code }];\\n    }[]);\\n  }\\n}\\n\\nexport const markdownAccumulator = new MarkdownAccumulator();\\n```\\n\\n```typescript\\n// cacheService.ts\\nimport { questionExtractor } from './questionExtractor';\\nimport { markdownAccumulator } from './markdownAccumulator';\\nimport * as fs from 'fs';\\nimport * as path from 'path';\\nimport * as Core from 'Core';\\n\\ninterface CachedEntry {\\n  id: string;\\n  filename: string;\\n  mtime: Date;\\n  questions: string[];\\n  notebook: string;\\n  language: string;\\n  from: number;\\n  to: number;\\n}\\n\\nclass CacheService {\\n  async cacheCells(filename: string): Promise<CachedEntry[]> {\\n    const cells = await Core.getCells(filename);\\n    const newCache = markdownAccumulator.accumulateMarkdown(cells);\\n    const cachedEntries = newCache.map((entry) => ({\\n      id: entry.id,\\n      filename: filename,\\n      mtime: new Date(fs.statSync(filename).mtimeMs * 1000),\\n      questions: questionExtractor.extractQuestions(entry.markdownentry.code),\\n      notebook: path.basename(filename),\\n      language: entry.language,\\n      from: entry.from,\\n      to: entry.to,\\n     ...entry,\\n    }));\\n    return cachedEntries;\\n  }\\n}\\n\\nexport const cacheService = new CacheService();\\n```\\n\\n```typescript\\n// main.tsx\\nimport React{ useStateuseEffect } from'react';\\nimport { cacheService } from './cacheService';\\n\\ninterface Notebook {\\n  id: string;\\n  filename: string;\\n  mtime: Date;\\n  questions: string[];\\n  notebook: string;\\n  language: string;\\n  from: number;\\n  to: number;\\n}\\n\\nconst NotebookList: React.FC = () => {\\n  const [notebookssetNotebooks] = useState<Notebook[]>([]);\\n\\n  useEffect(() => {\\n    cacheService.cacheCells('notebook.ipynb').then((notebooks) => setNotebooks(notebooks));\\n  }[]);\\n\\n  return (\\n    <ul>\\n      {notebooks.map((notebook) => (\\n        <li key={notebook.id}>\\n          <p>Filename: {notebook.filename}</p>\\n          <p>Mtime: {notebook.mtime.toISOString()}</p>\\n          <p>Questions: {notebook.questions.join('')}</p>\\n          <p>Notebook: {notebook.notebook}</p>\\n          <p>Language: {notebook.language}</p>\\n          <p>From: {notebook.from}</p>\\n          <p>To: {notebook.to}</p>\\n        </li>\\n      ))}\\n    </ul>\\n  );\\n};\\n\\nexport default NotebookList;\\n```\"",
                "\"amazing\": \" Llama - Large Language Model Assistant\\n Documentation\\n Functions\\n cache_cells\\nExtracts relevant cells from a Jupyter notebookgenerates cache entriesand returns a list of dictionaries containing cache information.\\n\\n accumulate_markdown\\nAccumulates markdown leading up to code cells in a Jupyter notebook.\\n\\n get_questions\\nExtracts questions from markdown and source code.\\n\\n Implementation\\n```python\\nimport os\\nimport json\\nimport re\\nimport datetime\\nfrom typing import ListDict\\n\\nfrom Core import get_cells\\n\\ndef get_questions(source: strmarkdown: str) -> List[str]:\\n    \\\"\\\"\\\"Extracts questions from markdown and source code.\\n\\n    Args:\\n        source: Source code from which questions are to be extracted.\\n        markdown: Markdown text from which questions are to be extracted.\\n\\n    Returns:\\n        list: A list of extracted questions.\\n    \\\"\\\"\\\"\\n     Regular expression pattern to match questions\\n    match_questions = re.compile(r'^.*\\\\.*$'re.IGNORECASE | re.MULTILINE)\\n    \\n     Extract questions from markdown and source code\\n    questions = [\\n         Remove prefix and suffix from questions using regular expression\\n        re.sub(r'|\\\\|+'''qflags=re.IGNORECASE).strip()\\n        for q in match_questions.findall(markdown)\\n    ]\\n    \\n     Append questions from source code if '' is present\\n    questions += [\\n        re.sub(r'|\\\\|+'''qflags=re.IGNORECASE).strip()\\n        for q in match_questions.findall(source) if '' in q.lower()\\n    ]\\n    \\n     Sort questions in ascending order of length\\n    questions.sort(key=len)\\n    \\n     Return first question if none or append it to the list\\n    return questions + [questions[0]] if questions else ['']\\n\\ndef accumulate_markdown(cells: List[Dict]notebook_path: str) -> List[Dict]:\\n    \\\"\\\"\\\"Accumulates markdown leading up to code cells.\\n\\n    Args:\\n        cells: List of dictionaries containing cell information.\\n        notebook_path: Path to the Jupyter notebook.\\n\\n    Returns:\\n        list: A list of dictionaries containing accumulated markdown and code information.\\n    \\\"\\\"\\\"\\n     Filter code cells\\n    codes = [c for c in cells if c[\\\"cell_type\\\"] == \\\"code\\\"]\\n    \\n    result = []\\n    \\n     Iterate over code cells\\n    for icode_cell in enumerate(codes):\\n         Calculate index of previous code cell\\n        from_idx = cells.index(codes[i-1]) + 1 if i > 0 else 0\\n        \\n         Calculate index of current code cell\\n        to_idx = cells.index(code_cell)\\n        \\n         Accumulate markdown\\n        markdown = \\\"\\\\n\\\".join(\\\"\\\".join(m[\\\"source\\\"]) for m in cells[from_idx:to_idx])\\n        \\n         Accumulate code\\n        code = \\\"\\\".join(code_cell[\\\"source\\\"])\\n        \\n         Append dictionary to result\\n        result.append({\\n            \\\"id\\\": f\\\"{os.path.basename(notebook_path)}[{i}]\\\"\\n            \\\"language\\\": code_cell[\\\"language\\\"]\\n            \\\"from\\\": from_idx\\n            \\\"to\\\": to_idx\\n            \\\"markdown\\\": markdown\\n            \\\"code\\\": code\\n        })\\n    \\n    return result\\n\\ndef cache_cells(filename: str) -> List[Dict]:\\n    \\\"\\\"\\\"Parses a Jupyter notebookextracts relevant cellsand generates cache entries.\\n\\n    Args:\\n        filename: Path to the Jupyter notebook.\\n\\n    Returns:\\n        list: A list of dictionaries containing cache information.\\n    \\\"\\\"\\\"\\n    from Core import get_cells\\n    filename = os.path.abspath(filename)\\n    mtime = os.path.getmtime(filename)\\n    \\n    cells = get_cells(filename)\\n    new_cache = accumulate_markdown(cellsfilename)\\n    \\n    return [\\n        {\\n            \\\"id\\\": c[\\\"id\\\"],\\n            \\\"filename\\\": filename,\\n            \\\"mtime\\\": datetime.datetime.fromtimestamp(mtime),\\n            \\\"questions\\\": get_questions(c[\\\"code\\\"]c[\\\"markdown\\\"]),\\n            \\\"notebook\\\": os.path.basename(filename),\\n            **c\\n        }\\n        for c in new_cache\\n    ]\\n\\n__all__ = {\\n    \\\"cache_cells\\\": cache_cells,\\n    \\\"accumulate_markdown\\\": accumulate_markdown,\\n    \\\"get_questions\\\": get_questions\\n}\",",
                "\"react\": \"Here is the refactored code in React TypeScript:\\n```typescript\\n// LlamaFunctions.ts\\nimport { importer } from './importer';\\nimport { storeLlamaFunction } from './store-llama-function';\\nimport { askLlamaAboutFunctions } from './ask-llama-about-functions';\\nimport { askLlamaToGeneralizeAll } from './ask-llama-to-generalize-all';\\n\\ninterface FunctionCache {\\n  [key: string]: {\\n    categories: string[];\\n    exports: any;\\n    description: string;\\n    summary: string;\\n    mtime: number;\\n  };\\n}\\n\\nconst functionCache: FunctionCache = importer.import('cache rpc functions with llm descriptions');\\n\\ninterface Categories {\\n  categories: string[];\\n  update: boolean;\\n}\\n\\nconst askLlamaGeneralizeCategories = async (categories: Categories['categories']): Promise<string[]> => {\\n  if (!categories) {\\n    const keys = Object.keys(functionCache);\\n    categories = keys.map((k) => functionCache[k].categories)\\n     .filter((aiarr) => a && arr.indexOf(a) == i &&!a.includes('\\\\n'));\\n  }\\n\\n  if (categories.length == 0) {\\n    return [];\\n  }\\n\\n  const uniqueValues = await askLlamaToGeneralizeAll(categories);\\n  console.log(uniqueValues);\\n\\n  if (uniqueValues.length > 100) {\\n    return askLlamaGeneralizeCategories(uniqueValues).then((categories) => categories.filter((aiarr) => arr.indexOf(a) == i));\\n  }\\n\\n  return uniqueValues;\\n};\\n\\nexport { askLlamaGeneralizeCategories };\\n```\\n\\n```typescript\\n// AskLlamaToGeneralizeAll.ts\\nimport { importer } from './importer';\\n\\nconst askLlamaToGeneralizeAll = async (categories: string[]): Promise<string[]> => {\\n  // TODO: Implement askLlamaToGeneralizeAll function\\n  throw new Error('Not implemented');\\n};\\n\\nexport { askLlamaToGeneralizeAll };\\n```\\n\\n```typescript\\n// AskLlamaAboutFunctions.ts\\nimport { importer } from './importer';\\n\\nconst askLlamaAboutFunctions = async (categories: string[]uniqueValues: string[]args: any[]update: boolean): Promise<string> => {\\n  // TODO: Implement askLlamaAboutFunctions function\\n  throw new Error('Not implemented');\\n};\\n\\nexport { askLlamaAboutFunctions };\\n```\\n\\n```typescript\\n// StoreLlamaFunction.ts\\nimport { importer } from './importer';\\n\\nconst storeLlamaFunction = async (key: stringmtime: numberexports: anydescription: stringsummary: stringcategories: string[]newCategory: string) => {\\n  // TODO: Implement storeLlamaFunction function\\n  throw new Error('Not implemented');\\n};\\n\\nexport { storeLlamaFunction };\\n```\\n\\n```typescript\\n// importer.ts\\nclass Importer {\\n  private cache: { [key: string]: any };\\n\\n  public import(moduleName: string): any {\\n    if (!this.cache[moduleName]) {\\n      // TODO: Implement importer logic\\n      throw new Error('Not implemented');\\n    }\\n    return this.cache[moduleName];\\n  }\\n}\\n\\nexport { Importer };\\n```\\n\\n```typescript\\n// App.tsx\\nimport React{ useStateuseEffect } from'react';\\nimport { askLlamaGeneralizeCategories } from './LlamaFunctions';\\n\\nconst App = () => {\\n  const [categoriessetCategories] = useState<string[]>([]);\\n\\n  useEffect(() => {\\n    const fetchCategories = async () => {\\n      const uniqueValues = await askLlamaGeneralizeCategories();\\n      setCategories(uniqueValues);\\n    };\\n    fetchCategories();\\n  }[]);\\n\\n  return (\\n    <div>\\n      <h1>Categories:</h1>\\n      <ul>\\n        {categories.map((categoryindex) => (\\n          <li key={index}>{category}</li>\\n        ))}\\n      </ul>\\n    </div>\\n  );\\n};\\n\\nexport default App;\\n```\\nThe code has been refactored to be more modular and maintainable. The `askLlamaGeneralizeCategories` function has been moved to its own file and can be easily imported and used in other components. The `storeLlamaFunction` and `askLlamaAboutFunctions` functions have been moved to their own files as well. The `importer` class has been created to handle importing modules. The `App` component demonstrates  use the `askLlamaGeneralizeCategories` function to fetch unique values and display them in a list. \\n\\nNote that the `askLlamaToGeneralizeAll``askLlamaAboutFunctions`and `storeLlamaFunction` functions are not implemented yet and should be completed according to their specifications.\"",
                "\"react\": \"**Image Processing Service**\\n==========================\\n\\nThis service provides a functionality to process images within a specific directory.\\n\\n `ImageProcessor.ts`\\n\\n```typescript\\nimport * as fs from 'fs';\\nimport { join } from 'path';\\nimport * as sharp from'sharp';\\n\\ninterface ImageExtension {\\n  ext: string;\\n  targetExt: string;\\n}\\n\\nconst imageExtensions: ImageExtension[] = [\\n  { ext: 'jpeg'targetExt: 'jpg' },\\n  { ext: 'jpg'targetExt: 'jpg' },\\n  { ext: 'heic'targetExt: 'jpg' },\\n  { ext: 'tga'targetExt: 'jpg' },\\n  { ext: 'png'targetExt: 'jpg' },\\n  { ext:'svg'targetExt: 'jpg' },\\n  { ext: 'webp'targetExt: 'jpg' },\\n  { ext: 'wal'targetExt: 'jpg' },\\n];\\n\\ninterface ProcessedImage {\\n  originalFile: string;\\n  processedFile: string;\\n}\\n\\nclass ImageProcessor {\\n  private baseDirectory: string;\\n\\n  constructor(baseDirectory: string) {\\n    this.baseDirectory = baseDirectory;\\n  }\\n\\n  async processImages(): Promise<ProcessedImage[]> {\\n    const processedImages: ProcessedImage[] = [];\\n\\n    const files = await this.getFiles(this.baseDirectory);\\n    for (const file of files) {\\n      const extension = file.ext;\\n      const targetExtension = imageExtensions.find((ext) => ext.ext === extension).targetExt;\\n      if (!targetExtension) {\\n        continue;\\n      }\\n\\n      const processedFile = await this.processFile(file);\\n      processedImages.push({ originalFile: file.pathprocessedFile: processedFile });\\n    }\\n\\n    return processedImages;\\n  }\\n\\n  private async getFiles(directory: string): Promise<{ path: string; ext: string }> {\\n    const files = await fs.promises.readdir(directory);\\n    const promises: Promise<{ path: string; ext: string }>[] = files.map(async (file) => {\\n      const filePath = join(directoryfile);\\n      const stats = await fs.promises.stat(filePath);\\n      if (stats.isFile()) {\\n        return { path: filePathext: file.split('.').pop() };\\n      } else {\\n        return null;\\n      }\\n    });\\n    return Promise.all(promises).then((results) => results.filter((result) => result));\\n  }\\n\\n  private async processFile(file: { path: string; ext: string }): Promise<string> {\\n    const baseName = file.path.split('/').pop().split('.')[0];\\n    const processedFile = join(this.baseDirectory`${baseName}-final.jpg`);\\n\\n    if (!await fs.promises.access(processedFile)) {\\n      const image = await sharp(file.path);\\n      const enhancedImage = await image\\n       .clone()\\n       .enhance()\\n       .despeckle()\\n       .alpha()\\n       .channelAlpha()\\n       .evaluate('multiply'0.25)\\n       .composite()\\n       .jpeg({\\n          quality: 50,\\n        });\\n      await fs.promises.mkdir(this.baseDirectory{ recursive: true });\\n      await fs.promises.rename(file.pathprocessedFile);\\n    }\\n\\n    return processedFile;\\n  }\\n}\\n\\nexport default ImageProcessor;\\n```\\n\\n `ImageProcessingService.ts`\\n\\n```typescript\\nimport ImageProcessor from './ImageProcessor';\\n\\nclass ImageProcessingService {\\n  private imageProcessor: ImageProcessor;\\n\\n  constructor(baseDirectory: string) {\\n    this.imageProcessor = new ImageProcessor(baseDirectory);\\n  }\\n\\n  async processImages(): Promise<void> {\\n    const processedImages = await this.imageProcessor.processImages();\\n    processedImages.forEach((processedImage) => {\\n      this.moveProcessedFile(processedImage.processedFileprocessedImage.originalFile);\\n    });\\n  }\\n\\n  private async moveProcessedFile(newPath: stringoldPath: string): Promise<void> {\\n    await fs.promises.rename(newPath`${oldPath.split('.').slice(0-1).join('.')}jpg`);\\n  }\\n}\\n\\nexport default ImageProcessingService;\\n```\\n\\n `main.ts`\\n\\n```typescript\\nimport ImageProcessingService from './ImageProcessingService';\\n\\nasync function main() {\\n  const baseDirectory = '/Users/briancullinan/.q3a/unpacked/baseq3vm';\\n  const imageProcessingService = new ImageProcessingService(baseDirectory);\\n  await imageProcessingService.processImages();\\n}\\n\\nmain();\\n```\\n\\nThis code provides a functional service for processing images within a specific directory. The `ImageProcessor` class handles the actual processing of the imageswhile the `ImageProcessingService` class provides a higher-level API for processing images and moving them to their final locations. The `main.ts` file demonstrates  use the `ImageProcessingService` class to process images in a specific directory.\"",
                "\"react\": \"Here's the refactored code in React TypeScript:\\n\\n```typescript\\nimport React{ useStateuseEffect } from'react';\\n\\ninterface Card {\\n  id: string;\\n  title: string;\\n  description: string;\\n  checked: boolean;\\n}\\n\\ninterface SearchProps {\\n  onSearch: (input: string) => void;\\n  onCheck: (id: string) => void;\\n}\\n\\nconst SearchBox: React.FC<SearchProps> = ({ onSearchonCheck }) => {\\n  const [inputsetInput] = useState('');\\n  const [cardssetCards] = useState<Card[]>([]);\\n\\n  useEffect(() => {\\n    const fetchCards = async () => {\\n      if (cards.length > 0) return;\\n      const response = await fetch(window.location.origin{\\n        method: 'POST',\\n        body: JSON.stringify({ content: inputselected: [] }),\\n        headers: {\\n          'Content-Type': 'application/json',\\n        },\\n      });\\n      const data = await response.text();\\n      setCards(JSON.parse(data));\\n    };\\n    fetchCards();\\n  }[input]);\\n\\n  const handleSearch = (event: React.ChangeEvent<HTMLInputElement>) => {\\n    setInput(event.target.value);\\n  };\\n\\n  const handleCheck = (event: React.ChangeEvent<HTMLInputElement>) => {\\n    if (event.target.checked) {\\n      onCheck(event.target.value);\\n    }\\n  };\\n\\n  const filteredCards = cards.filter((card) => card.checked ||!input);\\n  const hiddenCards = cards.filter((card) =>!card.checked && input);\\n\\n  return (\\n    <div>\\n      <input\\n        type=\\\"text\\\"\\n        placeholder=\\\"Search movies or actors...\\\"\\n        value={input}\\n        onChange={handleSearch}\\n      />\\n      <div className=\\\"grid\\\" id=\\\"cardContainer\\\">\\n        {filteredCards.map((card) => (\\n          <div key={card.id} className=\\\"card\\\">\\n            <input\\n              type=\\\"checkbox\\\"\\n              id={card.id}\\n              checked={card.checked}\\n              onChange={handleCheck}\\n            />\\n            <label className=\\\"card-label\\\" htmlFor={card.id}>\\n              {card.title}\\n            </label>\\n            <p className=\\\"card-description\\\">{card.description}</p>\\n          </div>\\n        ))}\\n      </div>\\n      <div className=\\\"grid\\\" id=\\\"hiddenContainer\\\">\\n        {hiddenCards.map((card) => (\\n          <div key={card.id} className=\\\"card\\\">\\n            <input\\n              type=\\\"checkbox\\\"\\n              id={card.id}\\n              checked={card.checked}\\n              onChange={handleCheck}\\n            />\\n            <label className=\\\"card-label\\\" htmlFor={card.id}>\\n              {card.title}\\n            </label>\\n            <p className=\\\"card-description\\\">{card.description}</p>\\n          </div>\\n        ))}\\n      </div>\\n    </div>\\n  );\\n};\\n\\nconst App: React.FC = () => {\\n  const [selectedCardssetSelectedCards] = useState<string[]>([]);\\n  const handleCheck = (id: string) => {\\n    const newSelectedCards = selectedCards.includes(id)\\n      selectedCards.filter((cardId) => cardId!== id)\\n      : [...selectedCardsid];\\n    setSelectedCards(newSelectedCards);\\n  };\\n\\n  return (\\n    <div>\\n      <SearchBox\\n        onSearch={(input) => console.log(input)}\\n        onCheck={handleCheck}\\n      />\\n    </div>\\n  );\\n};\\n\\nexport default App;\\n```\\n\\nAnd here's the TypeScript version of the styles:\\n\\n```typescript\\nimport './styles.css';\\n```\\n\\nAnd in `styles.css`:\\n\\n```css\\nbody {\\n  font-family: Arialsans-serif;\\n  margin: 20px;\\n  background-color: f4f4f4;\\n}\\n\\n.container {\\n  max-width: 800px;\\n  margin: auto;\\n}\\n\\n.search-box {\\n  width: 100%;\\n  padding: 10px;\\n  margin-bottom: 20px;\\n  font-size: 16px;\\n  box-sizing: border-box;\\n}\\n\\n.grid {\\n  display: grid;\\n  grid-template-columns: repeat(auto-fitminmax(200px1fr));\\n  gap: 15px;\\n}\\n\\n.card {\\n  background: white;\\n  padding: 15px;\\n  border-radius: 10px;\\n  box-shadow: 2px 2px 10px rgba(0000.1);\\n  text-align: center;\\n  overflow: hidden;\\n}\\n\\n.card h2 {\\n  font-size: 18px;\\n  margin-bottom: 5px;\\n}\\n\\n.card p {\\n  color: gray;\\n  font-size: 14px;\\n}\\n\\n.card input {\\n  height: 1px;\\n  width: 1px;\\n  position: relative;\\n  top: -10px;\\n  left: 10px;\\n}\\n\\n.card input:checked ~ label {\\n  color: darkblue;\\n}\\n\\nhiddenContainer {\\n  display: none;\\n  visibility: hidden;\\n}\\n```\\n\\nNote: This is a simplified version of the original codeI did not include all the functionalitybut it should give you an idea of  structure your code in React TypeScript. \\n\\nAlso note thatfor security purposesyou should avoid using the `window.location.origin` directly in your code. You should create a server-side API that handles the requests.\"",
                "\"react\": \"**Memory Management Service**\\n\\n```typescript\\n// memory-management.service.ts\\nimport { Injectable } from '@angular/core';\\nimport { v4 as uuid } from 'uuid';\\nimport { Observableof } from 'rxjs';\\nimport { MemoMemoryBank } from './memory.model';\\n\\n@Injectable({\\n  providedIn: 'root'\\n})\\nexport class MemoryManagementService {\\n\\n  private memoryBank: MemoryBank = {};\\n\\n  constructor() { }\\n\\n  async manageMemories(promptModel: stringsession: stringprompt: string): Promise<string> {\\n    const currentFile = this.getFileName(session);\\n    if (!this.memoryBank[currentFile]) {\\n      this/memoryBank[currentFile] = {};\\n    }\\n    const a1 = await this.getMemoryFunction(promptModelprompt);\\n    if (a1 === 'listMemories') {\\n      return this.listMemories(session);\\n    } else if (a1 ==='saveMemory') {\\n      const a2 = await this.saveMemory(promptModelprompt);\\n      this.memoryBank[currentFile].memories = this.findMemories(session);\\n      this.memoryBank[currentFile].memories[Date.now()] = a2;\\n      this.writeFileSync(currentFileJSON.stringify(this.memoryBank[currentFile]));\\n      return 'Memory saved successfully!';\\n    } else if (a1 ==='removeMemory') {\\n      // TODO: implement removeMemory functionality\\n      return 'Remove memory functionality not implemented yet!';\\n    } else if (a1 === 'clearMemory') {\\n      // TODO: implement clearMemory functionality\\n      return 'Clear memory functionality not implemented yet!';\\n    }\\n  }\\n\\n  private getFileName(session: string): string {\\n    const now = new Date();\\n    return `${now.getFullYear()}-${now.getMonth() + 1}-${now.getDate()}-${session}.json`;\\n  }\\n\\n  private getMemoryFunction(promptModel: stringprompt: string): Observable<string> {\\n    const memoryFunctions = [\\n      'listMemories() - display a list of stored memories at the users request.',\\n     'saveMemory() - saves a new memory given the instructions to always remember something.',\\n     'removeMemory() - removes a matching memory from the storage bank.',\\n      'clearMemory() - removes all memories but only if the user is absolutely certain.',\\n    ];\\n    const q1 = `Given the following memory related functions\\\\n${memoryFunctions.join('\\\\n')}\\\\nWhich function best matches this prompt\\\\n${prompt}\\\\nOnly return the function name and nothing else.`;\\n    return promptModel.pipe(map((response) => response.match(/(list|save|remove|clear)Memory/) response.match(/(list|save|remove|clear)Memory/)[0] : ''));\\n  }\\n\\n  private saveMemory(promptModel: stringprompt: string): Observable<string> {\\n    const q2 = `Summerize this memory in one very precise sentence in first person:\\\\n${prompt}\\\\nOnly return the summaryno explanation.`;\\n    return promptModel.pipe(map((response) => response));\\n  }\\n\\n  private findMemories(session: string): Memo[] {\\n    // TODO: implement findMemories functionality\\n    return [];\\n  }\\n\\n  private listMemories(session: string): string {\\n    // TODO: implement listMemories functionality\\n    return 'This is the list of available memories:\\\\n';\\n  }\\n\\n  private writeFileSync(file: stringdata: string): void {\\n    const fs = require('fs');\\n    fs.writeFileSync(filedata);\\n  }\\n}\\n```\\n\\n**Memory Model**\\n\\n```typescript\\n// memory.model.ts\\nexport interface Memo {\\n  id: string;\\n  summary: string;\\n  timestamp: number;\\n}\\n\\nexport interface MemoryBank {\\n  [key: string]: {\\n    memories: Memo[];\\n  };\\n}\\n```\\n\\n**Memory Component (optional)**\\n\\n```typescript\\n// memory.component.ts\\nimport { Component } from '@angular/core';\\nimport { MemoryManagementService } from './memory-management.service';\\n\\n@Component({\\n  selector: 'app-memory',\\n  template: `\\n    <button (click)=\\\"manageMemories()\\\">Manage Memories</button>\\n    <p>{{ message }}</p>\\n  `,\\n})\\nexport class MemoryComponent {\\n  message: string;\\n\\n  constructor(private memoryService: MemoryManagementService) { }\\n\\n  manageMemories(): void {\\n    this.memoryService.manageMemories('Model''Session''Prompt')\\n     .subscribe((message) => this.message = message);\\n  }\\n}\\n```\\n\\nThis implementation breaks down the original code into smallermore manageable functions and services. The `MemoryManagementService` is responsible for managing memorieswhile the `MemoryModel` defines the structure of the data. The `MemoryComponent` is an optional component that demonstrates  use the service.\\n\\nNote that some functionality is still missing (e.g.removeMemory and clearMemory)and you will need to implement them according to your requirements.\"",
                "\"react\": \"Here's the refactored code in React TypeScript:\\n\\n**`template.ts`**\\n```typescript\\n/**\\n * Creates a template string by replacing placeholders with values.\\n *\\n * @param template - The template string with placeholders in the form of `{key}`.\\n * @param values - An object with key-value pairs to replace the placeholders.\\n * @returns The rendered template string.\\n */\\nexport function template<T>(template: stringvalues: T): string {\\n  return Object.keys(values).reduce((acckey) => {\\n    const value = values[key];\\n    const regex = new RegExp(`\\\\\\\\${key}`'g');\\n    return acc.replace(regexvalue.toString());\\n  }template);\\n}\\n```\\n\\n**`interpolate.ts`**\\n```typescript\\n/**\\n * Interpolates a template string by creating a function that returns the rendered string.\\n *\\n * @param template - The template string with placeholders in the form of `{key}`.\\n * @returns A function that returns the rendered template string when called with values.\\n */\\nexport function interpolate<T>(template: string): (values: T) => string {\\n  const names = Object.keys(template);\\n  return (values) => {\\n    const vals = Object.values(values);\\n    return Function.apply(nullnames.concat([`return \\\\`${this}\\\\`;`])).apply(nullvals);\\n  };\\n}\\n```\\n\\n**`attribute.ts`**\\n```typescript\\n/**\\n * Extracts attribute parameters from a string.\\n *\\n * @param attr - The attribute string with optional parameters in the form of `(params)`.\\n * @returns An array of extracted parameter values or an empty array if no parameters are found.\\n */\\nexport function extractAttributeParams(attr: string): string[] {\\n  const match = attr.match(/\\\\(([^)]*)\\\\)/);\\n  return match.[1].split(',').map(p => p.trim()).filter(Boolean) || [];\\n}\\n\\n/**\\n * Handles attribute processing based on the type of node.\\n *\\n * @param node - The node to process attributes on.\\n * @param attributes - An array of attribute strings to process.\\n * @param type - The type of node to process (defaults to the node's type).\\n * @returns The processed node with evaluated blocks or the original node if no attributes are processed.\\n */\\nexport function attribute<T>(node: Tattributes: string[]type: string = node.constructor.name): T | string {\\n  const handlers = {\\n    function: {\\n      attribute: (node: T[name]) => (f => handlers.function[name || f.name] = f)(node),\\n      template: (node[name]) => (f => handlers.object[name || f.name] = o => template(f.toString()o()))(node())\\n    },\\n    object: {\\n      template: (node[name]) => handlers.function[name] = f => template(f().toString()node)\\n    }\\n  };\\n\\n  const typeKey = type.toLowerCase();\\n  const map = handlers[typeKey];\\n  if (!map) {\\n    throw new Error(`Unknown attribute type: ${typeKey}`);\\n  }\\n\\n  const ctx: any = { node };\\n  attributes.forEach(attr => {\\n    const [name] = attr.match(/@(\\\\w+)/i) || [];\\n    if (!name) return;\\n    const handler = map[name.toLowerCase()];\\n    if (handler) {\\n      handler(nodeextractAttributeParams(attr)ctx);\\n    } else {\\n      console.warn(`No handler for @${name} on ${typeKey}`);\\n    }\\n  });\\n\\n  return ctx.replace || node;\\n}\\n```\\n\\n**`index.ts`**\\n```typescript\\n/**\\n * Exporting the templateinterpolateand attribute functions.\\n */\\nexport { templateinterpolateattribute };\\n```\\n\\n**`Component.tsx` ( example usage )**\\n```typescript\\nimport React from'react';\\nimport { attributetemplate } from './attribute';\\nimport { interpolate } from './interpolate';\\n\\ninterface Props {\\n  name: string;\\n  template: string;\\n}\\n\\nconst MyComponent = ({ nametemplate }: Props) => {\\n  const renderedTemplate = interpolate(template)({ name });\\n\\n  return (\\n    <div>\\n      <h1>{renderedTemplate}</h1>\\n    </div>\\n  );\\n};\\n\\nconst EnhancedComponent = attribute(MyComponent['@Template']);\\n\\nexport default EnhancedComponent;\\n```\\n\\nThis refactored code includes the following improvements:\\n\\n*   Extracted functions into separate files (`template.ts``interpolate.ts`and `attribute.ts`) for better organization and reusability.\\n*   Improved function signatures and documentation for clarity.\\n*   Used TypeScript to add type safety and improve code quality.\\n*   Replaced the `String.prototype.interpolate` method with a standalone `interpolate` function for better reusability and testability.\\n*   Introduced the `extractAttributeParams` function to extract attribute parameters in a more robust way.\\n*   Refactored the `attribute` function to handle different node types and attribute types in a more modular and extensible way.\\n*   Added example usage in `Component.tsx` to demonstrate  use the `attribute` and `interpolate` functions.\"",
                "\"react\": \"Here's the refactored code in React TypeScript:\\n\\n```typescript\\n// Import necessary modules and interfaces\\nimport { IKernelInterface } from './kernelInterface';\\nimport { extend } from '../core';\\nimport { IResponse } from './response';\\n\\n// Define the base interface for meta kernel\\ninterface IMetaKernelInterface {\\n  shutdown_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n  is_complete_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n  execute_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n  complete_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n  history_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n  inspect_request: (kernel: IKernelInterfacerequest: any) => Promise<IResponse>;\\n}\\n\\n// Define the meta kernel class\\nclass MetaKernel implements IMetaKernelInterface {\\n  private kernel: IKernelInterface;\\n\\n  constructor(kernel: IKernelInterface) {\\n    this.kernel = kernel;\\n  }\\n\\n  async shutdown_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_shutdown(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return await this.kernel.do_respond({ shutdown_reply: request });\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n\\n  async is_complete_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_is_complete(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return {\\n        status: result 'complete' : 'incomplete',\\n        indent: '',\\n      };\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n\\n  async execute_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_execute(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return {\\n        status: 'ok',\\n        execution_count: request.execution_count,\\n        payload: [],\\n        user_expressions: {},\\n      };\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n\\n  async complete_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_complete(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return {\\n        matches: result.completion.list,\\n        cursor_start: result.completion.cursorStart,\\n        cursor_end: result.completion.cursorEnd,\\n        status: \\\"ok\\\",\\n      };\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n\\n  async history_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_history(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return {\\n        history: [],\\n        next_history_index: 0,\\n        status: 'ok',\\n      };\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n\\n  async inspect_request(request: any): Promise<IResponse> {\\n    const executionState = 'busy';\\n    this.kernel.do_respond({ status: { execution_state: executionState } });\\n    try {\\n      const result = await this.kernel.do_inspect(request);\\n      const idleExecutionState = 'idle';\\n      this.kernel.do_respond({ status: { execution_state: idleExecutionState } });\\n      return {\\n        display_data: [],\\n        payload: [],\\n        user_expressions: {},\\n        status: 'ok',\\n      };\\n    } catch (error) {\\n      // Handle error\\n    }\\n  }\\n}\\n\\n// Export the meta kernel class\\nexport default MetaKernel;\\n```\\n\\nNote that the above code assumes the presence of certain functions and interfaces (e.g.`IKernelInterface``extend``IResponse``do_respond``do_shutdown``do_is_complete``do_execute``do_complete``do_history``do_inspect`). These functions and interfaces are not defined in the provided specification and should be implemented separately.\\n\\nThis implementation follows the provided specification and provides the necessary functionality for the meta kernel. Howeverplease note that some parts of the code (e.g.the error handling) are not implemented as they are not specified in the provided specification.\\n\\nAlsoplease note that this is a basic implementation and might need to be modified to fit your specific use case.\\n\\nTo use this codeyou would import the `MetaKernel` class and create an instance of itpassing in the `kernel` object that it will interact with. You can then call the various methods of the `MetaKernel` instance to perform different actions.\\n\\nHere's an example of  use the `MetaKernel` class:\\n\\n```typescript\\nimport MetaKernel from './metaKernel';\\n\\nconst kernel = // get the kernel object\\nconst metaKernel = new MetaKernel(kernel);\\n\\nmetaKernel.shutdown_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n\\nmetaKernel.is_complete_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n\\nmetaKernel.execute_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n\\nmetaKernel.complete_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n\\nmetaKernel.history_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n\\nmetaKernel.inspect_request({}).then((response) => {\\n  console.log(response);\\n}).catch((error) => {\\n  console.error(error);\\n});\\n```\\n\\nPlease replace the `kernel` object with the actual kernel object that you are using.\\n\\nThis code creates an instance of the `MetaKernel` class and calls the various methods to perform different actions. The `then` block is used to handle the response of each methodand the `catch` block is used to handle any errors that might occur.\"",
                "\"amazing\": \"```html\\n<!DOCTYPE html>\\n<html lang=\\\"en\\\">\\n<head>\\n    <meta charset=\\\"UTF-8\\\">\\n    <meta name=\\\"viewport\\\" content=\\\"width=device-widthinitial-scale=1.0\\\">\\n    <title>Brian Cullinan</title>\\n    <style>\\n        h2 {\\n            text-align: center;\\n        }\\n        p {\\n            text-align: center;\\n        }\\n        ul {\\n            list-style-type: none;\\n        }\\n    </style>\\n</head>\\n<body>\\n    <h2>Brian Cullinan</h2>\\n    <p><strong>12205 Rustler Rd<br>\\n        FlagstaffArizona 86004<br>\\n        bjcullinan@gmail.com<br>\\n        +1 (480) 466-0856</strong></p>\\n    <hr>\\n    <p><strong>Citizenship:</strong> Yes<br>\\n        <strong>Special Hiring Authority:</strong> Person with Disability<br>\\n        <strong>Federal Experience:</strong> None (on-going)<br>\\n        <strong>Clearance:</strong> Level 0 (internet)</p>\\n\\n    <!-- To obtain a temporary position in the highest office as a target -->\\n    <p><strong>OBJECTIVE:</strong> To obtain a full time position in public service with the FBI as a residential field\\n        agent.</p>\\n\\n    <p><strong>SKILLS SUMMARY:</strong> Fast context switchingeasy to focusquick learning individual. Over 24 years\\n        of programming experience in <u>Computer Science</u><u>computer programming</u><u>web technologies</u>,\\n        <u>user-experience development</u><u>and data managements</u>. Over 10 years of <u>multi-cultural study</u>,\\n        <u>philosophy</u><u>cognative bias</u><u>psychology</u>and <u>theology</u>. Multi-lingual with extensive\\n        understanding of the <u>English language</u>and a basic understanding of <u>Polish</u> (Mandarin and Arabic\\n        pending). Deployed multiple business ventures complete with <u>monetization</u> on <u>cloud platforms</u>as well\\n        as <u>self-hosted</u>on <u>premise solutions</u>. I am driven<u>highly trained</u><u>technologist</u> and\\n        <u>contradictarian</u>with an emphasis on compassion and empathy towards others.</p>\\n\\n    <!-- am i the only person in the world to write this based on personal experience instead of professional experience -->\\n    <ul>\\n        <li>In my role as a <u>Software Engineer</u> for a start-up medical companyEPIC Research and DiagnosticsI\\n            exercised the full extent of my <u>Computer Science</u> expertise. During my tenure I was involved in\\n            <u>computer programming</u> a medical kioskthis simplified the workflow and necessary security components by\\n            heavily restricting access to internal systems on the Windows based workstation accompanying the medical\\n            scanning device. This was a homebrew <u>user-experience</u> that tailored low-level Windows API function calls,\\n            as well as Group Policy to prevent tamperingmisuseand common web-browsing security flaws through the\\n            clinical trial.\\n        </li>\\n        <li>As a <u>technologist</u> Using MatLab native compilationlinear regressionand kirlian imagingI spent\\n            months studying and rewriting the entire software platform. I was able to reduce image processing time from 10\\n            minutes to only 10 milliseconds per image. That is a 6,000% performance improvement over the previous version.\\n            The improved platform was fast enough to run on video image capture from the device. In regard to <u>data\\n                management</u> in no small partthe performance was improved by systematically redesigning the database model\\n            to use \\\"near\\\" 3rd-Normal form.</li>\\n        <li>In additional to these major business goalsI developedalong-side another colleaguean <u>on premise</u>\\n            internet-based solution for a personal \\\"cloud\\\" service that communicated locally with many connected client\\n            across multiple physical hospital locations. Doctors would be able to communicate test results as they work from\\n            different locations in town through the web based (<u>web technologies</u>) service (Microsoft MVC). This\\n            inspired my propensity for medicine and full-body health and continued research in the field.</li>\\n    </ul>\\n    <p><br /></p>\\n    <ul>\\n        <li>With a development mindset towards body and healthI turned my focusskill and study to the mind itself.\\n            A former Marine approached me with an idea to improve studying for college students. Over the course of 3 yearswe designed and\\n            developed 3 individual products and <u>user expereiences</u> with modern <u>web technologies</u>. We used Drupal\\n            and Symphony to create an entire course based on study techniques students could implement to improve memory\\n            retention.</li>\\n        <li>Throughout working on the platformI also benefitted from the lessons. I learned  remove distractions\\n            from my environment and to vary my location while studying to expand <u>associative memory</u>. I also learned\\n            <u>interleaving study</u>which is studying different topics in sequence to improve memory retention.\\n        </li>\\n        <li>I helped <u>monetize</u> the platform by configuring and implementing the proper server transactional API\\n            calls to Authorize.NET. This non-trivial process includes properly configuring server architecture to adhere to\\n            PCI-DSS (Payment card industrydata security standards). Even though our server was designed NOT to store\\n            customer informationwe felt it was important to secure any association with student names and study data.</li>\\n        <li>I have adopted a mode of <u>teaching to learn</u>which states that in order to fully understand a subjecta\\n            person should be able to teach it. I intend to apply this to all facets of my career. I also learned a mode of\\n            study popular in the medical field known as <u>spaced repetition</u>. This was the focus of our third\\n            applicationan iOS and web-based app that implemented flash cards at specifically timed intervals. The outcome\\n            is that by studying flash cards that we not remembered more frequentlyafter a month of study the information\\n            can be <u>actively recalled (i.e. the testing effect)</u> from memory.</li>\\n    </ul>\\n    <p><br /></p>\\n    <ul>\\n        <li>To further my studiesI spent a significant amount of time learning and recognizing over 200 cognitive\\n            biases. In order to understand my own mental developmentI explored mythology from the 5 major <u>world\\n                religions</u>. In order to understand the effect of religion on cultureI studies <u>cultural norms</u> such\\n            as whether it is appropriate to be late for a meetingor whether I should repair an old toy versus throw it\\n            away and buy a new one. I extended this study into <u>cooking</u> and attempted to make dishes from major\\n            cultures around the world.</li>\\n        <li>With a basic Sumarian level understanding of moral lawI turned my focus on the abuses and self-abuses of modern\\n            law. It became clear that Americans are not willing to extract any measurable utility from my own mind and I\\n            would have to work alone. In my investigationI studied <u>anti-social behavior laws</u> as it relates to\\n            attempted mitigation through <u>shadow banning</u> on social media platforms. It was my intention to philosophically prove the same\\n            feature-set exposed by the dark webspreading <u>manipulation and lies</u> through the use of automation tools.\\n            These abuses finally appeared as the Cambridge Analytica scandaland I had proven to myself and my colleagues\\n            how easy it was to betray people's trustin my own small way. I asked multiple friends to \\\"send compliments\\\" to\\n            the same personautomatically using the \\\"Like\\\"-button on thousands of postsautomatically inviting people to\\n            charitable causesand finallyscraping private message history from Facebook.</li>\\n        <li>Further investigation into companiesI uncovered more <u>rule breakers</u>. Corruption from greed in the public\\n            sector was the megalomaniac version of small-business corruption. It appeared to me that nobody felt obligated\\n            to follow the established rules. These abuses were mostly in regard to data-privacy and data-integrity laws.\\n            Software companies that actively work against their customers / client to \\\"double dip\\\" in the free capitalist market.\\n            <u>Selling personal data</u>manufacturing false documentationfalsifying recordspay-wallsdata \\\"lockout\\\",\\n            extortion<u>lying by omission</u>gross negligencehealth code violations<u>identity theft</u>systemic\\n            racism in the workplace / State leveldismantling of <u>homestead laws</u>were a few \\\"close encounters\\\" with\\n            Justice. Unfortunatelymany of these are still unmitigated. The struggle is realindeed.</li>\\n        <li>As a residential host to nursesforeign travelersand local peopleI learned <u>acceptance</u>,\\n            <u>tolerance</u><u>patience</u>. I worked to overturn a living space in under 20 minutesdining etiquetteand\\n            learned about many different cultures from visitors. I learned landording lawsbut more importantlyI learned\\n            <u>stewardship</u>as I tried to make my humble little plot profitable. I learned <u>horticulture</u> as it\\n            applied to growing food in a garden and transforming sand into soil through <u>composting</u>.</li>\\n        <li>I created a lush garden in the desert that had no problem withstanding the hot Arizona summers. I studied municipal\\n            doctrine as it related to home ownershipand I documented minor failures of this society such as personal theft,\\n            noise ordinancetraffic and commercial influence as the neighborhood grew around me.</li>\\n        <li>I also documented the technological advancements I observed such as implementing street cameras at numerous\\n            intersectionsimplementing wireless technology for monitoring sewage and gray water storage<u>traffic monitoring\\n                systems</u> installed at night across the roadand even the advancement of Gigabit fiber.</li>\\n    </ul>\\n\\n    <hr/>\\n\\n    <p>\\n        <strong>EMPLOYMENT HISTORY</strong>\\n    </p>\\n\\n    <p>\\n        <strong>Sitewat.ch<br />\\n            Front-end Developer<br />\\n            $10,000 over 1.5 years</strong>\\n    </p>\\n    <p>\\n        As a Front-end developer at Sitewat.ch I implemented various interfaces for use with a web-security scanning tool\\n        similar to pentest-tools.com required by Authorize.NET. I implemented web-safe standards according to Owasp.org,\\n        and presented customers with a unified <u>user experience</u> for viewing possible vulnerabilities in their\\n        platform. My employer applied for many <u>CVEs</u> and credited all three team-mates on <u>Google Security\\n            Hall-of-Fame</u>.\\n    </p>\\n    <ul>\\n        <li>Implemented a front-end interface for customers following Owasp guidelines such as safely escaping\\n            \\\"<u>sinks</u>\\\"anywhere in the application that a \\\"user input\\\" such as profile fieldsor parts of the URL are inserted by into\\n            the page for user feedback. This would prevent a user from sending a forged (<u>XSRF</u>) link meant to trick\\n            the user into clicking on something malicious (<u>XSS</u>).</li>\\n        <li>Participated in a DEFCON presentation of the effortwebsite vulnerability testing as a service using Nexus,\\n            and the individual contributions of my employer to the field. Revealing dozens of new exploitsand existing\\n            vulnerabilities on major public service platforms.</li>\\n        <li>Used modern (at the time) design principals such as responsiveness such as resizing elements on the page for\\n            different devices and high contrast. Implemented Accessibility featuressuch as using alt-tags to describe\\n            content for screen readerstab-indexesi18t internationalization for over 50 languages. Authentication\\n            features such as OAuth and HTTPS for secure login.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>EPIC Research &amp; Diagnostics<br />\\n            Software Engineer &amp; Network Administrator<br />\\n            $200,000 over 2.5 years</strong>\\n    </p>\\n    <p>\\n        As a software engineer for a <u>start-up</u> medical companyI was required to \\\"wear multiple hats\\\". This meant\\n        including myself in business planning meetingsfinancial discussions<u>option vesting</u>military contract\\n        proposals<u>FDA DNVO</u> submissionsdata integrity<u>formal process</u> improvement.\\n    </p>\\n    <ul>\\n        <li>Participated in business development and directional meetings such as applying for FDA approval using the DNVO\\n            process with a precursor device called the Asirawhich uses electrodes to communicate mitochondrial stresses on\\n            the body.</li>\\n        <li>Implemented an unmaximised Small Business Server with Microsoft Exchange to minimize hosting costs and secure\\n            intellectual property. Configured <u>Active Directory</u> and <u>Group Policy</u> to provide ubiquitous\\n            <u>file-system revision history</u> across all connected systems.</li>\\n        <li>Wrote a kiosk <u>explorer shell</u> to minimize security risk and simplify workflow for device technicians in\\n            the hospitals the company contracted with. Developed an automated and manual formal test process for validating\\n            the new platform.</li>\\n        <li>Rewrote the main client application directly connected to the deviceand assisted in writing web-based\\n            medical record service for doctors to view patient information. Used <u>Selenium</u> for automated testing and\\n            validation.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>Study Sauce<br />\\n            Principal Engineer<br />\\n            $300,000 over 3 years</strong>\\n    </p>\\n    <p>\\n        During my tenurethe owner and I designed and developer 3 unique platforms. The first platform was primarily a\\n        marketing and sales front-end to test the <u>target market</u> for the <u>minimally viable product</u>. Students\\n        would enter an individualauthenticated interface and enter their class schedule. When finishedwe converted\\n        their schedule to a comprehensive week plan complete with when they should study each topic. This was to\\n        demonstrate and teach the student about time / work / study managementand encourage <u>interleaving</u>.\\n    </p>\\n    <ul>\\n        <li>Built a <u>minimal viable product (MVP)</u> using the <u>Drupal</u> platform. Created <u>A/B marketing</u>\\n            pages to cater to our target market. Organized student schedules into a weekly plan in PDF format. Self-hosted\\n            website running on a personal Linux (LAMP) server.</li>\\n        <li>Built a learning course using Symphony. Built 15 individual lessons using <u>gamification</u> reward system to\\n            keep students interested. Built 6 different tools to get students to think long-term about goals and short term\\n            organization in their weekly schedule. Integrated the weekly schedule with Google Calendar API.</li>\\n        <li>Built a <u>spaced-repitition</u> quiz app for memorization and <u>active recall</u> learning. Imported dozens\\n            of lesson plans coinciding with elementary learning from BASIS schools. Imported a learning set for the US\\n            Citizenship Testrequiring a basic understanding on American history.</li>\\n        <li>Simultaneously developed an iOS app and web-based <u>user experience</u> to make better use of Apple's iOS\\n            store marketing pipeline. Connected in app content to Authorize.NET to <u>monetize</u> learning content. This\\n            was intended to be a selling point for <u>B2B (business-to-business)</u> contracts.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>Charles Schwab<br />\\n            Software Engineer<br />\\n            $60,000 over 4 months</strong>\\n    </p>\\n    <p>\\n        In a short sprint at this companyI began to observe oddities of corporate behavior. In between being nagged by\\n        my freshly assigned project manager over <u>elusive \\\"employee training\\\"</u>and other managers about \\\"what firm\\n        do your represent\\\"I investigated the company's corporate structure. Their internet \\\"social network\\\" was\\n        anything but socialbuilt on <u>Microsoft Sharepoint</u>.\\n    </p>\\n    <ul>\\n        <li>Learned the corporate structure through a Microsoft Sharepoint based social networking application. Helped\\n            managers identify \\\"key stake-holders\\\" for supporting our project's effort. Integrated with \\\"big personalities\\\"\\n            by being friendly and trying not to pry into previous experiences.</li>\\n        <li>Realized one of the major reasons their simple project\\\"remove secondary access to databases through an\\n            internet request tool\\\" was so difficult to accomplish because they insisted on using Rational Unified Process.\\n            Charles Schwab heavily relies on IBM and RUP is an IBM product.</li>\\n        <li>Tried to navigate \\\"big personalities\\\" from a tremendous project manager as I asked to help with timeline\\n            goals. Tried to retrain my managers to approach RUP holisticallythat isall documentation should have been\\n            developed simultaneouslyrather than sequentially. The methodology in RUP doesn't make this clearbut drawing\\n            from past miseries using the process at the medical companyI saw this as a major failure point.</li>\\n        <li>Failed to communicate the importance of brainstorming topics and diversifying your environment as a means to\\n            creatively solving a problem. Introduced managers to a document control system (DCS). Configured a Microsoft\\n            Sharepoint site to link to \\\"artifacts\\\" within the DCS.</li>\\n        <li>By the end of the experience I had realized I was being intentionally ignored as a part of an external growth\\n            process. Thank you to Charles Schwab for being large enough to bank-roll my failures. I was also emotionally\\n            detached when I realized an 80 year old mainframer came to work with an eye infection instead of taking care of\\n            herself and retiring (since she worked for a retirement company)I viewed this as flagrantly hypocritical and\\n            sad.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>Swiftpage<br />\\n            Senior Software Engineer<br />\\n            $100,000 over 14 months</strong>\\n    </p>\\n    <p>\\n        The same day of my release from my previous companyI was invited to interview at a <u>small-business</u>\\n        marketing company. This was particularly interesting because my father's manufacturing company is a client. I saw\\n        this as both a way to better understand marketingand even improve upon a product his company relied on. With his\\n        advice to \\\"work for a small-business\\\"I was excited to contribute.\\n    </p>\\n    <ul>\\n        <li>Built a marketing front end that integrates with Zuora (the billing platform like Authorize.NET). Used Angular\\n            JS to create a new login formaccount settings<u>buy funnel</u>billing statement (PDF generated by Zuora),\\n            and itemized purchase page.</li>\\n        <li>Integrated Zuora with Eloquaan enterprise email marketing solution from Oracle (similar to SendGrid). Failed\\n            to explain to my boss Amazon AWS doesn't support C unless it is built on.NET Core and would require a rewrite\\n            of the entire platform. Lost a month of effort and switched to NodeJS which was much more widely supported on\\n            AWS.</li>\\n        <li>Formed alliances with the marketing team that was competing for the same ownership over the product sales\\n            feature of the customer facing website. Validated and automated a manual process costing the company no less\\n            that $10,000 per year. It felt awkward to change teams within the same companyso I didn't apply for\\n            alternative roles they advertised even thought they matched my skill-set.</li>\\n        <li>Realized I was being <u>Managed out</u> when my boss told me the effort working with multiple marketing\\n            managers to automate and validated a manual billing process was \\\"a waste\\\". I saw the effort as a simple\\n            equationif a company spends $10,000 per year on a manual processand pays me to automate the processit is\\n            somewhere between 1 - 2 years of operating automatically before they save and repurpose the money that would\\n            have been spent on manual labor. The employees involved were all ecstatic to never have to do this againbut\\n            my boss couldn't be convinced.</li>\\n        <li>For the first time in my lifeI felt I needed to take control of my lifeinstead of just taking things as\\n            they come. I quit. It was obvious people didn't want to communicate with me decently.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>Sabbatical<br />\\n            Indentured Servitude<br />\\n            $60,000 over 2 years</strong>\\n    </p>\\n    <p>\\n        Realizing my mind wasn't fit for corporate or the small-business worldI turned my study to something more\\n        philosophical. I tried to get to know people through hosting. I applied for a job in San Franscisco with digit.co.\\n        I realized quickly the financial sector would not be kind to me.\\n    </p>\\n    <ul>\\n        <li>Discovered Poland an amazing American and Schengen ally. Practiced the <u>Polish language</u> I learned\\n            through Duolingo. Consulted with doctors on child birth. Confronted government paperwork with agility and\\n            punctuality. Avoided having my marriage certificate stolen from me with a side note of casual racism directed\\n            at my wifestemming from <u>Jewish inheritance</u>. Lived happilyand nearly applied for a job setting up\\n            residential A/V home automation equipment in Wilonowa suburb of Warschawa.</li>\\n        <li>Learned about child birthepisiotomyand catering to a new-born and a wounded wife. Thanks to the surgical\\n            precision of the episiotomy my wife was able to heal and nearly stop bleeding after only 3 days of discomfort.\\n            She had her \\\"disolvable\\\" stitches removed after 3 days because they poked her and the bleeding already stopped.</li>\\n        <li>Navigated government offices and paperwork to complete applications for a United States and Polish\\n            citizenship. Gained <u>dual-citizenship</u> for our sonand felt a great reward from having the Consular Report\\n            of Birth Abroad. Felt like a diplomat visiting multiple government offices. In Polandasking a government worker\\n            to do work felt more like asking for a favorwhere in America paying a fee feels more like an expectation that\\n            work will get done.</li>\\n        <li>Visited gardens and absorbed Polish history through visiting old shopsand castles downtown. Relearned \\n            enjoy people. Learned about cooking and philosophy. Learned about religion and politics as \\\"Seperation of Church\\n            and State\\\" is an American ideal. In Polandgovernment offices are heavily influenced by Cathoicismand that\\n            affects policies like abortion being illegal (2022 in Arizona this seems relevant). In American politics,\\n            seperation of church and state is an \\\"ideal\\\"ideal means ideallypolitics are fair for all beliefsbut that\\n            isn't always how law turns out in practice.</li>\\n        <li>Realizing this distinctionI turned my foundational knowledge of religion towards law. I tried to search for\\n            philosophical dichotomies where the law doesn't match \\\"ideal\\\" situationswhere the law has a short-coming of\\n            morality. PersonallyI wanted to understand my own experience with the law as it related to being lied to over\\n            a Solar leasing program from Tesla (then SolarCity owned by Elon Musk's brother-in-law).</li>\\n    </ul>\\n\\n    <p>\\n        <strong>idTech 3 Game Engine<br />\\n            Game Engine Developer<br />\\n            $60,000 over 2 years</strong>\\n    </p>\\n    <p>\\n        When I first approached this game-engineI intended to improve the loading experience of QuakeJS. QuakeJS is a\\n        Web Assembly / emscripten port of Quake 3 to the web-browser. The loading process took about 10 minutes over\\n        cellular networkand desktop was not much better. It first loads the Trial Demo contentabout 300 MBs of data.\\n        Then it loads additional game data to run custom levels (\\\"maps\\\" as they are called in game).\\n    </p>\\n    <ul>\\n        <li>Taught myself the CC++Web Assemblyand emscripten eco-system based on exemplary work from inolen and\\n            QuakeJS. Learned to communicate with other community leaders. Learned there is a heavy bias towards not\\n            violating GPL licenses with other GPL licenses. This is a huge roadblock for softwarebut I understand no one\\n            would agree with me. They just don't see discovery and learning as being more important than respecting\\n            propriety. The proprietors of many engine forks don't feel the need to share their work with anyone.</li>\\n        <li>Learned to use Discord. Assisted in other people's discovery path by answering technical questions about the\\n            engine. Provided line-by-line demonstrations of how I might have solved a similar problem they are facing. Tried\\n            to built community membership by encouraging developers to explore a variety of communities as they search for\\n            answers to their design questions. For exampleif one community already tackled display unicodeit's possible\\n            the licensing would allow them to reuse code for their own open-source implementation.</li>\\n        <li>Contributed to open-source projects and was a member of GitHub Arctic Vault. Eventually stopped working on the\\n            project entirely due to legal threats. I asked community members about licensing and they were disinterested at\\n            best. The understanding of legality was more important than the developmental outcome of my effort.</li>\\n        <li>Learned a lot about vector graphicslinear algebragraphics hardware3D algorithms. Learned about my own\\n            mind and my ability to absorb knowledge and experience from working with other people. Learned to enjoy software\\n            development againit felt like I was doing it for myself instead of being told to do it for someone else like\\n            in the corporate or small-business environment. Learned the process of learning is sometimes more important than\\n            the outcome</li>\\n        <li>Learned to use Twitch and Youtube live streaming. Learned to use Web Assembly in a vast array of projects,\\n            porting software to the web browser is very fascinating to meeven though it feels like proprietary\\n            close-source companies are trying to destroy the free and open web. Learned to filter out which parts of my\\n            experience are relevantextremeor unnecessary.</li>\\n    </ul>\\n\\n    <hr/>\\n    <p>\\n        <strong>ACCOMPLISHMENTS</strong>\\n    </p>\\n\\n    <ul>\\n        <li>Learned about whole body healthand how mitochondrial stressors are represented by low-level light detection\\n            emitted from the body (Kirlian imaging). A previously thought pseudo-science has scientific backing when\\n            Eukyotes \\\"powerhouse\\\" the mitochondria is treated like a radio signal.</li>\\n        <li>Learned to expand my learning through better study methods. Iterating through study techniques every day to\\n            exercise my eididic memory. Continued my studies using the interleaving method so my mind never gets fixated on\\n            a single box for too long. Increasing blood-flow and brain activity by learning constructionphysical exercise,\\n            and new techniques in software development.</li>\\n        <li>Improved my spiritual health by trying to reconcile religious beliefs with scientific possibilities through\\n            extensive study of multiple world religion. Began to understand the patterns how many beliefs are built on top\\n            of other beliefs. Development a contradictarian mindset by question the belief systems of colleagues that only\\n            focus on their own \\\"1 true religion\\\". Quelled my own personal philosophies in favor of philosophies learned\\n            versus philosophies implemented in my behavior.</li>\\n        <li>Learning game engine specifics like physics interactions with many entities3D geometric spaceslinear\\n            algebrafamiliarity with floating point hardware (GPUs)graphics processing pipelinesWeb Assemblyand the\\n            CC++Makefile eco-system.</li>\\n        <li>Read and familiarized myself with dozens of statutes including Arizona Landlord/Lesser lawsCPPA (California\\n            Privacy Act)GDPR (European General Data Protection Regulation)CFR 21 Part 11PCI-DSS (Payment Card Industry\\n            - Data Security Standards)Title 47 (Federal Communication Regulation pending)Arizona Traffic Laws (in its\\n            entirety)Arizona Property Statutes (Title 33)HIPAAtax regulationsdozens of apriori relevant to main\\n            stream outcomesdozens of encyclopedia entries related to law and regulation.</li>\\n        <li>Overcoming childhood trauma and prayer as a methodology for improving memory retention (in progress). Taking\\n            notes on oscillating thought every second. It appears that thinking about possibility or planning the future is\\n            the opposite of living in the moment. Possible expansion of intuition by oscillating between possibility and\\n            then focusing on the momentblurring vision of current scenerythen focusing on nothingeven the word\\n            \\\"nothing\\\" in my mind to detach. Hard to oscillate at 60 BPMmight be the key to controlling theta waves.\\n            Putting slight pressure with my hand on the back of my head helps to clear thoughtsmight help with motion\\n            sickness. Indication not to pop my knuckles or crack my jointsconflict with chiropracty (in progress)</li>\\n    </ul>\\n\\n    <p>\\n        <strong>COMPUTER SOFTWARE</strong>\\n    </p>\\n\\n    <ul>\\n        <li>NodeJS - 10,000+ hours - Expert</li>\\n        <li>LinuxWindowsMacOS - 10,000+ hours - Expert</li>\\n        <li>CC++Makefile - 10,000+ hours - Expert</li>\\n        <li>PHP - 10,000+ hours - Expert</li>\\n        <li>English - 10,000+ hours - Expert</li>\\n        <li>Web-based UX design - 10,000+ hours - Expert</li>\\n        <li>Web-based Validation - 10,000+ hours - Expert</li>\\n        <li>Time Management - 10,000+ hours - Expert (just ask to see my calendar)</li>\\n        <li>Studying Demons - 10,000+ hours - Expert</li>\\n    </ul>\\n\\n    <p>\\n        <strong>ADDITIONAL TRAINING</strong>\\n    </p>\\n\\n    <ul>\\n        <li>120 hours of LinkedIn Learning Management Courses</li>\\n        <li>10,000+ hours studying PhilosophyPsychologyand Theology (I'm like an encyclopedia of barely useful\\n            information)</li>\\n        <li>The full ET experience thanks to Nikola Tesla.</li>\\n    </ul>\\n\\n    <p>\\n        <strong>EDUCATION</strong>\\n    </p>\\n\\n    <p>\\n        BA - Bachelor's of Science in Computer ScienceNorthern Arizona UniversityFlagstaffArizona2010 (GPA 2.5)\\n    </p>\\n\\n    <p>\\n        <strong>HONORS AND AWARDS</strong>\\n    </p>\\n\\n    <p>\\n        None\\n    </p>\\n\\n    <p>\\n        <strong>VOLUNTEER WORK &amp; COMMUNITY INVOLVEMENT</strong>\\n    </p>\\n\\n    <p>\\n        ACM - Association for Computing Machinery<br />\\n        IEEE - Institute of Electronics Engineers\\n    </p>\\n</body>\\n</html>\""
            ]
        }
    ],
    [
        1745296585626,
        "/Users/briancullinan/jupyter_ops/Databases/caches.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "cache chat history with llm descriptions",
                "cache chat history with llm descriptions"
            ]
        }
    ],
    [
        1575743794000,
        "/Users/briancullinan/jupyter_ops/Databases/database commands.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "organize downloads",
                "organize downloads",
                "organize downloads folder using AI"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "categorize home directory",
                "categorize home directory"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "categories downloads",
                "categories downloads"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "download reorganize bash",
                "download reorganize bash"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "bash make categories",
                "bash make categories"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "run bash on downloads",
                "run bash on downloads"
            ]
        }
    ],
    [
        1744677922535,
        "/Users/briancullinan/jupyter_ops/Databases/filesystem.ipynb[6]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "macos time added",
                "macos time added"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "git discard whitespace",
                "git discard whitespace"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "compare git branches",
                "compare git branches"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "interesting git commands",
                "interesting git commands"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "cache git credentials",
                "cache git credentials"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "git auto commit",
                "git auto commit"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "git auto rebase",
                "git auto rebase"
            ]
        }
    ],
    [
        1581524399000,
        "/Users/briancullinan/jupyter_ops/Databases/git commands.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "git add submodule",
                "git add submodule"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "git stylesheet",
                "git stylesheet"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "git service index",
                "git service index"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[10]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "git tag fragment",
                "git tag fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[11]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "html code view",
                "html code view"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[12]",
        {
            "from": 28,
            "to": 30,
            "questions": [
                "github service config",
                "github service config"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[13]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                "git service branches",
                "git service branches"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[14]",
        {
            "from": 33,
            "to": 34,
            "questions": [
                "git branches",
                "git branches"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[15]",
        {
            "from": 35,
            "to": 36,
            "questions": [
                "git serve bare",
                "git serve bare"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[16]",
        {
            "from": 37,
            "to": 38,
            "questions": [
                "git serve repo list",
                "git serve repo list"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[17]",
        {
            "from": 39,
            "to": 40,
            "questions": [
                "git serve tree",
                "git serve tree"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[18]",
        {
            "from": 41,
            "to": 42,
            "questions": [
                "git serve blob",
                "git serve blob"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[19]",
        {
            "from": 43,
            "to": 44,
            "questions": [
                "determine language",
                "determine language"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[2]",
        {
            "from": 6,
            "to": 8,
            "questions": [
                "repositories fragment",
                "repositories fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[20]",
        {
            "from": 45,
            "to": 46,
            "questions": [
                "git serve commits",
                "git serve commits"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "file list fragment",
                "file list fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "branch list fragment",
                "branch list fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "commit list fragment",
                "commit list fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "git tag list",
                "git tag list"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[7]",
        {
            "from": 17,
            "to": 19,
            "questions": [
                "repo fragment",
                "repo fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[8]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "branch fragment",
                "branch fragment"
            ]
        }
    ],
    [
        1745090804038,
        "/Users/briancullinan/jupyter_ops/Databases/git index.ipynb[9]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "commit fragment",
                "commit fragment"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Rename a bunch of files using Git and renamer",
                "Rename a bunch of files using Git and renamer"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test github renamer",
                "test github renamer"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "json git tree",
                "json git tree"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "source tree",
                "source tree",
                "tip git tree",
                "just the tip"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Display graphs of different commands and scenarios using dry-run",
                "Display graphs of different commands and scenarios using dry-run"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "update git",
                "update git"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "github updates",
                "github updates",
                "check github for commits since the current one"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[7]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "git project directory",
                "git project directory",
                "get fresh git project directory",
                "Interface for searching all TODO: items  Calendar",
                "Use unit testing to warn about unnecessary modules",
                "Rebase squash cherrypick automatically to create perfect feature branch",
                "Reset each file one at a time to make sure there aren't any unnecessary changes",
                "Plot when a TODO: item is added and when it is removed on Google Calendarthen verify by checking commit -messagesremaining comments/function names with matching keywords (split by camel casing)plot dates completed for measurement of self set tasks and short term goalswhich activities are harder than others  better insight in to erroneous developer time estimates"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[8]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "glob git branch",
                "glob git branch"
            ]
        }
    ],
    [
        1602138669000,
        "/Users/briancullinan/jupyter_ops/Databases/git.ipynb[9]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "convert git history to google calendar events",
                "convert git history to google calendar events"
            ]
        }
    ],
    [
        1744599745087,
        "/Users/briancullinan/jupyter_ops/Databases/mongodb basics.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599745087,
        "/Users/briancullinan/jupyter_ops/Databases/mongodb basics.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599745087,
        "/Users/briancullinan/jupyter_ops/Databases/mongodb basics.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599745087,
        "/Users/briancullinan/jupyter_ops/Databases/mongodb basics.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744599745087,
        "/Users/briancullinan/jupyter_ops/Databases/mongodb basics.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559873381000,
        "/Users/briancullinan/jupyter_ops/Databases/npm.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Replace package.json latest with actual latest version numbers",
                "Replace package.json latest with actual latest version numbers"
            ]
        }
    ],
    [
        1559873381000,
        "/Users/briancullinan/jupyter_ops/Databases/npm.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Check if there is a local-npm server running on Brian's machine",
                "Check if there is a local-npm server running on Brian's machine"
            ]
        }
    ],
    [
        1559873381000,
        "/Users/briancullinan/jupyter_ops/Databases/npm.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Run NPM in javscript with in-memory file-system",
                "Run NPM in javscript with in-memory file-system"
            ]
        }
    ],
    [
        1559873381000,
        "/Users/briancullinan/jupyter_ops/Databases/npm.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Databases/orm comparison.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Databases/postgresql setup.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1576096475000,
        "/Users/briancullinan/jupyter_ops/Databases/raid.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Databases/registry.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Databases/sql queries.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511998987000,
        "/Users/briancullinan/jupyter_ops/Databases/sql to json.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511998987000,
        "/Users/briancullinan/jupyter_ops/Databases/sql to json.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511998987000,
        "/Users/briancullinan/jupyter_ops/Databases/sql to json.ipynb[2]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1575755674000,
        "/Users/briancullinan/jupyter_ops/Databases/sqlite node.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1575755674000,
        "/Users/briancullinan/jupyter_ops/Databases/sqlite node.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1575755674000,
        "/Users/briancullinan/jupyter_ops/Databases/sqlite node.ipynb[2]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[1]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[10]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[11]",
        {
            "from": 24,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[12]",
        {
            "from": 25,
            "to": 25,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[13]",
        {
            "from": 26,
            "to": 26,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[14]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                "Install build stack on colab",
                "Install build stack on colab"
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[15]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                "update notebook repo on colab",
                "update notebook repo on colab"
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[3]",
        {
            "from": 5,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[4]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[5]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[6]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[7]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[8]",
        {
            "from": 17,
            "to": 20,
            "questions": [
                "use Docker",
                "use Docker",
                "Markdown cell: What question does this code set out to answer",
                "Code cell: A short function with any necessary definition functions"
            ]
        }
    ],
    [
        1562088110000,
        "/Users/briancullinan/jupyter_ops/demo.ipynb[9]",
        {
            "from": 21,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Docker/containers vs vms.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use Docker",
                "use Docker",
                "Build a docker image"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Restart the docker service",
                "Restart the docker service"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Actually delete everything",
                "Actually delete everything"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Delete all containers",
                "Delete all containers"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Delete all images",
                "Delete all images"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "Delete all images in Powershell",
                "Delete all images in Powershell"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "Delete containers in cmd",
                "Delete containers in cmd"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "Delete images in cmd",
                "Delete images in cmd"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "Delete danglings images in cmd",
                "Delete danglings images in cmd"
            ]
        }
    ],
    [
        1619335432000,
        "/Users/briancullinan/jupyter_ops/Docker/docker commands.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Docker/docker demo.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "What is Docker",
                "What is Docker"
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Docker/docker demo.ipynb[1]",
        {
            "from": 2,
            "to": 5,
            "questions": [
                "selenium docker",
                "selenium docker"
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Docker/docker security.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Docker/docker-compose basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510894402000,
        "/Users/briancullinan/jupyter_ops/Docker/dotnet.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Mac.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install Docker on Mac",
                "install Docker on Mac",
                "Is Docker already installed"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Mac.ipynb[1]",
        {
            "from": 2,
            "to": 4,
            "questions": [
                "Where do I download dockerWhat other tools do I need",
                "Where do I download dockerWhat other tools do I need"
            ]
        }
    ],
    [
        1515456289000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Windows.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install Docker on Windows",
                "install Docker on Windows",
                "Is Docker already installed"
            ]
        }
    ],
    [
        1515456289000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Windows.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "How do I installed elevated from the command line",
                "How do I installed elevated from the command line"
            ]
        }
    ],
    [
        1515456289000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Windows.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Where do I download the Windows Docker installer",
                "Where do I download the Windows Docker installer"
            ]
        }
    ],
    [
        1515456289000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Windows.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "How do I fix the firewall for Docker",
                "How do I fix the firewall for Docker",
                "How do I install elevated using msiexec"
            ]
        }
    ],
    [
        1515456289000,
        "/Users/briancullinan/jupyter_ops/Docker/install Docker on Windows.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "install build tools on Windows",
                "install build tools on Windows"
            ]
        }
    ],
    [
        1511994770000,
        "/Users/briancullinan/jupyter_ops/Docker/node install.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use nodejs",
                "use nodejs",
                "install node on Linux"
            ]
        }
    ],
    [
        1511994770000,
        "/Users/briancullinan/jupyter_ops/Docker/node install.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Install dev tools on Linux",
                "Install dev tools on Linux"
            ]
        }
    ],
    [
        1511994770000,
        "/Users/briancullinan/jupyter_ops/Docker/node install.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Docker/volume management.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Exercises/data structures.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[2]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[3]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[4]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[5]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561561527000,
        "/Users/briancullinan/jupyter_ops/Exercises/falsey javascript.ipynb[6]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562109499000,
        "/Users/briancullinan/jupyter_ops/Exercises/fizz buzz.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "http://wiki.c2.com/FizzBuzzTest",
                "http://wiki.c2.com/FizzBuzzTest"
            ]
        }
    ],
    [
        1562109499000,
        "/Users/briancullinan/jupyter_ops/Exercises/fizz buzz.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562109499000,
        "/Users/briancullinan/jupyter_ops/Exercises/fizz buzz.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562109499000,
        "/Users/briancullinan/jupyter_ops/Exercises/fizz buzz.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561399827000,
        "/Users/briancullinan/jupyter_ops/Exercises/math.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561399827000,
        "/Users/briancullinan/jupyter_ops/Exercises/math.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[10]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[11]",
        {
            "from": 24,
            "to": 26,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[12]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[13]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[6]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[7]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[8]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563036424000,
        "/Users/briancullinan/jupyter_ops/Exercises/python codekatas solutions.ipynb[9]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Exercises/recursion examples.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Exercises/string manipulation.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[0]",
        {
            "from": 0,
            "to": 9,
            "questions": [
                "What is Angular 2",
                "What is Angular 2",
                "How has Angular changed the architecture of software design"
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[1]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[2]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[3]",
        {
            "from": 13,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[4]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513056858000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular 2.ipynb[5]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Display search notebook component",
                "Display search notebook component"
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Search notebook service",
                "Search notebook service"
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Display code results",
                "Display code results"
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Sockify search notebook provider",
                "Sockify search notebook provider"
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[4]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[5]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "Display login form",
                "Display login form"
            ]
        }
    ],
    [
        1744155297726,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/Angular components.ipynb[6]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "Authentication service",
                "Authentication service"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/angular core modules.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Useful list of material and core modules",
                "Useful list of material and core modules"
            ]
        }
    ],
    [
        1561487089000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/angular render service.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "render Angular modules instead of using express",
                "render Angular modules instead of using express"
            ]
        }
    ],
    [
        1561487089000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/angular render service.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/angular-cli.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/angular-cli.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513119086000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/build Angular components.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Run angular-cli commands from nodejs",
                "Run angular-cli commands from nodejs",
                "build Angular components using nodejs",
                "Add a component to the specified project using angular-cli"
            ]
        }
    ],
    [
        1513119086000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/build Angular components.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Compile the project using webpack",
                "Compile the project using webpack"
            ]
        }
    ],
    [
        1513119086000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/build Angular components.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513119086000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/build Angular components.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "Set angular theme colors",
                "Set angular theme colors",
                "run angular project from memory",
                "- more debuggable ES5 sourcemaps"
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Include the script in the response",
                "Include the script in the response"
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Display angular modules in jupyter",
                "Display angular modules in jupyter"
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[4]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[5]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561487153000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Angular/display angular.ipynb[6]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/autorewire.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install autorewire",
                "install autorewire"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/autorewire.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "mock all properties and functions using rewire",
                "mock all properties and functions using rewire"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/autorewire.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1575774788000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3 dates.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "d3 swimlane",
                "d3 swimlane"
            ]
        }
    ],
    [
        1575774788000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3 dates.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "d3 calendar",
                "d3 calendar"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Create a word-cloud",
                "Create a word-cloud"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "d3 tiered pie chart",
                "d3 tiered pie chart"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "d3 pie chart",
                "d3 pie chart"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[4]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "Format d3 tree",
                "Format d3 tree"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[5]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "Display d3 tree",
                "Display d3 tree"
            ]
        }
    ],
    [
        1557605560000,
        "/Users/briancullinan/jupyter_ops/Frameworks/d3.ipynb[6]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "delint notebooks",
                "delint notebooks"
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "lint notebook",
                "lint notebook"
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "delint specific cells",
                "delint specific cells"
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "notebook lint watcher",
                "notebook lint watcher"
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "test de-linting service",
                "test de-linting service"
            ]
        }
    ],
    [
        1744493414944,
        "/Users/briancullinan/jupyter_ops/Frameworks/de-linting.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "delint using webstorm",
                "delint using webstorm"
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Frameworks/django overview.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Frameworks/flask basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "test repository",
                "test repository",
                "What repositories do we need"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "selenium repository",
                "selenium repository"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "project repository",
                "project repository"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[3]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "Setup git credentials in docker",
                "Setup git credentials in docker"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "use in Docker",
                "use in Docker"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "use npm cache inside of docker",
                "use npm cache inside of docker"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "run the Docker image",
                "run the Docker image"
            ]
        }
    ],
    [
        1573841251000,
        "/Users/briancullinan/jupyter_ops/Frameworks/git repos.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "Selenium test scripts",
                "Selenium test scripts"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use gulp",
                "use gulp"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "gulp tasks to promise",
                "gulp tasks to promise",
                "use a Promise as a gulp task",
                "convert a gulp task to a Promise"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "Apply universal to angular project",
                "Apply universal to angular project"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[4]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "Fix project relatives as a service for updating files when they change",
                "Fix project relatives as a service for updating files when they change"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[5]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "search notebooks gulp",
                "search notebooks gulp",
                "Search notebooks for a string using gulp"
            ]
        }
    ],
    [
        1744736148500,
        "/Users/briancullinan/jupyter_ops/Frameworks/gulp.ipynb[6]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/How code should look.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "How code should look",
                "How code should look",
                "How do I want code to look"
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[2]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "set up identity server",
                "set up identity server"
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[4]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744767762262,
        "/Users/briancullinan/jupyter_ops/Frameworks/identity server.ipynb[5]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/jupyter interaction.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Interactive cells in juypter",
                "Interactive cells in juypter"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install bash kernel",
                "install bash kernel",
                "install extra kernels in jupyter"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "install ipython kernel",
                "install ipython kernel"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "install ICSharp (C) kernel",
                "install ICSharp (C) kernel"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[4]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[5]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "install ruby",
                "install ruby"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[6]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "install Powershell",
                "install Powershell"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[7]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/Jupyter language kernels.ipynb[8]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563206693000,
        "/Users/briancullinan/jupyter_ops/Frameworks/jupyter.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738621455428,
        "/Users/briancullinan/jupyter_ops/Frameworks/jwt in csharp.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use JSON web tokens in C",
                "use JSON web tokens in C"
            ]
        }
    ],
    [
        1738621455428,
        "/Users/briancullinan/jupyter_ops/Frameworks/jwt in csharp.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/karma template.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Generate a bunch of spec files",
                "Generate a bunch of spec files"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/karma template.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Frameworks/nextjs setup.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1514776662000,
        "/Users/briancullinan/jupyter_ops/Frameworks/ngx-translate.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1514776662000,
        "/Users/briancullinan/jupyter_ops/Frameworks/ngx-translate.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Find unused/misplaced translation strings",
                "Find unused/misplaced translation strings"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "express service",
                "express service",
                "express website",
                "use nodejs express"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "import new service",
                "import new service"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[10]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "run a DOM query on a remote HTML page",
                "run a DOM query on a remote HTML page"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[11]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "save a list of image elements",
                "save a list of image elements",
                "display a list of image elements in markdown",
                "{output} image elements in the requested {format}"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[12]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "stop express server",
                "stop express server"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[13]",
        {
            "from": 28,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[14]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                "zuora eloqua express mock",
                "zuora eloqua express mock"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[15]",
        {
            "from": 33,
            "to": 35,
            "questions": [
                "directory to html",
                "directory to html"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[16]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                "node express directory handler",
                "node express directory handler"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[17]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                "html music player",
                "html music player"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[18]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                "html image viewer",
                "html image viewer"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[19]",
        {
            "from": 42,
            "to": 43,
            "questions": [
                "rpc index",
                "rpc index"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "websocket subscription service",
                "websocket subscription service"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[20]",
        {
            "from": 44,
            "to": 45,
            "questions": [
                "rpc list fragement",
                "rpc list fragement"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[21]",
        {
            "from": 46,
            "to": 47,
            "questions": [
                "rpc fragement",
                "rpc fragement"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[22]",
        {
            "from": 48,
            "to": 49,
            "questions": [
                "rpc service index",
                "rpc service index"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[23]",
        {
            "from": 50,
            "to": 51,
            "questions": [
                "node express styles",
                "node express styles"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "express rpc service",
                "express rpc service"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "browser socket service",
                "browser socket service"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "reliable-websocket.js",
                "reliable-websocket.js"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[6]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "express router",
                "express router"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[7]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "parse environment",
                "parse environment"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[8]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "start a proxy on {port}",
                "start a proxy on {port}",
                "start a proxy using express"
            ]
        }
    ],
    [
        1745388616419,
        "/Users/briancullinan/jupyter_ops/Frameworks/node express.ipynb[9]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "display the output from express",
                "display the output from express",
                "test express is working with HTML"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "node imap client",
                "node imap client",
                "use nodejs simple-imap",
                "temporarily allow less secure apps",
                "connect to Gmail using simple-imap",
                "log in to Gmail using imap-simple/node IMAP",
                "How do I connect using simple-imap and {credentials}"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "search for messages using simple-imap",
                "search for messages using simple-imap",
                "search for messages in the last few {days}",
                "search for messages {from} a sender's email address"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "test search messages imap",
                "test search messages imap"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[3]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "scan commands email",
                "scan commands email"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[4]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "list the subjects from simple-imap messages",
                "list the subjects from simple-imap messages",
                "list the sender's email address from simple-imap messages"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[5]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "get attachments using simple-imap",
                "get attachments using simple-imap",
                "save an attachments from simple-imap",
                "get messages {from} and email address",
                "get attachements from the last few {days}",
                "extract a list of image attachments from an email",
                "{output} all attachments in the requested {format}"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[6]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "send email",
                "send email"
            ]
        }
    ],
    [
        1578449241000,
        "/Users/briancullinan/jupyter_ops/Frameworks/node simple-imap.ipynb[7]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561923854000,
        "/Users/briancullinan/jupyter_ops/Frameworks/notebook extensions.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "notebook extension",
                "notebook extension",
                "TODO: connect output to jupyter kernelusing display mime types possibly"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "mock and pass through",
                "mock and pass through",
                "install promisify autorewire socketio notebook"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Create an RPC socker.io server for any module using promisify",
                "Create an RPC socker.io server for any module using promisify"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Mock any module with promisify and socket.io-client",
                "Mock any module with promisify and socket.io-client"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Check if sockify server is running",
                "Check if sockify server is running"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[4]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/promisify automock socketio.ipynb[5]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Frameworks/react hooks.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Frameworks/register jupter kernels.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "install a kernel from just the spec",
                "install a kernel from just the spec"
            ]
        }
    ],
    [
        1573599809000,
        "/Users/briancullinan/jupyter_ops/Frameworks/svn commands.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "test cells",
                "test cells"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "mocha test the test cell runner",
                "mocha test the test cell runner"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[10]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "color-fader.js",
                "color-fader.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[11]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                "update-view.js",
                "update-view.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[12]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                "render-remote.js",
                "render-remote.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[13]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                "animate-nodes.js",
                "animate-nodes.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[14]",
        {
            "from": 31,
            "to": 33,
            "questions": [
                "cell nodes",
                "cell nodes"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[15]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                "git repo nodes",
                "git repo nodes"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[16]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                "match language to grammar",
                "match language to grammar"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[17]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                "git service list",
                "git service list"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "test runner",
                "test runner",
                "watch files run tests",
                "TODO: restart this test script every loop fork and abandon current thread",
                "TODO: git apply without whitespacereset the rest  separate index  how do make git-scenario app",
                "TODO: re-import cells on Utilities/.modules folder change. run all \"watcher\" commands from a test below  mocha.grep"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[3]",
        {
            "from": 7,
            "to": 9,
            "questions": [
                "test sigma index",
                "test sigma index"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[4]",
        {
            "from": 10,
            "to": 12,
            "questions": [
                "test sigma template",
                "test sigma template"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "notebook-colors.js",
                "notebook-colors.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "sigma-style.css",
                "sigma-style.css"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "sigma-script.js",
                "sigma-script.js"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "trace nodes",
                "trace nodes"
            ]
        }
    ],
    [
        1745253146366,
        "/Users/briancullinan/jupyter_ops/Frameworks/test runner.ipynb[9]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "highlight-nodes.js",
                "highlight-nodes.js"
            ]
        }
    ],
    [
        1557546362000,
        "/Users/briancullinan/jupyter_ops/Frameworks/twilio.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "accept incoming twilio message",
                "accept incoming twilio message"
            ]
        }
    ],
    [
        1557546362000,
        "/Users/briancullinan/jupyter_ops/Frameworks/twilio.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "twilio reminder",
                "twilio reminder",
                "send me a reminder in a few minutes"
            ]
        }
    ],
    [
        1557546362000,
        "/Users/briancullinan/jupyter_ops/Frameworks/twilio.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "send a twilio message",
                "send a twilio message"
            ]
        }
    ],
    [
        1557546362000,
        "/Users/briancullinan/jupyter_ops/Frameworks/twilio.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "make a twilio call",
                "make a twilio call"
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Frameworks/vite vs webpack.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "zuora export service",
                "zuora export service"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "zuora export catalog",
                "zuora export catalog"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[10]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "eloqua import create template",
                "eloqua import create template"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[11]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "test eloqua import create template",
                "test eloqua import create template"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[12]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "eloqua import blueprints",
                "eloqua import blueprints"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[13]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "eloqua existing import",
                "eloqua existing import"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[14]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "test eloqua existing import",
                "test eloqua existing import"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[15]",
        {
            "from": 30,
            "to": 31,
            "questions": [
                "aws entry point",
                "aws entry point"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[16]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                "test aws entry point",
                "test aws entry point"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[17]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                "notify entry point",
                "notify entry point"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[18]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                "test notify entry point",
                "test notify entry point"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[19]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                "zuora export month",
                "zuora export month"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "zuora export service test",
                "zuora export service test"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[20]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                "test zuora export month",
                "test zuora export month"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[21]",
        {
            "from": 42,
            "to": 43,
            "questions": [
                "zuora account service",
                "zuora account service"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[22]",
        {
            "from": 44,
            "to": 45,
            "questions": [
                "test zuora account service",
                "test zuora account service"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[23]",
        {
            "from": 46,
            "to": 47,
            "questions": [
                "bulk upload eloqua",
                "bulk upload eloqua"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[24]",
        {
            "from": 48,
            "to": 49,
            "questions": [
                "test bulk upload eloqua",
                "test bulk upload eloqua"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[25]",
        {
            "from": 50,
            "to": 51,
            "questions": [
                "sync zuora eloqua end to end",
                "sync zuora eloqua end to end"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[26]",
        {
            "from": 52,
            "to": 53,
            "questions": [
                "calculate price",
                "calculate price"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[27]",
        {
            "from": 54,
            "to": 55,
            "questions": [
                "calculate price test",
                "calculate price test",
                "// TODO:  handle australia",
                "// TODO:  handle not in worksheet"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[28]",
        {
            "from": 56,
            "to": 57,
            "questions": [
                "readme",
                "readme",
                "Create an access key for AWS: https://console.aws.amazon.com/iam/homenc2=h_m_sc/security_credential"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[29]",
        {
            "from": 58,
            "to": 59,
            "questions": [
                ""
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "zuora renewals query",
                "zuora renewals query"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[30]",
        {
            "from": 60,
            "to": 61,
            "questions": [
                ""
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[31]",
        {
            "from": 62,
            "to": 62,
            "questions": [
                ""
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[32]",
        {
            "from": 63,
            "to": 63,
            "questions": [
                ""
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "test zuora renewals query",
                "test zuora renewals query"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "eloqua import service",
                "eloqua import service"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "test eloqua import service",
                "test eloqua import service"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "zuora eloqua mapper",
                "zuora eloqua mapper"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "zuora eloqua mapper test",
                "zuora eloqua mapper test"
            ]
        }
    ],
    [
        1652316506000,
        "/Users/briancullinan/jupyter_ops/Frameworks/zuora to eloqua.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "zuora account blueprints",
                "zuora account blueprints"
            ]
        }
    ],
    [
        1574310758000,
        "/Users/briancullinan/jupyter_ops/Frontends/blog.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1574310758000,
        "/Users/briancullinan/jupyter_ops/Frontends/blog.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "get a list of folder topics from bookmarks",
                "get a list of folder topics from bookmarks"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "update insterests resume sheet",
                "update insterests resume sheet"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[10]",
        {
            "from": 23,
            "to": 25,
            "questions": [
                "resume chat program",
                "resume chat program"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[11]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "chat page",
                "chat page"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[12]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "brians resume in html",
                "brians resume in html"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[2]",
        {
            "from": 6,
            "to": 8,
            "questions": [
                "load message history",
                "load message history"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "brians resume",
                "brians resume"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "file system access",
                "file system access"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "access web information",
                "access web information"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "ask another llm for help",
                "ask another llm for help"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "render message history",
                "render message history"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "resume express chat service",
                "resume express chat service"
            ]
        }
    ],
    [
        1744492567579,
        "/Users/briancullinan/jupyter_ops/Frontends/brians resume.ipynb[9]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "render history navigation",
                "render history navigation"
            ]
        }
    ],
    [
        1744669359576,
        "/Users/briancullinan/jupyter_ops/Frontends/calendar.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "frequency calendar index",
                "frequency calendar index"
            ]
        }
    ],
    [
        1744669359576,
        "/Users/briancullinan/jupyter_ops/Frontends/calendar.ipynb[1]",
        {
            "from": 2,
            "to": 4,
            "questions": [
                "calendar frequency timeline",
                "calendar frequency timeline"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "readme.md",
                "readme.md"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[1]",
        {
            "from": 5,
            "to": 8,
            "questions": [
                "get sheet purchases",
                "get sheet purchases",
                "google sheet handler",
                "get sheet identifier from link"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[10]",
        {
            "from": 35,
            "to": 36,
            "questions": [
                "sheet to web",
                "sheet to web",
                "test google sheets resources"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[11]",
        {
            "from": 37,
            "to": 39,
            "questions": [
                "convert sheet helper functions",
                "convert sheet helper functions"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[12]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                "package.json",
                "package.json"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[2]",
        {
            "from": 9,
            "to": 11,
            "questions": [
                "sheet marketing import",
                "sheet marketing import",
                "sheet marketing import handler"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[3]",
        {
            "from": 12,
            "to": 15,
            "questions": [
                "filter data sheet based on url",
                "filter data sheet based on url"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[4]",
        {
            "from": 16,
            "to": 18,
            "questions": [
                "google sheet template properties",
                "google sheet template properties"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[5]",
        {
            "from": 19,
            "to": 21,
            "questions": [
                "google sheet layout template",
                "google sheet layout template"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[6]",
        {
            "from": 22,
            "to": 24,
            "questions": [
                "output google sheet template",
                "output google sheet template"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[7]",
        {
            "from": 25,
            "to": 28,
            "questions": [
                "find known routes to sheets",
                "find known routes to sheets"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[8]",
        {
            "from": 29,
            "to": 31,
            "questions": [
                "collect external content and resources",
                "collect external content and resources"
            ]
        }
    ],
    [
        1738640531491,
        "/Users/briancullinan/jupyter_ops/Frontends/convert spreadsheet.ipynb[9]",
        {
            "from": 32,
            "to": 34,
            "questions": [
                "collect google sheets resources",
                "collect google sheets resources"
            ]
        }
    ],
    [
        1744505629643,
        "/Users/briancullinan/jupyter_ops/Frontends/discord activities.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "discord client code",
                "discord client code"
            ]
        }
    ],
    [
        1744505629643,
        "/Users/briancullinan/jupyter_ops/Frontends/discord activities.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "discord client auth code",
                "discord client auth code"
            ]
        }
    ],
    [
        1744505629643,
        "/Users/briancullinan/jupyter_ops/Frontends/discord activities.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "start a bunch of discord services",
                "start a bunch of discord services"
            ]
        }
    ],
    [
        1744505629643,
        "/Users/briancullinan/jupyter_ops/Frontends/discord activities.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "discord authenticate instances",
                "discord authenticate instances"
            ]
        }
    ],
    [
        1744505629643,
        "/Users/briancullinan/jupyter_ops/Frontends/discord activities.ipynb[4]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "discord express token endpoint",
                "discord express token endpoint"
            ]
        }
    ],
    [
        1738469658616,
        "/Users/briancullinan/jupyter_ops/Frontends/discord games.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738469658616,
        "/Users/briancullinan/jupyter_ops/Frontends/discord games.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "discord game commands",
                "discord game commands"
            ]
        }
    ],
    [
        1744505856291,
        "/Users/briancullinan/jupyter_ops/Frontends/discord imagine.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "update discord interaction with attachments",
                "update discord interaction with attachments"
            ]
        }
    ],
    [
        1744505856291,
        "/Users/briancullinan/jupyter_ops/Frontends/discord imagine.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "discord stability proxy",
                "discord stability proxy"
            ]
        }
    ],
    [
        1744505856291,
        "/Users/briancullinan/jupyter_ops/Frontends/discord imagine.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "discord mid journey commands",
                "discord mid journey commands"
            ]
        }
    ],
    [
        1744505856291,
        "/Users/briancullinan/jupyter_ops/Frontends/discord imagine.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "discord ollama vision",
                "discord ollama vision"
            ]
        }
    ],
    [
        1744777244241,
        "/Users/briancullinan/jupyter_ops/Frontends/discord player.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "discord music player",
                "discord music player"
            ]
        }
    ],
    [
        1744777244241,
        "/Users/briancullinan/jupyter_ops/Frontends/discord player.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "discord music player server",
                "discord music player server"
            ]
        }
    ],
    [
        1744505665676,
        "/Users/briancullinan/jupyter_ops/Frontends/discord remote.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "discord remote control",
                "discord remote control"
            ]
        }
    ],
    [
        1744505665676,
        "/Users/briancullinan/jupyter_ops/Frontends/discord remote.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "discord remote proxy server",
                "discord remote proxy server"
            ]
        }
    ],
    [
        1744505665676,
        "/Users/briancullinan/jupyter_ops/Frontends/discord remote.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "express automation routes",
                "express automation routes"
            ]
        }
    ],
    [
        1744505665676,
        "/Users/briancullinan/jupyter_ops/Frontends/discord remote.ipynb[3]",
        {
            "from": 11,
            "to": 14,
            "questions": [
                "remote desktop client",
                "remote desktop client"
            ]
        }
    ],
    [
        1744505665676,
        "/Users/briancullinan/jupyter_ops/Frontends/discord remote.ipynb[4]",
        {
            "from": 15,
            "to": 17,
            "questions": [
                "client input remote code",
                "client input remote code"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "discord pdf converter",
                "discord pdf converter"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "discord notebook connector",
                "discord notebook connector"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "discord llm connector",
                "discord llm connector"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "discord writing llms",
                "discord writing llms"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "discord mesh generator",
                "discord mesh generator"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "discord llm interactions",
                "discord llm interactions"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[6]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "sync discord llm tools",
                "sync discord llm tools"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "sync deceptive chat",
                "sync deceptive chat"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "discord handy tools",
                "discord handy tools"
            ]
        }
    ],
    [
        1740329365088,
        "/Users/briancullinan/jupyter_ops/Frontends/discord tools.ipynb[9]",
        {
            "from": 21,
            "to": 23,
            "questions": [
                "create discord message with attachments",
                "create discord message with attachments"
            ]
        }
    ],
    [
        1740949966035,
        "/Users/briancullinan/jupyter_ops/Frontends/discord voice.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "send voice on discord",
                "send voice on discord"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "discord request",
                "discord request",
                "authorize discord",
                "discord authorize",
                "authorize and connect to discord",
                "Why do this At the time the nodejs discord API only supported the prior version and I wanted new command featuresso I reimplemented it over REST like the documentation instructs. NowI believe the project has caught up.",
                "https://discord.com/oauth2/authorizeresponse_type=code&client_id=1335769252409380884&scope=applications.commands%20bot%20guilds%20guilds.join%20email%20identify%20messages.read&permissions=54596725809&redirect_uri=https://briancullinan.com"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "discord gateway",
                "discord gateway"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[10]",
        {
            "from": 33,
            "to": 36,
            "questions": [
                "discord utilities",
                "discord utilities"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[11]",
        {
            "from": 37,
            "to": 39,
            "questions": [
                "delete all commands",
                "delete all commands"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "discord bot configuration",
                "discord bot configuration"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "discord api",
                "discord api"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[4]",
        {
            "from": 14,
            "to": 17,
            "questions": [
                "discord messages",
                "discord messages"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[5]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "discord channels",
                "discord channels"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[6]",
        {
            "from": 21,
            "to": 23,
            "questions": [
                "discord commands",
                "discord commands"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[7]",
        {
            "from": 24,
            "to": 26,
            "questions": [
                "discord guilds",
                "discord guilds"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[8]",
        {
            "from": 27,
            "to": 29,
            "questions": [
                "discord threads",
                "discord threads"
            ]
        }
    ],
    [
        1744777071197,
        "/Users/briancullinan/jupyter_ops/Frontends/discord.ipynb[9]",
        {
            "from": 30,
            "to": 32,
            "questions": [
                "discord users",
                "discord users"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "convert notebook to open api",
                "convert notebook to open api"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test convert notebook api",
                "test convert notebook api"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[10]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "javadoc template",
                "javadoc template"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[11]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                "documentation stylesheet",
                "documentation stylesheet"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "convert google discovery to documentation",
                "convert google discovery to documentation",
                "https://developers.google.com/apis-explorer/p/discovery/v1/discovery.apis.list_h=1&",
                "https://developers.google.com/apis-explorer/s/discovery/v1/discovery.apis.getRestapi=discovery&version=v1&_h=2&"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "discover well-known apis",
                "discover well-known apis"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[4]",
        {
            "from": 10,
            "to": 12,
            "questions": [
                "export documentation",
                "export documentation"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "docs mustache template",
                "docs mustache template"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "code fix render template",
                "code fix render template"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[7]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "render template pass through",
                "render template pass through"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[8]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                "render notebook mustache",
                "render notebook mustache"
            ]
        }
    ],
    [
        1745124366540,
        "/Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[9]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "highlight js",
                "highlight js"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[0]",
        {
            "from": 0,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[1]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "read gist files",
                "read gist files"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[10]",
        {
            "from": 34,
            "to": 36,
            "questions": [
                "read crawl files",
                "read crawl files",
                "get scraped page"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[11]",
        {
            "from": 37,
            "to": 38,
            "questions": [
                "ckeditor configuration",
                "ckeditor configuration"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[12]",
        {
            "from": 39,
            "to": 40,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[2]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[3]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "write gist files",
                "write gist files"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[4]",
        {
            "from": 15,
            "to": 17,
            "questions": [
                "save git",
                "save git",
                "git save"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[5]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "git file tree",
                "git file tree"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[6]",
        {
            "from": 21,
            "to": 24,
            "questions": [
                "apply acl to html",
                "apply acl to html"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[7]",
        {
            "from": 25,
            "to": 27,
            "questions": [
                "load ckeditor",
                "load ckeditor"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[8]",
        {
            "from": 28,
            "to": 31,
            "questions": [
                "scope css",
                "scope css",
                "restrain css"
            ]
        }
    ],
    [
        1649474240000,
        "/Users/briancullinan/jupyter_ops/Frontends/edit anywhere.ipynb[9]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745206216828,
        "/Users/briancullinan/jupyter_ops/Frontends/frontends.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "start a bunch of front end services",
                "start a bunch of front end services"
            ]
        }
    ],
    [
        1745206216828,
        "/Users/briancullinan/jupyter_ops/Frontends/frontends.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745270880895,
        "/Users/briancullinan/jupyter_ops/Frontends/grafana.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "grafana llm service",
                "grafana llm service"
            ]
        }
    ],
    [
        1745270880895,
        "/Users/briancullinan/jupyter_ops/Frontends/grafana.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "show grafana metrics",
                "show grafana metrics"
            ]
        }
    ],
    [
        1745270880895,
        "/Users/briancullinan/jupyter_ops/Frontends/grafana.ipynb[2]",
        {
            "from": 4,
            "to": 6,
            "questions": [
                "start a bunch of grafana services",
                "start a bunch of grafana services"
            ]
        }
    ],
    [
        1745270880895,
        "/Users/briancullinan/jupyter_ops/Frontends/grafana.ipynb[3]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "linkedin stylesheet",
                "linkedin stylesheet"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "linkedin network stylesheet",
                "linkedin network stylesheet"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[10]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "linkedin edit jobs template",
                "linkedin edit jobs template"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[11]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "linkedin job list item",
                "linkedin job list item"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[12]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "linkedin job edit item",
                "linkedin job edit item"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[13]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "linkedin profile",
                "linkedin profile"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[14]",
        {
            "from": 30,
            "to": 31,
            "questions": [
                "linkedin education view row",
                "linkedin education view row"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[15]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                "linkedin education edit row",
                "linkedin education edit row"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[16]",
        {
            "from": 34,
            "to": 36,
            "questions": [
                "linkedin experience edit row",
                "linkedin experience edit row"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[17]",
        {
            "from": 37,
            "to": 38,
            "questions": [
                "linkedin experience view row",
                "linkedin experience view row"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[18]",
        {
            "from": 39,
            "to": 40,
            "questions": [
                "linkedin edit",
                "linkedin edit"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[19]",
        {
            "from": 41,
            "to": 43,
            "questions": [
                "linkedin load profile",
                "linkedin load profile"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "linkedin profile stylesheet",
                "linkedin profile stylesheet"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[20]",
        {
            "from": 44,
            "to": 45,
            "questions": [
                "serve linkedin homepage",
                "serve linkedin homepage"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[21]",
        {
            "from": 46,
            "to": 47,
            "questions": [
                "serve linkedin profile",
                "serve linkedin profile"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[22]",
        {
            "from": 48,
            "to": 49,
            "questions": [
                "serve linkedin login",
                "serve linkedin login",
                "linkedin login/register"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[23]",
        {
            "from": 50,
            "to": 51,
            "questions": [
                "verify linkedin email",
                "verify linkedin email"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[24]",
        {
            "from": 52,
            "to": 53,
            "questions": [
                "save linkedin profile",
                "save linkedin profile"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[25]",
        {
            "from": 54,
            "to": 56,
            "questions": [
                "save linkedin pic",
                "save linkedin pic"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[26]",
        {
            "from": 57,
            "to": 58,
            "questions": [
                "linkedin profile blob",
                "linkedin profile blob"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[27]",
        {
            "from": 59,
            "to": 61,
            "questions": [
                "linkedin network",
                "linkedin network"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[28]",
        {
            "from": 62,
            "to": 63,
            "questions": [
                "linkedin jobs",
                "linkedin jobs"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[29]",
        {
            "from": 64,
            "to": 65,
            "questions": [
                "linkedin create",
                "linkedin create"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "linkedin home stylesheet",
                "linkedin home stylesheet"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[4]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "linkedin index",
                "linkedin index"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[5]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "linkedin login page",
                "linkedin login page"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[6]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "linkedin homepage",
                "linkedin homepage"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[7]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "linkedin network html",
                "linkedin network html"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[8]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "linkedin network profile list",
                "linkedin network profile list"
            ]
        }
    ],
    [
        1744769297462,
        "/Users/briancullinan/jupyter_ops/Frontends/linkedin.ipynb[9]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "linkedin jobs template",
                "linkedin jobs template"
            ]
        }
    ],
    [
        1576710442000,
        "/Users/briancullinan/jupyter_ops/Frontends/marketing scripts.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "make a purchase with stripe",
                "make a purchase with stripe"
            ]
        }
    ],
    [
        1576710442000,
        "/Users/briancullinan/jupyter_ops/Frontends/marketing scripts.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "contact us handler",
                "contact us handler"
            ]
        }
    ],
    [
        1576710442000,
        "/Users/briancullinan/jupyter_ops/Frontends/marketing scripts.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "package.json",
                "package.json"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "opengl context",
                "opengl context"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test opengl renderer",
                "test opengl renderer"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test shader toy",
                "test shader toy"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "opengl frame",
                "opengl frame"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "shader toy",
                "shader toy"
            ]
        }
    ],
    [
        1741319518493,
        "/Users/briancullinan/jupyter_ops/Frontends/opengl.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "fragment shader",
                "fragment shader"
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[1]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[2]",
        {
            "from": 6,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[4]",
        {
            "from": 11,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[5]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[6]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[7]",
        {
            "from": 19,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579629974000,
        "/Users/briancullinan/jupyter_ops/Frontends/qemu configs.ipynb[8]",
        {
            "from": 20,
            "to": 20,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740529062865,
        "/Users/briancullinan/jupyter_ops/Frontends/qt.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "qt application",
                "qt application"
            ]
        }
    ],
    [
        1740529062865,
        "/Users/briancullinan/jupyter_ops/Frontends/qt.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test qt application",
                "test qt application"
            ]
        }
    ],
    [
        1740529062865,
        "/Users/briancullinan/jupyter_ops/Frontends/qt.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "qt qml widget",
                "qt qml widget"
            ]
        }
    ],
    [
        1740529062865,
        "/Users/briancullinan/jupyter_ops/Frontends/qt.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Frontends/react components.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744668999354,
        "/Users/briancullinan/jupyter_ops/Frontends/scheduler.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "scheduler index",
                "scheduler index"
            ]
        }
    ],
    [
        1744668999354,
        "/Users/briancullinan/jupyter_ops/Frontends/scheduler.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "schedule stylesheet",
                "schedule stylesheet"
            ]
        }
    ],
    [
        1744668999354,
        "/Users/briancullinan/jupyter_ops/Frontends/scheduler.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "schedule client script",
                "schedule client script"
            ]
        }
    ],
    [
        1744668999354,
        "/Users/briancullinan/jupyter_ops/Frontends/scheduler.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "schedule service",
                "schedule service"
            ]
        }
    ],
    [
        1744668999354,
        "/Users/briancullinan/jupyter_ops/Frontends/scheduler.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "list google calendars",
                "list google calendars"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "convert anki package to study sauce",
                "convert anki package to study sauce"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "render study sauce cards page",
                "render study sauce cards page"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "authorize app to read profile info",
                "authorize app to read profile info"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "receive an authentication code from google",
                "receive an authentication code from google"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "create a study sauce user directory",
                "create a study sauce user directory"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "package.json",
                "package.json"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "study-demo.js",
                "study-demo.js"
            ]
        }
    ],
    [
        1738639719542,
        "/Users/briancullinan/jupyter_ops/Frontends/study sauce.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "study sauce express service",
                "study sauce express service"
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Frontends/svelte basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1742010584948,
        "/Users/briancullinan/jupyter_ops/Frontends/threejs.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "show timeline history",
                "show timeline history"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "timeline template",
                "timeline template"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[10]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "timeline backend tools",
                "timeline backend tools"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[11]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "ask llm to write timeline history",
                "ask llm to write timeline history"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "timeline-styles.css",
                "timeline-styles.css"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "timeline-script.js",
                "timeline-script.js"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "category map controls",
                "category map controls"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "timeline-scales.js",
                "timeline-scales.js"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "timeline-sliders.js",
                "timeline-sliders.js"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "timeline-search.js",
                "timeline-search.js"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "occupations.json",
                "occupations.json"
            ]
        }
    ],
    [
        1745338204705,
        "/Users/briancullinan/jupyter_ops/Frontends/timeline map.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "lat/lon provider",
                "lat/lon provider"
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Frontends/ui libraries overview.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Frontends/vue intro.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "list wikimedia articles",
                "list wikimedia articles"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "load wikimedia index",
                "load wikimedia index"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[10]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "wikiemedia clone index",
                "wikiemedia clone index"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[11]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                "wikimedia-style",
                "wikimedia-style"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "extract wikimedia chunk",
                "extract wikimedia chunk"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "wikimedia-page.html",
                "wikimedia-page.html"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[4]",
        {
            "from": 10,
            "to": 12,
            "questions": [
                "mediawiki text preprocessor",
                "mediawiki text preprocessor"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "match wikitext elements",
                "match wikitext elements"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "coalesce regex tool",
                "coalesce regex tool"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[7]",
        {
            "from": 17,
            "to": 19,
            "questions": [
                "wikimedia mustache template",
                "wikimedia mustache template"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[8]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "wikimedia reflist mustache",
                "wikimedia reflist mustache"
            ]
        }
    ],
    [
        1745430358269,
        "/Users/briancullinan/jupyter_ops/Frontends/wikimedia.ipynb[9]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "wikimedia service",
                "wikimedia service"
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Games/game loop explained.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[10]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[11]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[12]",
        {
            "from": 31,
            "to": 32,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[13]",
        {
            "from": 33,
            "to": 35,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[14]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[15]",
        {
            "from": 38,
            "to": 40,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[16]",
        {
            "from": 41,
            "to": 42,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[2]",
        {
            "from": 6,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "Try streaming the game over VNCis it awful Kind ofand inputs don't work as expected.",
                "Try streaming the game over VNCis it awful Kind ofand inputs don't work as expected."
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[6]",
        {
            "from": 15,
            "to": 18,
            "questions": [
                "https://quake.games/set%20name%20player_name",
                "https://quake.games/set%20name%20player_name",
                "or http://www.quakejs.com/playset%20fs_game%20cpma&set%20mode_start%20FFA&set%20g_teamAutoJoin%201&map%20cpm1a"
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[7]",
        {
            "from": 19,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[8]",
        {
            "from": 22,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019091997,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 commands.ipynb[9]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738019391216,
        "/Users/briancullinan/jupyter_ops/Games/quake 3 model conv.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "convert a model using python and blender",
                "convert a model using python and blender"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[1]",
        {
            "from": 4,
            "to": 6,
            "questions": [
                "find pk3 files in zips",
                "find pk3 files in zips"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[10]",
        {
            "from": 26,
            "to": 28,
            "questions": [
                "scan map graphs",
                "scan map graphs"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[11]",
        {
            "from": 29,
            "to": 31,
            "questions": [
                "convert nonalpha",
                "convert nonalpha"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[12]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                "convert quake 3 audio",
                "convert quake 3 audio"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[13]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                "copy quake 3 script files",
                "copy quake 3 script files"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[14]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                "quake 3 file whitelist",
                "quake 3 file whitelist"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[15]",
        {
            "from": 38,
            "to": 39,
            "questions": [
                "quake 3 menu image list",
                "quake 3 menu image list"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[16]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                "Does TrenchBroom really require everything be in 1 folder",
                "Does TrenchBroom really require everything be in 1 folder"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[17]",
        {
            "from": 42,
            "to": 44,
            "questions": [
                "convert quake 2 map to quake 3",
                "convert quake 2 map to quake 3"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[18]",
        {
            "from": 45,
            "to": 46,
            "questions": [
                "scale quake map",
                "scale quake map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[19]",
        {
            "from": 47,
            "to": 48,
            "questions": [
                "translate quake map",
                "translate quake map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "find only bsps in map paks",
                "find only bsps in map paks"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[20]",
        {
            "from": 49,
            "to": 50,
            "questions": [
                "replace common textures in map",
                "replace common textures in map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[21]",
        {
            "from": 51,
            "to": 52,
            "questions": [
                "replace known classes in map",
                "replace known classes in map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[22]",
        {
            "from": 53,
            "to": 54,
            "questions": [
                "replace entities in map",
                "replace entities in map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[23]",
        {
            "from": 55,
            "to": 56,
            "questions": [
                "add skybox to map",
                "add skybox to map",
                "add a skybox and remove sky entities"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[24]",
        {
            "from": 57,
            "to": 58,
            "questions": [
                "draw hints in map",
                "draw hints in map",
                "draw hints from x and y coordinates"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[25]",
        {
            "from": 59,
            "to": 60,
            "questions": [
                "split map hints",
                "split map hints",
                "copy brushes inside a polygon",
                "split map using hints and portals"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[26]",
        {
            "from": 61,
            "to": 66,
            "questions": [
                "brush to vertex",
                "brush to vertex",
                "get vertex from brush strokes",
                "TODO: convert quake 2 entities to quake 3",
                "TODO: use \"bodyque\" for dead body model explosions these have a high health in quake 2they should drop the goods."
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[27]",
        {
            "from": 67,
            "to": 68,
            "questions": [
                ""
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[28]",
        {
            "from": 69,
            "to": 70,
            "questions": [
                "list noises in a quake 3 map",
                "list noises in a quake 3 map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[29]",
        {
            "from": 71,
            "to": 72,
            "questions": [
                "list textures in quake 3 map",
                "list textures in quake 3 map"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "list bsps in a pak",
                "list bsps in a pak"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[30]",
        {
            "from": 73,
            "to": 74,
            "questions": [
                "list shaders in quake 3 shader",
                "list shaders in quake 3 shader"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[31]",
        {
            "from": 75,
            "to": 76,
            "questions": [
                "find all shaders",
                "find all shaders"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[32]",
        {
            "from": 77,
            "to": 78,
            "questions": [
                "search textures directory for paths",
                "search textures directory for paths"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[33]",
        {
            "from": 79,
            "to": 81,
            "questions": [
                "disassemble all QVMs",
                "disassemble all QVMs"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "unpack pk3s",
                "unpack pk3s",
                "unpack entire folder of pk3s"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[6]",
        {
            "from": 15,
            "to": 17,
            "questions": [
                "make pk3 indexes",
                "make pk3 indexes"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[7]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "md4 checksum",
                "md4 checksum"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[8]",
        {
            "from": 21,
            "to": 23,
            "questions": [
                "crc checksum file",
                "crc checksum file"
            ]
        }
    ],
    [
        1725690235536,
        "/Users/briancullinan/jupyter_ops/Games/quake 3.ipynb[9]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "test crc file",
                "test crc file"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "test sending a discord message",
                "test sending a discord message",
                "https://discord.com/oauth2/authorizeresponse_type=code&client_id=723583889779589221&scope=bot&permissions=2248399936&redirect_uri=https://discord.com/channels/@me"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[10]",
        {
            "from": 31,
            "to": 34,
            "questions": [
                "decode client messages",
                "decode client messages"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[11]",
        {
            "from": 35,
            "to": 37,
            "questions": [
                "huffman decode",
                "huffman decode"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[12]",
        {
            "from": 38,
            "to": 40,
            "questions": [
                "monitor q3 servers",
                "monitor q3 servers"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[13]",
        {
            "from": 41,
            "to": 43,
            "questions": [
                "index",
                "index",
                "discord bot index"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[14]",
        {
            "from": 44,
            "to": 46,
            "questions": [
                "ssh remote wget",
                "ssh remote wget"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[15]",
        {
            "from": 47,
            "to": 49,
            "questions": [
                "dns lookup",
                "dns lookup"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[16]",
        {
            "from": 50,
            "to": 52,
            "questions": [
                "remove ctrl characters",
                "remove ctrl characters"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[17]",
        {
            "from": 53,
            "to": 55,
            "questions": [
                "quake3 server status",
                "quake3 server status"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[18]",
        {
            "from": 56,
            "to": 58,
            "questions": [
                "spectate q3 server",
                "spectate q3 server"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "respond discord commands",
                "respond discord commands"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[3]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "test specific channel",
                "test specific channel"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[4]",
        {
            "from": 13,
            "to": 15,
            "questions": [
                "challenge discord command",
                "challenge discord command"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[5]",
        {
            "from": 16,
            "to": 18,
            "questions": [
                "discord bot",
                "discord bot",
                "discord commands"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[6]",
        {
            "from": 19,
            "to": 22,
            "questions": [
                "quake 3 server commands",
                "quake 3 server commands"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[7]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "test quake 3 rcon commands",
                "test quake 3 rcon commands"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[8]",
        {
            "from": 25,
            "to": 27,
            "questions": [
                "quake 3 server responses",
                "quake 3 server responses"
            ]
        }
    ],
    [
        1738469780090,
        "/Users/briancullinan/jupyter_ops/Games/quake3 server connector.ipynb[9]",
        {
            "from": 28,
            "to": 30,
            "questions": [
                "format quake 3 response",
                "format quake 3 response"
            ]
        }
    ],
    [
        1603053969000,
        "/Users/briancullinan/jupyter_ops/Games/quakejs.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "quakejs web master",
                "quakejs web master"
            ]
        }
    ],
    [
        1603053969000,
        "/Users/briancullinan/jupyter_ops/Games/quakejs.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "quakejs connection",
                "quakejs connection"
            ]
        }
    ],
    [
        1603053969000,
        "/Users/briancullinan/jupyter_ops/Games/quakejs.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "quakejs parsing",
                "quakejs parsing"
            ]
        }
    ],
    [
        1603053969000,
        "/Users/briancullinan/jupyter_ops/Games/quakejs.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "quakejs utilities",
                "quakejs utilities"
            ]
        }
    ],
    [
        1603053969000,
        "/Users/briancullinan/jupyter_ops/Games/quakejs.ipynb[4]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "test quakejs master",
                "test quakejs master"
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Games/unity basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Games/webgl demos.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Google/google analytics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Google/google api authentication.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "test calendar sum all hours worked on study sauce since september 2016",
                "test calendar sum all hours worked on study sauce since september 2016"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "convert chrome date to calendar date",
                "convert chrome date to calendar date"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Parse bookmarks file",
                "Parse bookmarks file"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "get bookmarks from local chrome database",
                "get bookmarks from local chrome database"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "test parse bookmarks",
                "test parse bookmarks"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "sync chrome bookmarks",
                "sync chrome bookmarks"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "sync chrome history",
                "sync chrome history"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "sync chrome data",
                "sync chrome data"
            ]
        }
    ],
    [
        1602124207000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar data.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "test syn chrome data",
                "test syn chrome data"
            ]
        }
    ],
    [
        1603471792000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar graphs.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "test calendar api graph Iga's cycle for as long as there are valid dates",
                "test calendar api graph Iga's cycle for as long as there are valid dates"
            ]
        }
    ],
    [
        1603471792000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar graphs.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test google calendar search heatmap",
                "test google calendar search heatmap"
            ]
        }
    ],
    [
        1603471792000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar graphs.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test google calendar search swimlane",
                "test google calendar search swimlane"
            ]
        }
    ],
    [
        1603471792000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar graphs.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "test google calendar search pie chart",
                "test google calendar search pie chart",
                "Show a d3 pie chart of time spent on projects"
            ]
        }
    ],
    [
        1603471792000,
        "/Users/briancullinan/jupyter_ops/Google/google calendar graphs.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use Google calendar API",
                "use Google calendar API",
                "authorize Google calendar"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "list events",
                "list events"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Run todays calendar events",
                "Run todays calendar events"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "correct dates time",
                "correct dates time",
                "Correct calendar dates for timeMax and timeMin"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "lookup calendar name",
                "lookup calendar name",
                "Lookup calendar id by name or id"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "days events",
                "days events",
                "get days events"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "Update create merge delete event",
                "Update create merge delete event"
            ]
        }
    ],
    [
        1744609540769,
        "/Users/briancullinan/jupyter_ops/Google/google calendar.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "create new calendar event",
                "create new calendar event"
            ]
        }
    ],
    [
        1511887066000,
        "/Users/briancullinan/jupyter_ops/Google/google contacts.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "list google contact",
                "list google contact"
            ]
        }
    ],
    [
        1511887066000,
        "/Users/briancullinan/jupyter_ops/Google/google contacts.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "google contact settings",
                "google contact settings"
            ]
        }
    ],
    [
        1511887066000,
        "/Users/briancullinan/jupyter_ops/Google/google contacts.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511887066000,
        "/Users/briancullinan/jupyter_ops/Google/google contacts.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "authorize google drive",
                "authorize google drive"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "test list google drive",
                "test list google drive"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[10]",
        {
            "from": 25,
            "to": 27,
            "questions": [
                "create a copy of study sauce template",
                "create a copy of study sauce template"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[11]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                "create a sheet handler",
                "create a sheet handler",
                "create a copy of marketing template"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "list google drive files",
                "list google drive files"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "insert google drive permissions",
                "insert google drive permissions"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "create a sheet in google drive",
                "create a sheet in google drive"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "test google sheet create",
                "test google sheet create"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "copy a file on google drive",
                "copy a file on google drive"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[7]",
        {
            "from": 17,
            "to": 20,
            "questions": [
                "merge google drive",
                "merge google drive",
                "compare google drive",
                "What is the purpose 🐬 of this WellGoogle Drive sync is awful for these reason:",
                "8. Of course! Good Google sync is only available for paying customers. Even though I am paying for extra storage every month  How much sense does this make",
                "1. Drive creates an MD5 hash for every single file. Anyone familiar with rsync principles knows there are plenty of faster metrics to compare before hashing a filesuch asdoes it exist Did the file time change"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[8]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                "download all docs as actual data files",
                "download all docs as actual data files"
            ]
        }
    ],
    [
        1738639761450,
        "/Users/briancullinan/jupyter_ops/Google/google drive.ipynb[9]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                "sign a file url for uploading to google storage",
                "sign a file url for uploading to google storage"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "use places nearby API",
                "use places nearby API",
                "find a place using Google maps"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "place details google maps",
                "place details google maps"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "use Google Geocaching",
                "use Google Geocaching"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "cache locations nearby",
                "cache locations nearby"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "extract depth maps",
                "extract depth maps",
                "extract depth maps"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "load google panorama",
                "load google panorama"
            ]
        }
    ],
    [
        1624116520000,
        "/Users/briancullinan/jupyter_ops/Google/google maps.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "test google depth maps",
                "test google depth maps"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "space out purple event",
                "space out purple event"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[1]",
        {
            "from": 4,
            "to": 6,
            "questions": [
                "schedule study time",
                "schedule study time",
                "schedule hours per event"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[2]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "schedule inside outside of time bounds",
                "schedule inside outside of time bounds"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[3]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "remove overlaps",
                "remove overlaps",
                "remove overlaps from events with alloable wiggle room"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[4]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "prioritize events based on glob summary",
                "prioritize events based on glob summary"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[5]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "calendar search heatmap",
                "calendar search heatmap"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[6]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                "search calendar",
                "search calendar"
            ]
        }
    ],
    [
        1602100679000,
        "/Users/briancullinan/jupyter_ops/Google/google scheduling.ipynb[7]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1592600875000,
        "/Users/briancullinan/jupyter_ops/Google/google search api.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "search the web",
                "search the web"
            ]
        }
    ],
    [
        1592600875000,
        "/Users/briancullinan/jupyter_ops/Google/google search api.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test custom search",
                "test custom search"
            ]
        }
    ],
    [
        1592600875000,
        "/Users/briancullinan/jupyter_ops/Google/google search api.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "authorize custom search",
                "authorize custom search"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "templates google doc",
                "templates google doc",
                "templates google sheet"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test templates google sheet",
                "test templates google sheet"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[10]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "authorize sheets api",
                "authorize sheets api"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "get google sheet info",
                "get google sheet info"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "get google data sheet",
                "get google data sheet",
                "google sheet array objects"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "get worksheet rows",
                "get worksheet rows"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "test google data sheet",
                "test google data sheet"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "add row data google sheet",
                "add row data google sheet",
                "add a row of data to a google sheet"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "get worksheet by name",
                "get worksheet by name"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "update a row in google sheets",
                "update a row in google sheets"
            ]
        }
    ],
    [
        1624077444000,
        "/Users/briancullinan/jupyter_ops/Google/google sheets.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "test google sheet add row",
                "test google sheet add row"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[1]",
        {
            "from": 4,
            "to": 7,
            "questions": [
                "select antlr tree",
                "select antlr tree"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[10]",
        {
            "from": 27,
            "to": 28,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[11]",
        {
            "from": 29,
            "to": 30,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[12]",
        {
            "from": 31,
            "to": 33,
            "questions": [
                "ext to lang",
                "ext to lang",
                "file extension to language",
                "language from file extension"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[13]",
        {
            "from": 34,
            "to": 36,
            "questions": [
                "generate antlr parsers",
                "generate antlr parsers",
                "Because why the heck not"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[2]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[3]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "test select jison on some quake 3 C code",
                "test select jison on some quake 3 C code"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[4]",
        {
            "from": 12,
            "to": 15,
            "questions": [
                "get antlr tool",
                "get antlr tool",
                "get antlr language"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[5]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "antlr to html",
                "antlr to html"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[6]",
        {
            "from": 18,
            "to": 20,
            "questions": [
                "antlr tree visitor",
                "antlr tree visitor"
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[7]",
        {
            "from": 21,
            "to": 22,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[8]",
        {
            "from": 23,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922555545,
        "/Users/briancullinan/jupyter_ops/Languages/antlr.ipynb[9]",
        {
            "from": 25,
            "to": 26,
            "questions": [
                "antlr tree to es",
                "antlr tree to es"
            ]
        }
    ],
    [
        1576626817000,
        "/Users/briancullinan/jupyter_ops/Languages/balanced.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "balanced",
                "balanced"
            ]
        }
    ],
    [
        1576626817000,
        "/Users/briancullinan/jupyter_ops/Languages/balanced.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[1]",
        {
            "from": 4,
            "to": 7,
            "questions": [
                "select jison tree",
                "select jison tree"
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[2]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[3]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[4]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[5]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579635693000,
        "/Users/briancullinan/jupyter_ops/Languages/bison.ipynb[6]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740682674276,
        "/Users/briancullinan/jupyter_ops/Languages/csharp.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "rosetta euler csharp 005",
                "rosetta euler csharp 005"
            ]
        }
    ],
    [
        1740682674276,
        "/Users/briancullinan/jupyter_ops/Languages/csharp.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test edge.js",
                "test edge.js"
            ]
        }
    ],
    [
        1740682674276,
        "/Users/briancullinan/jupyter_ops/Languages/csharp.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "list csharp namespaces",
                "list csharp namespaces"
            ]
        }
    ],
    [
        1740682674276,
        "/Users/briancullinan/jupyter_ops/Languages/csharp.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "list csharp classes",
                "list csharp classes"
            ]
        }
    ],
    [
        1740682674276,
        "/Users/briancullinan/jupyter_ops/Languages/csharp.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "get csharp params",
                "get csharp params"
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Languages/go basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Languages/javascript essentials.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563421830000,
        "/Users/briancullinan/jupyter_ops/Languages/json.ipynb[0]",
        {
            "from": 0,
            "to": 5,
            "questions": [
                "select json",
                "select json",
                "stream json using a {match} function",
                "Why parse when JSON is built in To do it in as few lines as possible. Learn something about syntax and parsing. Create an API or pattern for lexing languages."
            ]
        }
    ],
    [
        1563421830000,
        "/Users/briancullinan/jupyter_ops/Languages/json.ipynb[1]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "test stream by creating a slow stream",
                "test stream by creating a slow stream"
            ]
        }
    ],
    [
        1563421830000,
        "/Users/briancullinan/jupyter_ops/Languages/json.ipynb[2]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1579235767000,
        "/Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[0]",
        {
            "from": 0,
            "to": 3,
            "questions": [
                "expand",
                "expand"
            ]
        }
    ],
    [
        1579235767000,
        "/Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[1]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "mini parser",
                "mini parser"
            ]
        }
    ],
    [
        1579235767000,
        "/Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[2]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "minimatch",
                "minimatch"
            ]
        }
    ],
    [
        1579235767000,
        "/Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[3]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448820,
        "/Users/briancullinan/jupyter_ops/Languages/programming paradigms.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744922194290,
        "/Users/briancullinan/jupyter_ops/Languages/python.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "rosetta euler python 005",
                "rosetta euler python 005"
            ]
        }
    ],
    [
        1744922194290,
        "/Users/briancullinan/jupyter_ops/Languages/python.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "list c functions with python",
                "list c functions with python"
            ]
        }
    ],
    [
        1744922194290,
        "/Users/briancullinan/jupyter_ops/Languages/python.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "python params in antrl",
                "python params in antrl"
            ]
        }
    ],
    [
        1563387486000,
        "/Users/briancullinan/jupyter_ops/Languages/regexp.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "Why parse it when it is built in to javascript",
                "Why parse it when it is built in to javascript"
            ]
        }
    ],
    [
        1563387486000,
        "/Users/briancullinan/jupyter_ops/Languages/regexp.ipynb[1]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Languages/rust intro.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Languages/typescript in depth.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[4]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[5]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1563328519000,
        "/Users/briancullinan/jupyter_ops/Languages/xpath.ipynb[6]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[2]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[3]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[4]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/2017 Update.ipynb[5]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Marketing/A-B testing.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/ECommerce.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/ECommerce.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/ECommerce.ipynb[2]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/ECommerce.ipynb[3]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Marketing/ECommerce.ipynb[4]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Marketing/Email Campaigns.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Good UX Intro.ipynb[0]",
        {
            "from": 0,
            "to": 15,
            "questions": [
                "Pretty simple right",
                "Pretty simple right",
                "1. Who benefits from this",
                "How do the pieces fit together",
                "2. How does the company benefit from this",
                "3. How does the sales team benefit from this",
                "What benefits are we giving to the customer by presenting a subscription portal"
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Good UX Intro.ipynb[1]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Good UX Intro.ipynb[2]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Good UX Intro.ipynb[3]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Good UX Intro.ipynb[4]",
        {
            "from": 19,
            "to": 30,
            "questions": [
                "2. Product owner: Begin.  Good on regressions",
                "2. Product owner: Begin.  Good on regressions"
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[1]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[10]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[11]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[12]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[13]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[14]",
        {
            "from": 19,
            "to": 19,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[15]",
        {
            "from": 20,
            "to": 20,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[16]",
        {
            "from": 21,
            "to": 21,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[17]",
        {
            "from": 22,
            "to": 22,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[18]",
        {
            "from": 23,
            "to": 23,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[19]",
        {
            "from": 24,
            "to": 24,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[20]",
        {
            "from": 25,
            "to": 25,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[4]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[5]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[6]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[7]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[8]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Marketing/Landing Pages.ipynb[9]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Marketing/SEO Basics.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448822,
        "/Users/briancullinan/jupyter_ops/Marketing/Social Media Strategy.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[10]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[11]",
        {
            "from": 14,
            "to": 14,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[12]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[13]",
        {
            "from": 16,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[14]",
        {
            "from": 17,
            "to": 17,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[2]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[3]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[4]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[5]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[6]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[7]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[8]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1741632169785,
        "/Users/briancullinan/jupyter_ops/Marketing/Wireframing.ipynb[9]",
        {
            "from": 11,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1514049280000,
        "/Users/briancullinan/jupyter_ops/Selenium/avidbrain.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Test avidbrain",
                "Test avidbrain"
            ]
        }
    ],
    [
        1514049280000,
        "/Users/briancullinan/jupyter_ops/Selenium/avidbrain.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1514049280000,
        "/Users/briancullinan/jupyter_ops/Selenium/avidbrain.ipynb[2]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Selenium/browsers.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "collect facebook profiles",
                "collect facebook profiles"
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[2]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[3]",
        {
            "from": 4,
            "to": 6,
            "questions": [
                "connect on facebook",
                "connect on facebook",
                "sync facebook contacts with google contacts"
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[4]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "connect add friends facebook",
                "connect add friends facebook"
            ]
        }
    ],
    [
        1519922580000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook connections.ipynb[5]",
        {
            "from": 9,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Log in to facebook",
                "Log in to facebook"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Scrape facebook profile",
                "Scrape facebook profile"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[10]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[11]",
        {
            "from": 19,
            "to": 20,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Like all facebook posts",
                "Like all facebook posts"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Scrape facebook event",
                "Scrape facebook event"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Scrape facebook events",
                "Scrape facebook events"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[5]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[6]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "Scrape facebook friends",
                "Scrape facebook friends"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[7]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "Automatically diff facebook friends",
                "Automatically diff facebook friends"
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[8]",
        {
            "from": 15,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557807638000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook data.ipynb[9]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "Unfollow everyone on facebook",
                "Unfollow everyone on facebook"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "get unread threads facebook",
                "get unread threads facebook"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "scan commands facebook",
                "scan commands facebook"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "sync facebook threads",
                "sync facebook threads"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "List Facebook threads",
                "List Facebook threads"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Get messages from facebook",
                "Get messages from facebook"
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[5]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1562123449000,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook messaging.ipynb[6]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "send facebook message",
                "send facebook message"
            ]
        }
    ],
    [
        1738815480799,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook webdriver.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "facebook login",
                "facebook login"
            ]
        }
    ],
    [
        1738815480799,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook webdriver.ipynb[1]",
        {
            "from": 5,
            "to": 8,
            "questions": [
                "list friends",
                "list friends"
            ]
        }
    ],
    [
        1738815480799,
        "/Users/briancullinan/jupyter_ops/Selenium/facebook webdriver.ipynb[2]",
        {
            "from": 9,
            "to": 11,
            "questions": [
                "add friend",
                "add friend"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "form utilities",
                "form utilities",
                "Utilities for fillingclickingreading forms"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "fill select dropdown",
                "fill select dropdown"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "map object to form",
                "map object to form"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "log in to multiple sites",
                "log in to multiple sites"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "test sites logins",
                "test sites logins"
            ]
        }
    ],
    [
        1557163611000,
        "/Users/briancullinan/jupyter_ops/Selenium/forms.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "get all links html",
                "get all links html"
            ]
        }
    ],
    [
        1511381318000,
        "/Users/briancullinan/jupyter_ops/Selenium/github.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "download github",
                "download github",
                "Follow someone more closely",
                "Star all the modules I am using",
                "Ask them why they are FOMO instead of working",
                "Automated code reviews for common detectable coding mistakes",
                "Diff all package.json to identify project/stack/popularity trends",
                "Research every tech commenter and try to find their github activity",
                "Automatically heart issues where the solution works or a pull request is made"
            ]
        }
    ],
    [
        1511381318000,
        "/Users/briancullinan/jupyter_ops/Selenium/github.ipynb[1]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511381318000,
        "/Users/briancullinan/jupyter_ops/Selenium/github.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1736557774759,
        "/Users/briancullinan/jupyter_ops/Selenium/google authorize.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "google oauth token client",
                "google oauth token client",
                "authenticate to Google APIs",
                "exchange token for oauth client"
            ]
        }
    ],
    [
        1736557774759,
        "/Users/briancullinan/jupyter_ops/Selenium/google authorize.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "use selenium to authorize Google access",
                "use selenium to authorize Google access"
            ]
        }
    ],
    [
        1736557774759,
        "/Users/briancullinan/jupyter_ops/Selenium/google authorize.ipynb[2]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1736557774759,
        "/Users/briancullinan/jupyter_ops/Selenium/google authorize.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511803619000,
        "/Users/briancullinan/jupyter_ops/Selenium/google takeout.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "download google takeout",
                "download google takeout"
            ]
        }
    ],
    [
        1511803619000,
        "/Users/briancullinan/jupyter_ops/Selenium/google takeout.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "order google takeout",
                "order google takeout"
            ]
        }
    ],
    [
        1511803619000,
        "/Users/briancullinan/jupyter_ops/Selenium/google takeout.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1511803619000,
        "/Users/briancullinan/jupyter_ops/Selenium/google takeout.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "d3 stream my chrome history by category and site sub-divided",
                "d3 stream my chrome history by category and site sub-divided"
            ]
        }
    ],
    [
        1511803619000,
        "/Users/briancullinan/jupyter_ops/Selenium/google takeout.ipynb[4]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Scrape google timeline",
                "Scrape google timeline"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Read single google timeline page",
                "Read single google timeline page"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "Find the average latitute and longitude at each destination",
                "Find the average latitute and longitude at each destination"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Reconcile timeline data with calendar",
                "Reconcile timeline data with calendar"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Load location export data",
                "Load location export data"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "reconcile timeline",
                "reconcile timeline"
            ]
        }
    ],
    [
        1516259770000,
        "/Users/briancullinan/jupyter_ops/Selenium/google timeline.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "search timeline events",
                "search timeline events"
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Selenium/headless mode.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "sync linkedin contacts with google contacts",
                "sync linkedin contacts with google contacts"
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "scrape entire linkedin profile",
                "scrape entire linkedin profile"
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "scrape linkedin contacts",
                "scrape linkedin contacts"
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "connect on linkedin",
                "connect on linkedin"
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "connect add friends linkedin",
                "connect add friends linkedin"
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[6]",
        {
            "from": 12,
            "to": 12,
            "questions": [
                ""
            ]
        }
    ],
    [
        1528511210000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin connections.ipynb[7]",
        {
            "from": 13,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Log in to LinkedIn",
                "Log in to LinkedIn"
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Scrape LinkedIn profile",
                "Scrape LinkedIn profile"
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "List LinkedIn threads",
                "List LinkedIn threads"
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Read messages LinkedIn thread",
                "Read messages LinkedIn thread"
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "scrape linkedin threads",
                "scrape linkedin threads"
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[5]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1512869633000,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin messages.ipynb[6]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "scan for commands linkedin",
                "scan for commands linkedin"
            ]
        }
    ],
    [
        1738859271305,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin webdriver.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "login linkedin",
                "login linkedin"
            ]
        }
    ],
    [
        1738859271305,
        "/Users/briancullinan/jupyter_ops/Selenium/linkedin webdriver.ipynb[1]",
        {
            "from": 5,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "resize selenium window",
                "resize selenium window"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "only one window",
                "only one window"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "tile selenium chrome windows",
                "tile selenium chrome windows"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "get all session and window urls",
                "get all session and window urls"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "send a joke",
                "send a joke"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "send facebook thanks",
                "send facebook thanks"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "clean up old selenium sessions",
                "clean up old selenium sessions"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "screenshot all sessions",
                "screenshot all sessions"
            ]
        }
    ],
    [
        1650994137000,
        "/Users/briancullinan/jupyter_ops/Selenium/orchestration.ipynb[9]",
        {
            "from": 18,
            "to": 18,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "reddit login",
                "reddit login"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[1]",
        {
            "from": 5,
            "to": 8,
            "questions": [
                "reddit post",
                "reddit post"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[2]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "test reddit post",
                "test reddit post"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[3]",
        {
            "from": 11,
            "to": 14,
            "questions": [
                "reddit weekly",
                "reddit weekly"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[4]",
        {
            "from": 15,
            "to": 18,
            "questions": [
                "reddit month of links",
                "reddit month of links"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[5]",
        {
            "from": 19,
            "to": 21,
            "questions": [
                "reddit scraper",
                "reddit scraper"
            ]
        }
    ],
    [
        1740355785388,
        "/Users/briancullinan/jupyter_ops/Selenium/reddit.ipynb[6]",
        {
            "from": 22,
            "to": 23,
            "questions": [
                "test reddit scraper",
                "test reddit scraper"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "extract llm article",
                "extract llm article"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[1]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "test article extract",
                "test article extract"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[2]",
        {
            "from": 7,
            "to": 9,
            "questions": [
                "summarize llm article",
                "summarize llm article"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[3]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "test article summarizer",
                "test article summarizer"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[4]",
        {
            "from": 12,
            "to": 14,
            "questions": [
                "summarize all articles",
                "summarize all articles"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[5]",
        {
            "from": 15,
            "to": 17,
            "questions": [
                "convert summaries",
                "convert summaries"
            ]
        }
    ],
    [
        1740353079366,
        "/Users/briancullinan/jupyter_ops/Selenium/scraping.ipynb[6]",
        {
            "from": 18,
            "to": 21,
            "questions": [
                "default link collector",
                "default link collector"
            ]
        }
    ],
    [
        1744594947155,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium basics.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594947155,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium basics.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594947155,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium basics.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594947155,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium basics.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594947155,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium basics.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738787130700,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium commands.ipynb[0]",
        {
            "from": 0,
            "to": 2,
            "questions": [
                "What is Selenium",
                "What is Selenium"
            ]
        }
    ],
    [
        1738787130700,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium commands.ipynb[1]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738787130700,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium commands.ipynb[2]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738787130700,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium commands.ipynb[3]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "Run selenium inside of docker",
                "Run selenium inside of docker"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium demo.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Start the selenium server and crawl 16 search results at once",
                "Start the selenium server and crawl 16 search results at once"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium demo.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744595083939,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium nodejs.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744595083939,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium nodejs.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744595083939,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium nodejs.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744595083939,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium nodejs.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Start a selenium http express server",
                "Start a selenium http express server"
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744155297727,
        "/Users/briancullinan/jupyter_ops/Selenium/selenium server.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                ""
            ]
        }
    ],
    [
        1508452594000,
        "/Users/briancullinan/jupyter_ops/Selenium/send social message.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Decrypt passwords.json",
                "Decrypt passwords.json"
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Add encrypted to passwords.json",
                "Add encrypted to passwords.json"
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[4]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "download passwords from google",
                "download passwords from google"
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[5]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[6]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "log in to Google using webdriver",
                "log in to Google using webdriver"
            ]
        }
    ],
    [
        1559875029000,
        "/Users/briancullinan/jupyter_ops/Selenium/service auth.ipynb[7]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "Automatically fill any type of login form using various algorithms",
                "Automatically fill any type of login form using various algorithms"
            ]
        }
    ],
    [
        1744594448819,
        "/Users/briancullinan/jupyter_ops/Selenium/testing with selenium.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Scroll a specific element",
                "Scroll a specific element"
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "all elements until",
                "all elements until",
                "Get all elements until"
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "click spa link",
                "click spa link"
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "https://www.mathworks.com/help/vision/examples/automatically-detect-and-recognize-text-in-natural-images.htmls_tid=gn_loc_drop",
                "https://www.mathworks.com/help/vision/examples/automatically-detect-and-recognize-text-in-natural-images.htmls_tid=gn_loc_drop"
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "verify human",
                "verify human"
            ]
        }
    ],
    [
        1738860740883,
        "/Users/briancullinan/jupyter_ops/Selenium/utilities.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "test verify human",
                "test verify human"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[0]",
        {
            "from": 0,
            "to": 4,
            "questions": [
                "selenium client",
                "selenium client",
                "webdriver client"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[1]",
        {
            "from": 5,
            "to": 7,
            "questions": [
                "selenium session",
                "selenium session"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[10]",
        {
            "from": 30,
            "to": 31,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[11]",
        {
            "from": 32,
            "to": 33,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[12]",
        {
            "from": 34,
            "to": 35,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[13]",
        {
            "from": 36,
            "to": 37,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[14]",
        {
            "from": 38,
            "to": 38,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[15]",
        {
            "from": 39,
            "to": 39,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[16]",
        {
            "from": 40,
            "to": 41,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[17]",
        {
            "from": 42,
            "to": 43,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[2]",
        {
            "from": 8,
            "to": 10,
            "questions": [
                "webdriver test",
                "webdriver test"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[3]",
        {
            "from": 11,
            "to": 13,
            "questions": [
                "close all windows",
                "close all windows"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[4]",
        {
            "from": 14,
            "to": 16,
            "questions": [
                "selenium executor",
                "selenium executor"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[5]",
        {
            "from": 17,
            "to": 19,
            "questions": [
                "list sessions",
                "list sessions"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[6]",
        {
            "from": 20,
            "to": 23,
            "questions": [
                "selenium select",
                "selenium select"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[7]",
        {
            "from": 24,
            "to": 25,
            "questions": [
                "selenium query",
                "selenium query"
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[8]",
        {
            "from": 26,
            "to": 27,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740347450381,
        "/Users/briancullinan/jupyter_ops/Selenium/webdriver.ipynb[9]",
        {
            "from": 28,
            "to": 29,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557180269000,
        "/Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "What is Selenium",
                "What is Selenium",
                "set up selenium server",
                "What environment do we use",
                "See [What is Docker](What%20is%20Docker.ipynb)"
            ]
        }
    ],
    [
        1557180269000,
        "/Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "connect to VNC through the web browser",
                "connect to VNC through the web browser",
                "[Click here to open](http://localhost:6080/vnc.htmlpassword=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser"
            ]
        }
    ],
    [
        1557180269000,
        "/Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "run a selenium cell on the Docker machine",
                "run a selenium cell on the Docker machine"
            ]
        }
    ],
    [
        1557180269000,
        "/Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Test docker selenium",
                "Test docker selenium"
            ]
        }
    ],
    [
        1557180269000,
        "/Users/briancullinan/jupyter_ops/Selenium/What is Selenium.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "Transfer login state to current browser",
                "Transfer login state to current browser"
            ]
        }
    ],
    [
        1561486625000,
        "/Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "high five people in YouEarnedIt",
                "high five people in YouEarnedIt"
            ]
        }
    ],
    [
        1561486625000,
        "/Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "automate YouEarnedIt",
                "automate YouEarnedIt"
            ]
        }
    ],
    [
        1561486625000,
        "/Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561486625000,
        "/Users/briancullinan/jupyter_ops/Selenium/you earned it.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561854971000,
        "/Users/briancullinan/jupyter_ops/Untitled.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561854971000,
        "/Users/briancullinan/jupyter_ops/Untitled.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594017537,
        "/Users/briancullinan/jupyter_ops/Utilities/active directory.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[1]",
        {
            "from": 1,
            "to": 2,
            "questions": [
                "node webdriver support",
                "node webdriver support"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[2]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "install node",
                "install node"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                "install notebooks",
                "install notebooks"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[4]",
        {
            "from": 7,
            "to": 8,
            "questions": [
                "install systemd",
                "install systemd"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[5]",
        {
            "from": 9,
            "to": 10,
            "questions": [
                "install launchd",
                "install launchd"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[6]",
        {
            "from": 11,
            "to": 12,
            "questions": [
                "copy credentials",
                "copy credentials"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[7]",
        {
            "from": 13,
            "to": 14,
            "questions": [
                "run the same command",
                "run the same command"
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[8]",
        {
            "from": 15,
            "to": 16,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744650004950,
        "/Users/briancullinan/jupyter_ops/Utilities/bash.ipynb[9]",
        {
            "from": 17,
            "to": 18,
            "questions": [
                "better jupyter organization",
                "better jupyter organization"
            ]
        }
    ],
    [
        1740434597666,
        "/Users/briancullinan/jupyter_ops/Utilities/bitbuffer.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "bit buffer",
                "bit buffer",
                "bit stream"
            ]
        }
    ],
    [
        1738622135375,
        "/Users/briancullinan/jupyter_ops/Utilities/child process.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "spawn child process",
                "spawn child process",
                "spawning child processes"
            ]
        }
    ],
    [
        1738622135375,
        "/Users/briancullinan/jupyter_ops/Utilities/child process.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "test child process",
                "test child process"
            ]
        }
    ],
    [
        1738622135375,
        "/Users/briancullinan/jupyter_ops/Utilities/child process.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "convert bash to one liner",
                "convert bash to one liner"
            ]
        }
    ],
    [
        1740340529429,
        "/Users/briancullinan/jupyter_ops/Utilities/ciede2000.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "ciede2000",
                "ciede2000"
            ]
        }
    ],
    [
        1740340529429,
        "/Users/briancullinan/jupyter_ops/Utilities/ciede2000.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "rgb 2 lab",
                "rgb 2 lab"
            ]
        }
    ],
    [
        1740340529429,
        "/Users/briancullinan/jupyter_ops/Utilities/ciede2000.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "try ciede2000",
                "try ciede2000"
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Utilities/cli tools roundup.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1556086123000,
        "/Users/briancullinan/jupyter_ops/Utilities/compress.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Minified lzw compression",
                "Minified lzw compression"
            ]
        }
    ],
    [
        1556086123000,
        "/Users/briancullinan/jupyter_ops/Utilities/compress.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1556086123000,
        "/Users/briancullinan/jupyter_ops/Utilities/compress.ipynb[2]",
        {
            "from": 3,
            "to": 4,
            "questions": [
                "encode files with brotli",
                "encode files with brotli"
            ]
        }
    ],
    [
        1556086123000,
        "/Users/briancullinan/jupyter_ops/Utilities/compress.ipynb[3]",
        {
            "from": 5,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Utilities/data conversion.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "find Tuesdays late in the month",
                "find Tuesdays late in the month"
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Convert a date to ISO",
                "Convert a date to ISO"
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "sum a list of events",
                "sum a list of events"
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Number of days between events",
                "Number of days between events"
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "display recurrence line graph",
                "display recurrence line graph"
            ]
        }
    ],
    [
        1515722434000,
        "/Users/briancullinan/jupyter_ops/Utilities/dates.ipynb[5]",
        {
            "from": 10,
            "to": 10,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745170296696,
        "/Users/briancullinan/jupyter_ops/Utilities/diff.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "diff",
                "diff",
                "diff code blocks"
            ]
        }
    ],
    [
        1745170296696,
        "/Users/briancullinan/jupyter_ops/Utilities/diff.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "prism-style.css",
                "prism-style.css"
            ]
        }
    ],
    [
        1745170296696,
        "/Users/briancullinan/jupyter_ops/Utilities/diff.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "test prism js",
                "test prism js"
            ]
        }
    ],
    [
        1745170296696,
        "/Users/briancullinan/jupyter_ops/Utilities/diff.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "keep-markup.js",
                "keep-markup.js"
            ]
        }
    ],
    [
        1745170296696,
        "/Users/briancullinan/jupyter_ops/Utilities/diff.ipynb[4]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "include a dynamic lib",
                "include a dynamic lib"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[10]",
        {
            "from": 20,
            "to": 21,
            "questions": [
                "build dylib",
                "build dylib"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "test a csharp dylib",
                "test a csharp dylib"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "set mouse position",
                "set mouse position"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "get c parameters",
                "get c parameters"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "get c exports",
                "get c exports"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                ""
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "get c types",
                "get c types"
            ]
        }
    ],
    [
        1744682917044,
        "/Users/briancullinan/jupyter_ops/Utilities/dylib.ipynb[9]",
        {
            "from": 18,
            "to": 19,
            "questions": [
                "c type to data type",
                "c type to data type"
            ]
        }
    ],
    [
        1742004667035,
        "/Users/briancullinan/jupyter_ops/Utilities/heap.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1515130518000,
        "/Users/briancullinan/jupyter_ops/Utilities/heartbeat.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "heartbeat thump",
                "heartbeat thump"
            ]
        }
    ],
    [
        1515130518000,
        "/Users/briancullinan/jupyter_ops/Utilities/heartbeat.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "run todays heartbeat items",
                "run todays heartbeat items"
            ]
        }
    ],
    [
        1515130518000,
        "/Users/briancullinan/jupyter_ops/Utilities/heartbeat.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1745299648832,
        "/Users/briancullinan/jupyter_ops/Utilities/levenshtein.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Find the levenshtein distance",
                "Find the levenshtein distance"
            ]
        }
    ],
    [
        1745299648832,
        "/Users/briancullinan/jupyter_ops/Utilities/levenshtein.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "Sort by levenshtein distance",
                "Sort by levenshtein distance"
            ]
        }
    ],
    [
        1745299648832,
        "/Users/briancullinan/jupyter_ops/Utilities/levenshtein.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "search levenshtein distance",
                "search levenshtein distance"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/mono.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "run Mono with docker",
                "run Mono with docker"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/mono.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/mono.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/nmap.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/nmap.ipynb[1]",
        {
            "from": 1,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561142770000,
        "/Users/briancullinan/jupyter_ops/Utilities/openssl.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "create a trusted root CA",
                "create a trusted root CA"
            ]
        }
    ],
    [
        1561142770000,
        "/Users/briancullinan/jupyter_ops/Utilities/openssl.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1561142770000,
        "/Users/briancullinan/jupyter_ops/Utilities/openssl.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "generate a self-signed cert",
                "generate a self-signed cert"
            ]
        }
    ],
    [
        1561142770000,
        "/Users/briancullinan/jupyter_ops/Utilities/openssl.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1576695646000,
        "/Users/briancullinan/jupyter_ops/Utilities/packages.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1513807706000,
        "/Users/briancullinan/jupyter_ops/Utilities/parse.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                "//  escape a string for regex",
                "//  escape a string for regex",
                "//  flatten and array in javascript"
            ]
        }
    ],
    [
        1624084235000,
        "/Users/briancullinan/jupyter_ops/Utilities/polyfills.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "https request",
                "https request",
                "http request pollyfill"
            ]
        }
    ],
    [
        1624084235000,
        "/Users/briancullinan/jupyter_ops/Utilities/polyfills.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1573949166000,
        "/Users/briancullinan/jupyter_ops/Utilities/qr codes.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "generate a qr code using google APIs",
                "generate a qr code using google APIs"
            ]
        }
    ],
    [
        1745170783335,
        "/Users/briancullinan/jupyter_ops/Utilities/quine.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738016759771,
        "/Users/briancullinan/jupyter_ops/Utilities/raspberry pi.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1738016759771,
        "/Users/briancullinan/jupyter_ops/Utilities/raspberry pi.ipynb[1]",
        {
            "from": 1,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "node screenshots",
                "node screenshots"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "node window screenshot",
                "node window screenshot"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "get screen size",
                "get screen size"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "move mouse",
                "move mouse"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "mouse click",
                "mouse click"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "send keys",
                "send keys"
            ]
        }
    ],
    [
        1740350019065,
        "/Users/briancullinan/jupyter_ops/Utilities/robot.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "mouse control cocoa",
                "mouse control cocoa"
            ]
        }
    ],
    [
        1520396719000,
        "/Users/briancullinan/jupyter_ops/Utilities/scraping.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1520396719000,
        "/Users/briancullinan/jupyter_ops/Utilities/scraping.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "google maps data list",
                "google maps data list",
                "Gather resturant data from google maps"
            ]
        }
    ],
    [
        1520396719000,
        "/Users/briancullinan/jupyter_ops/Utilities/scraping.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1520396719000,
        "/Users/briancullinan/jupyter_ops/Utilities/scraping.ipynb[3]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740530933939,
        "/Users/briancullinan/jupyter_ops/Utilities/shell-parse.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "shell parse",
                "shell parse"
            ]
        }
    ],
    [
        1744594448821,
        "/Users/briancullinan/jupyter_ops/Utilities/system monitoring.ipynb[0]",
        {
            "from": 0,
            "to": 0,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740328788196,
        "/Users/briancullinan/jupyter_ops/Utilities/usage.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "system usage report",
                "system usage report"
            ]
        }
    ],
    [
        1740328788196,
        "/Users/briancullinan/jupyter_ops/Utilities/usage.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "system usage graph",
                "system usage graph"
            ]
        }
    ],
    [
        1740328788196,
        "/Users/briancullinan/jupyter_ops/Utilities/usage.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "system usage png buffer",
                "system usage png buffer"
            ]
        }
    ],
    [
        1510360183000,
        "/Users/briancullinan/jupyter_ops/Utilities/uuid.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "Generate a universally unique identifier",
                "Generate a universally unique identifier"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "What is VNC",
                "What is VNC"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[2]",
        {
            "from": 4,
            "to": 5,
            "questions": [
                "VNC HTML client",
                "VNC HTML client"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[3]",
        {
            "from": 6,
            "to": 7,
            "questions": [
                "Use vnc with live-server",
                "Use vnc with live-server"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[4]",
        {
            "from": 8,
            "to": 9,
            "questions": [
                "VNC in Docker",
                "VNC in Docker"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[5]",
        {
            "from": 10,
            "to": 11,
            "questions": [
                "selenium vnc server",
                "selenium vnc server"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[6]",
        {
            "from": 12,
            "to": 13,
            "questions": [
                "vnc entry point",
                "vnc entry point"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[7]",
        {
            "from": 14,
            "to": 15,
            "questions": [
                "install vnc entry",
                "install vnc entry"
            ]
        }
    ],
    [
        1557180626000,
        "/Users/briancullinan/jupyter_ops/Utilities/vnc.ipynb[8]",
        {
            "from": 16,
            "to": 17,
            "questions": [
                "install aws vnc selenium",
                "install aws vnc selenium"
            ]
        }
    ],
    [
        1611173047000,
        "/Users/briancullinan/jupyter_ops/Utilities/wget.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                ""
            ]
        }
    ],
    [
        1611173047000,
        "/Users/briancullinan/jupyter_ops/Utilities/wget.ipynb[1]",
        {
            "from": 2,
            "to": 2,
            "questions": [
                ""
            ]
        }
    ],
    [
        1611173047000,
        "/Users/briancullinan/jupyter_ops/Utilities/wget.ipynb[2]",
        {
            "from": 3,
            "to": 3,
            "questions": [
                ""
            ]
        }
    ],
    [
        1611173047000,
        "/Users/briancullinan/jupyter_ops/Utilities/wget.ipynb[3]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1611173047000,
        "/Users/briancullinan/jupyter_ops/Utilities/wget.ipynb[4]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740531605684,
        "/Users/briancullinan/jupyter_ops/Utilities/window enum.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "enum windows with cocoa",
                "enum windows with cocoa"
            ]
        }
    ],
    [
        1740531605684,
        "/Users/briancullinan/jupyter_ops/Utilities/window enum.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "lookup window by",
                "lookup window by"
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[0]",
        {
            "from": 0,
            "to": 1,
            "questions": [
                "list window handles",
                "list window handles"
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[1]",
        {
            "from": 2,
            "to": 3,
            "questions": [
                "monitor active windows",
                "monitor active windows"
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[2]",
        {
            "from": 4,
            "to": 4,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[3]",
        {
            "from": 5,
            "to": 5,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[4]",
        {
            "from": 6,
            "to": 6,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[5]",
        {
            "from": 7,
            "to": 7,
            "questions": [
                ""
            ]
        }
    ],
    [
        1740505191252,
        "/Users/briancullinan/jupyter_ops/Utilities/windows.ipynb[6]",
        {
            "from": 8,
            "to": 8,
            "questions": [
                ""
            ]
        }
    ]
]

module.exports = {
    cellCache
}




#### cache notebook?

```
cacheCells(cells, filepath) 
```


Created slightly differently than the import / export cache. Notebooks are provided below. It caches the keywords, but always loads the notebook from disk.

how to cache notebook questions in fuse?

how to find functions in jupyter notebooks?

TODO: classify the cell if it has unit tests, selenium, assert, compile errors



In [ ]:
var path = require('path');
var fs = require('fs')
var MATCH_QUESTIONS = (/^.*\?.*$/igm);

function getQuestions(source, markdown) {
    const {regexToArray} = require('../Core')
    var questions = regexToArray(MATCH_QUESTIONS, markdown)
        .map(r => r.replaceAll(/how to|\?|#+|, /ig, '').trim())
        .concat(regexToArray(MATCH_QUESTIONS, source)
            .filter(r => r.match(/how to/ig))
            .map(r => r.replaceAll(/how to|\?|#+|, /ig, '').trim()))
    // find the shortest words from the query to match the same cell
    questions.sort((a, b) => a.length - b.length)
    // TODO: weird fix, Fuse.js apparently doesn't work on lists with one element
    return questions[0] ? [questions[0]].concat(questions) : questions;
}

// TODO: convert to pattern like group everything leading up to match
function accumulateMarkdown(cells) {
    // read markdown leading up to code cells
    var codes = cells.filter(c => c.cell_type === 'code')
    return codes
        .map((c, i) => Object.assign(c, {
            from: i > 0 ? (cells.indexOf(codes[i - 1]) + 1) : 0,
            to: cells.indexOf(c)
        }))
        .map((c) => Object.assign(c, {
            markdown: cells.slice(c.from, c.to).map(m => m.source.join('')).join(''),
            code: c.source.join('')
        }))
}

const TIMES = {}
const CACHE = {}

function cacheCells(filename) {
    const {getCells} = require('../Core')
    filename = path.resolve(filename)
    const mtime = fs.statSync(filename).mtime.getTime()
    if(CACHE[filename] && TIMES[filename] >= mtime) {
        return CACHE[filename]
    }
    const cells = getCells(filename, ['*', 'markdown', 'code'])
    const newCache = accumulateMarkdown(cells)
    const result = newCache.map((c, i) => {
        var id = path.basename(filename) + '[' + i + ']'
        var questions = getQuestions(c.code, c.markdown)
        if (questions.length === 0) questions = ['']
        return Object.assign(c, {
            id,
            filename,
            mtime,
            questions,
            notebook: path.basename(filename)
        })
    })
    TIMES[filename] = mtime
    CACHE[filename] = result
    return result
}

module.exports = {
    cacheCells,
    accumulateMarkdown,
    getQuestions,
}


#### test cache notebook?


#### cache all?

```
findNotebooks(dirname)
```

How to get markdown from all notebooks in {directory}?

How to interpret a jupyter {directory}?

find all notebooks?


In [ ]:
var path = require('path')
var fs = require('fs')
var assert = require('assert')
var importer = require('../Core')
var {listInProject} = importer.import('list project files');
var {
    cacheCells, cellCache, refreshCache,
    updateCache, updateCode
} = importer.import([
    'cache notebook', 'cell cache', 'refresh regular cache',
    'update regular cache', 'update code cell'
])

function findNotebooks(dirname) {
    let notebooks = listInProject(dirname, ['/*.ipynb', '/*/*.ipynb', '/*/*/*.ipynb'])
    return notebooks
}

function cacheAll() {
    // include the path to this notebook first
    var notebooks = [
        path.resolve(__dirname, '../Core/cache.ipynb')
    ].concat(findNotebooks(path.resolve(__dirname, '../')))
    
    // all ids are the ones in the cache plus missing notebooks
    var allIds = notebooks
        .reduce((arr, n) => arr
                .concat([n + '[0]'])
                .concat(cellCache
                        .filter(c => c[1].substr(0, n.length) === n)
                        .map(c => c[1])), [])
    
    // update by loading every notebook
    var updates = notebooks.reduce((arr, n) => {
        return arr.concat(cacheCells(n)
            .map(({mtime, id, filename, from, to, questions}) => [
                mtime,
                path.join(path.dirname(filename), id),
                {
                    from,
                    to,
                    questions,
                }
            ]))
    }, [])
     
    // save the cache in this notebook
    updateCache(updates, cellCache, updates.map(u => u[1]))
    
    saveCache()
}

function saveCache() {
    var code = `
// cell cache automatically replaced
var cellCache = ${JSON.stringify(cellCache, null, 4)}

module.exports = {
    cellCache
}

`
    var cacheCell = importer.interpret('cell cache')
    updateCode(cacheCell, code)
}

module.exports = {
    findNotebooks,
    cacheAll,
    saveCache
}


### imports cache

A cache of all imports in every cell, for making project trees.

TODO: use this cache to detect when a module changes in import notebook.


#### the cache

imports cache?

This code is automatically replaced with import code and stored in this notebook.


In [ ]:

// imports cache automatically replaced
var importsCache = [
    [
        1563084143353,
        "files.ipynb[0]",
        [
            "importer",
            "import",
            "d3.ipynb[create word-cloud]",
            "relative paths and includes"
        ]
    ],
    [
        1563084143986,
        "files.ipynb[1]",
        [
            "importer",
            "import",
            "d3.ipynb[format tree]",
            "d3.ipynb[display d3 tree]",
            "relative paths and includes"
        ]
    ],
    [
        1563084148299,
        "files.ipynb[10]",
        []
    ],
    [
        1563084148477,
        "files.ipynb[11]",
        []
    ],
    [
        1563084148557,
        "files.ipynb[12]",
        []
    ],
    [
        1563084148818,
        "files.ipynb[13]",
        []
    ],
    [
        1563084145467,
        "files.ipynb[2]",
        []
    ],
    [
        1563084145545,
        "files.ipynb[3]",
        []
    ],
    [
        1563084145585,
        "files.ipynb[4]",
        [
            "importer",
            "import",
            "list project files"
        ]
    ],
    [
        1563084145678,
        "files.ipynb[5]",
        [
            "importer",
            "import",
            "list project files",
            "list all projects names"
        ]
    ],
    [
        1563084145854,
        "files.ipynb[6]",
        []
    ],
    [
        1563084145986,
        "files.ipynb[7]",
        [
            "importer",
            "import",
            "relative paths and includes",
            "task to promise",
            "similar filename in project",
            "runAllPromises",
            "undefined"
        ]
    ],
    [
        1563084147304,
        "files.ipynb[8]",
        [
            "importer",
            "import",
            "tasks to promise"
        ]
    ],
    [
        1563084148239,
        "files.ipynb[9]",
        []
    ]
]

module.exports = {
    importsCache
}


#### create import cache?



In [ ]:
var importer = require('../Core')
var {createCellCache, importsCache} = importer.import([
    'create cell cache', 'imports cache'
])
var {getImports} = importer.import('get imports from source')

function createImportCache(search) {
    var cacheCell = importer.interpret('imports cache')
    return createCellCache(search, importsCache, cacheCell, getImports)
}

module.exports = {
    createImportCache
}


#### test import cache?



In [ ]:
var importer = require('../Core')
var {createImportCache} = importer.import('create import cache')

module.exports = createImportCache

if(typeof $$ !== 'undefined') {
    createImportCache()
}


### exports cache

A cache of all exports, no matter the language.


#### the cache

exports cache?

This code is automatically replaced with export code and stored in this notebook.


In [ ]:

// exports cache automatically replaced
var exportsCache = [
    [
        1563066646373,
        "2017 Update.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[1]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[2]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[3]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646374,
        "2017 Update.ipynb[4]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066646414,
        "2017 Update.ipynb[5]",
        []
    ],
    [
        1563066821650,
        "active directory.ipynb[0]",
        []
    ],
    [
        1563066781145,
        "Angular 2.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066781146,
        "Angular 2.ipynb[1]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066781146,
        "Angular 2.ipynb[2]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066781212,
        "Angular 2.ipynb[3]",
        []
    ],
    [
        1563066781213,
        "Angular 2.ipynb[4]",
        "Line 1: Unexpected token"
    ],
    [
        1563066781213,
        "Angular 2.ipynb[5]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066778897,
        "Angular components.ipynb[0]",
        "Line 1: Unexpected token"
    ],
    [
        1563066778897,
        "Angular components.ipynb[1]",
        "Line 1: Unexpected token"
    ],
    [
        1563066778897,
        "Angular components.ipynb[2]",
        "Line 1: Unexpected token"
    ],
    [
        1563066779194,
        "Angular components.ipynb[3]",
        [
            "searchHandler",
            "searchAll"
        ]
    ],
    [
        1563066779240,
        "Angular components.ipynb[4]",
        []
    ],
    [
        1563066779241,
        "Angular components.ipynb[5]",
        "Line 1: Unexpected token"
    ],
    [
        1563066779241,
        "Angular components.ipynb[6]",
        "Line 1: Unexpected token"
    ],
    [
        1563066777016,
        "angular core modules.ipynb[0]",
        "Line 1: Unexpected token"
    ],
    [
        1563066775083,
        "angular render service.ipynb[0]",
        "Line 3: Unexpected token"
    ],
    [
        1563066775117,
        "angular render service.ipynb[1]",
        []
    ],
    [
        1563066773106,
        "angular-cli.ipynb[0]",
        "Line 24: Unexpected token switch"
    ],
    [
        1563066773141,
        "angular-cli.ipynb[1]",
        []
    ],
    [
        1563066934258,
        "aspects.ipynb[0]",
        [
            "inspectCallback",
            "inspectTemplate",
            "transpileInspect",
            "insertInspect"
        ]
    ],
    [
        1563066934293,
        "aspects.ipynb[1]",
        []
    ],
    [
        1563066934298,
        "aspects.ipynb[2]",
        []
    ],
    [
        1563066934302,
        "aspects.ipynb[3]",
        []
    ],
    [
        1563066970529,
        "audio.ipynb[0]",
        [
            "walkAudio",
            "selectAudio"
        ]
    ],
    [
        1563066765169,
        "autorewire.ipynb[0]",
        []
    ],
    [
        1563066765865,
        "autorewire.ipynb[1]",
        [
            "AutoMock",
            "parent",
            "simpleStubCreator"
        ]
    ],
    [
        1563066765873,
        "autorewire.ipynb[2]",
        []
    ],
    [
        1563066617996,
        "avidbrain.ipynb[0]",
        [
            "testLogin"
        ]
    ],
    [
        1563066618062,
        "avidbrain.ipynb[1]",
        []
    ],
    [
        1563066618067,
        "avidbrain.ipynb[2]",
        []
    ],
    [
        1563066952180,
        "aws.ipynb[0]",
        [
            "handler"
        ]
    ],
    [
        1563066952458,
        "aws.ipynb[1]",
        [
            "latestS3",
            "getS3Objects"
        ]
    ],
    [
        1563066952504,
        "aws.ipynb[2]",
        []
    ],
    [
        1563066505731,
        "bash.ipynb[0]",
        "Line 1: Unexpected token |"
    ],
    [
        1563066505732,
        "bash.ipynb[1]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066505732,
        "bash.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505732,
        "bash.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505733,
        "bash.ipynb[4]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066505733,
        "bash.ipynb[5]",
        "Line 1: Unexpected token /"
    ],
    [
        1563066505733,
        "bash.ipynb[6]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066505733,
        "bash.ipynb[7]",
        "Line 1: Unexpected token :"
    ],
    [
        1563066643508,
        "brians resume.ipynb[0]",
        [
            "getBookmarkFolders"
        ]
    ],
    [
        1563066643862,
        "brians resume.ipynb[1]",
        [
            "updateInterestPage"
        ]
    ],
    [
        1563066968567,
        "budgeting.ipynb[0]",
        []
    ],
    [
        1563066771046,
        "build Angular components.ipynb[0]",
        [
            "ng"
        ]
    ],
    [
        1563066771200,
        "build Angular components.ipynb[1]",
        [
            "webpackAngularProject"
        ]
    ],
    [
        1563066771207,
        "build Angular components.ipynb[2]",
        []
    ],
    [
        1563066771211,
        "build Angular components.ipynb[3]",
        []
    ],
    [
        1563082217207,
        "cache.ipynb[0]",
        []
    ],
    [
        1563082217244,
        "cache.ipynb[1]",
        []
    ],
    [
        1563082218098,
        "cache.ipynb[10]",
        [
            "memorySafe"
        ]
    ],
    [
        1563082218149,
        "cache.ipynb[11]",
        [
            "refreshCache"
        ]
    ],
    [
        1563082218198,
        "cache.ipynb[12]",
        []
    ],
    [
        1563082218201,
        "cache.ipynb[13]",
        []
    ],
    [
        1563082217268,
        "cache.ipynb[2]",
        [
            "createImportCache"
        ]
    ],
    [
        1563082217368,
        "cache.ipynb[3]",
        [
            "createImportCache"
        ]
    ],
    [
        1563066931182,
        "cache.ipynb[4]",
        [
            "exportsCache"
        ]
    ],
    [
        1563082217409,
        "cache.ipynb[5]",
        [
            "createExportCache",
            "createCellCache"
        ]
    ],
    [
        1563082217832,
        "cache.ipynb[6]",
        [
            "createExportCache"
        ]
    ],
    [
        1563082217854,
        "cache.ipynb[7]",
        []
    ],
    [
        1563082217857,
        "cache.ipynb[8]",
        [
            "updateCode"
        ]
    ],
    [
        1563082217934,
        "cache.ipynb[9]",
        [
            "updateCache",
            "cleanCache",
            "sortAlphaNumeric"
        ]
    ],
    [
        1563066966463,
        "calculations for life.ipynb[0]",
        []
    ],
    [
        1563066966591,
        "calculations for life.ipynb[1]",
        []
    ],
    [
        1563066966594,
        "calculations for life.ipynb[2]",
        []
    ],
    [
        1563066503749,
        "child process.ipynb[0]",
        [
            "execCmd",
            "bashToExec"
        ]
    ],
    [
        1563066503778,
        "child process.ipynb[1]",
        []
    ],
    [
        1563066503844,
        "child process.ipynb[2]",
        [
            "bashToOne"
        ]
    ],
    [
        1563066855622,
        "compile.ipynb[0]",
        []
    ],
    [
        1563066855628,
        "compile.ipynb[1]",
        []
    ],
    [
        1563066855696,
        "compile.ipynb[2]",
        [
            "test"
        ]
    ],
    [
        1563066855701,
        "compile.ipynb[3]",
        []
    ],
    [
        1563066495607,
        "compress.ipynb[0]",
        [
            "LZString",
            "o"
        ]
    ],
    [
        1563066501453,
        "compress.ipynb[1]",
        [
            "LZString",
            "getBaseValue"
        ]
    ],
    [
        1563066501470,
        "compress.ipynb[2]",
        []
    ],
    [
        1563066501473,
        "compress.ipynb[3]",
        []
    ],
    [
        1563066636093,
        "convert spreadsheet.ipynb[0]",
        []
    ],
    [
        1563066636622,
        "convert spreadsheet.ipynb[1]",
        [
            "filteredData",
            "unfilteredData"
        ]
    ],
    [
        1563066640036,
        "convert spreadsheet.ipynb[10]",
        [
            "getSheet",
            "safeName",
            "addSheet"
        ]
    ],
    [
        1563066640186,
        "convert spreadsheet.ipynb[11]",
        [
            "importSheet"
        ]
    ],
    [
        1563066640293,
        "convert spreadsheet.ipynb[12]",
        [
            "setupBackend"
        ]
    ],
    [
        1563066640396,
        "convert spreadsheet.ipynb[13]",
        [
            "copyMarketing"
        ]
    ],
    [
        1563066640397,
        "convert spreadsheet.ipynb[14]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066637206,
        "convert spreadsheet.ipynb[2]",
        [
            "getTemplateProperties",
            "createAssignFilter"
        ]
    ],
    [
        1563066637665,
        "convert spreadsheet.ipynb[3]",
        [
            "renderRows",
            "safeName",
            "escape",
            "getDataClasses",
            "defineProperty"
        ]
    ],
    [
        1563066637952,
        "convert spreadsheet.ipynb[4]",
        [
            "wrapTemplate",
            "safeName",
            "toJSON"
        ]
    ],
    [
        1563066638336,
        "convert spreadsheet.ipynb[5]",
        [
            "collectRoutes"
        ]
    ],
    [
        1563066638933,
        "convert spreadsheet.ipynb[6]",
        [
            "collectExternalResources",
            "safeName"
        ]
    ],
    [
        1563066639039,
        "convert spreadsheet.ipynb[7]",
        [
            "getTemplateByUrl"
        ]
    ],
    [
        1563066639428,
        "convert spreadsheet.ipynb[8]",
        [
            "collectTemplateResources"
        ]
    ],
    [
        1563066639663,
        "convert spreadsheet.ipynb[9]",
        [
            "importTest"
        ]
    ],
    [
        1563066761828,
        "d3 dates.ipynb[0]",
        [
            "d3Swimlane",
            "display",
            "moveBrush",
            "getPaths"
        ]
    ],
    [
        1563066763193,
        "d3 dates.ipynb[1]",
        [
            "d3Heatmap",
            "monthPath"
        ]
    ],
    [
        1563066751658,
        "d3.ipynb[0]",
        [
            "d3CloudToSVG",
            "drawD3Cloud"
        ]
    ],
    [
        1563066752423,
        "d3.ipynb[1]",
        [
            "d3TieredPieSVG",
            "arcTween",
            "computeTextRotation"
        ]
    ],
    [
        1563066752460,
        "d3.ipynb[2]",
        []
    ],
    [
        1563066752699,
        "d3.ipynb[3]",
        [
            "d3PieChart"
        ]
    ],
    [
        1563066752821,
        "d3.ipynb[4]",
        [
            "formatNodes"
        ]
    ],
    [
        1563066754018,
        "d3.ipynb[5]",
        [
            "displayBranches",
            "branchIndex"
        ]
    ],
    [
        1563066754022,
        "d3.ipynb[6]",
        []
    ],
    [
        1563066614353,
        "data collection.ipynb[0]",
        [
            "scrapeAlert"
        ]
    ],
    [
        1563066614850,
        "data collection.ipynb[1]",
        [
            "multiCrawl",
            "deQueue"
        ]
    ],
    [
        1563066615004,
        "data collection.ipynb[2]",
        [
            "searchResultsToJson"
        ]
    ],
    [
        1563066615184,
        "data collection.ipynb[3]",
        [
            "searchAll"
        ]
    ],
    [
        1563066615247,
        "data collection.ipynb[4]",
        [
            "scheduleSearch"
        ]
    ],
    [
        1563066615627,
        "data collection.ipynb[5]",
        [
            "crawlAll",
            "startCrawl",
            "fixLink"
        ]
    ],
    [
        1563066615700,
        "data collection.ipynb[6]",
        [
            "scheduleSearch"
        ]
    ],
    [
        1563066615842,
        "data collection.ipynb[7]",
        [
            "getJoke"
        ]
    ],
    [
        1563066486609,
        "dates.ipynb[0]",
        [
            "getTuesdays"
        ]
    ],
    [
        1563066486699,
        "dates.ipynb[1]",
        [
            "ISODateString",
            "pad"
        ]
    ],
    [
        1563066486827,
        "dates.ipynb[2]",
        [
            "sumEvents"
        ]
    ],
    [
        1563066486945,
        "dates.ipynb[3]",
        [
            "filterDistant"
        ]
    ],
    [
        1563066487233,
        "dates.ipynb[4]",
        [
            "graphDates"
        ]
    ],
    [
        1563066487237,
        "dates.ipynb[5]",
        []
    ],
    [
        1563066748912,
        "de-linting.ipynb[0]",
        [
            "delintCode"
        ]
    ],
    [
        1563066748987,
        "de-linting.ipynb[1]",
        [
            "delintCell"
        ]
    ],
    [
        1563066749161,
        "de-linting.ipynb[2]",
        []
    ],
    [
        1563066749262,
        "de-linting.ipynb[3]",
        [
            "delint"
        ]
    ],
    [
        1563066804315,
        "demo.ipynb[0]",
        []
    ],
    [
        1563066804344,
        "demo.ipynb[1]",
        []
    ],
    [
        1563066804402,
        "demo.ipynb[10]",
        []
    ],
    [
        1563066804406,
        "demo.ipynb[11]",
        []
    ],
    [
        1563066804411,
        "demo.ipynb[12]",
        []
    ],
    [
        1563066804417,
        "demo.ipynb[13]",
        []
    ],
    [
        1563066804417,
        "demo.ipynb[14]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066804417,
        "demo.ipynb[15]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563066804377,
        "demo.ipynb[2]",
        []
    ],
    [
        1563066804377,
        "demo.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066804382,
        "demo.ipynb[4]",
        []
    ],
    [
        1563066804386,
        "demo.ipynb[5]",
        []
    ],
    [
        1563066804390,
        "demo.ipynb[6]",
        []
    ],
    [
        1563066804390,
        "demo.ipynb[7]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066804394,
        "demo.ipynb[8]",
        []
    ],
    [
        1563066804398,
        "demo.ipynb[9]",
        []
    ],
    [
        1563066949819,
        "deploy host.ipynb[0]",
        [
            "deployAws"
        ]
    ],
    [
        1563066949820,
        "deploy host.ipynb[1]",
        "Line 1: Unexpected string"
    ],
    [
        1563066949920,
        "deploy host.ipynb[2]",
        []
    ],
    [
        1563066949921,
        "deploy host.ipynb[3]",
        "Line 1: Unexpected string"
    ],
    [
        1563066484060,
        "diff.ipynb[0]",
        [
            "diffTwoTexts",
            "htmlEntities",
            "prismHighlightHtml"
        ]
    ],
    [
        1563066484150,
        "diff.ipynb[1]",
        []
    ],
    [
        1563066484155,
        "diff.ipynb[2]",
        []
    ],
    [
        1563066768077,
        "display angular.ipynb[0]",
        [
            "getScriptsAndStyles"
        ]
    ],
    [
        1563066768360,
        "display angular.ipynb[1]",
        [
            "displAngular"
        ]
    ],
    [
        1563066768725,
        "display angular.ipynb[2]",
        [
            "outputSlowly",
            "compress"
        ]
    ],
    [
        1563066768907,
        "display angular.ipynb[3]",
        []
    ],
    [
        1563066768927,
        "display angular.ipynb[4]",
        []
    ],
    [
        1563066768930,
        "display angular.ipynb[5]",
        []
    ],
    [
        1563066768933,
        "display angular.ipynb[6]",
        []
    ],
    [
        1563066802325,
        "docker commands.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066802325,
        "docker commands.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066802326,
        "docker commands.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066802326,
        "docker commands.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066802326,
        "docker commands.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066802327,
        "docker commands.ipynb[5]",
        "Line 1: Unexpected string"
    ],
    [
        1563066802327,
        "docker commands.ipynb[6]",
        "Line 1: Unexpected string"
    ],
    [
        1563066802327,
        "docker commands.ipynb[7]",
        "Line 1: Unexpected string"
    ],
    [
        1563066800282,
        "docker demo.ipynb[0]",
        []
    ],
    [
        1563066800429,
        "docker demo.ipynb[1]",
        [
            "seleniumDocker",
            "bashToRun"
        ]
    ],
    [
        1563066964113,
        "documentation.ipynb[0]",
        [
            "getRpcSpecFromCells",
            "getTail",
            "getPathId"
        ]
    ],
    [
        1563066964202,
        "documentation.ipynb[1]",
        []
    ],
    [
        1563066964208,
        "documentation.ipynb[2]",
        []
    ],
    [
        1563066964212,
        "documentation.ipynb[3]",
        []
    ],
    [
        1563066798272,
        "dotnet.ipynb[0]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066633791,
        "ECommerce.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[1]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[2]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633792,
        "ECommerce.ipynb[3]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066633829,
        "ECommerce.ipynb[4]",
        []
    ],
    [
        1563066611346,
        "facebook connections.ipynb[0]",
        [
            "collectFacebookProfiles",
            "parseFacebookFriends"
        ]
    ],
    [
        1563066611399,
        "facebook connections.ipynb[1]",
        []
    ],
    [
        1563066611433,
        "facebook connections.ipynb[2]",
        []
    ],
    [
        1563066611554,
        "facebook connections.ipynb[3]",
        [
            "friendsOnFacebook"
        ]
    ],
    [
        1563066611995,
        "facebook connections.ipynb[4]",
        [
            "addFacebookFriends",
            "loadNewConnections"
        ]
    ],
    [
        1563066612013,
        "facebook connections.ipynb[5]",
        []
    ],
    [
        1563066606347,
        "facebook data.ipynb[0]",
        [
            "loginFacebook",
            "enterFacebook"
        ]
    ],
    [
        1563066606474,
        "facebook data.ipynb[1]",
        [
            "readFacebookProfileInfo"
        ]
    ],
    [
        1563066608451,
        "facebook data.ipynb[10]",
        []
    ],
    [
        1563066608561,
        "facebook data.ipynb[11]",
        [
            "clickInvite",
            "getUninvited",
            "clickConnect"
        ]
    ],
    [
        1563066607341,
        "facebook data.ipynb[2]",
        [
            "likeAllPosts",
            "listFacebookPosts",
            "scrapeFacebookPost",
            "likeFacebookPost"
        ]
    ],
    [
        1563066607665,
        "facebook data.ipynb[3]",
        [
            "scrapeFacebookEvent"
        ]
    ],
    [
        1563066607908,
        "facebook data.ipynb[4]",
        [
            "scrapeFacebookEvents",
            "getEvents"
        ]
    ],
    [
        1563066607923,
        "facebook data.ipynb[5]",
        []
    ],
    [
        1563066607970,
        "facebook data.ipynb[6]",
        [
            "scrapeFacebookFriends"
        ]
    ],
    [
        1563066608279,
        "facebook data.ipynb[7]",
        [
            "getFriendsDiff",
            "getFriends"
        ]
    ],
    [
        1563066608299,
        "facebook data.ipynb[8]",
        []
    ],
    [
        1563066608376,
        "facebook data.ipynb[9]",
        [
            "unfollowFacebook"
        ]
    ],
    [
        1563066602492,
        "facebook messaging.ipynb[0]",
        [
            "getUnreadThreads"
        ]
    ],
    [
        1563066602854,
        "facebook messaging.ipynb[1]",
        [
            "scanCommandsFacebook"
        ]
    ],
    [
        1563066602970,
        "facebook messaging.ipynb[2]",
        []
    ],
    [
        1563066603212,
        "facebook messaging.ipynb[3]",
        [
            "listFacebookThreads",
            "getFacebookThreads",
            "listArchivedFacebook"
        ]
    ],
    [
        1563066604032,
        "facebook messaging.ipynb[4]",
        [
            "readFacebookThread",
            "switchToParticipantThread",
            "getThreadParticipants",
            "readFacebookMessages"
        ]
    ],
    [
        1563066604059,
        "facebook messaging.ipynb[5]",
        []
    ],
    [
        1563066604178,
        "facebook messaging.ipynb[6]",
        [
            "sendFacebookMessage"
        ]
    ],
    [
        1563066789913,
        "falsey javascript.ipynb[0]",
        []
    ],
    [
        1563066789930,
        "falsey javascript.ipynb[1]",
        []
    ],
    [
        1563066789943,
        "falsey javascript.ipynb[2]",
        []
    ],
    [
        1563066789959,
        "falsey javascript.ipynb[3]",
        []
    ],
    [
        1563066789976,
        "falsey javascript.ipynb[4]",
        []
    ],
    [
        1563066790001,
        "falsey javascript.ipynb[5]",
        []
    ],
    [
        1563066790005,
        "falsey javascript.ipynb[6]",
        []
    ],
    [
        1563066972815,
        "files.ipynb[0]",
        [
            "projectWordCloud",
            "wordCount"
        ]
    ],
    [
        1563066973398,
        "files.ipynb[1]",
        [
            "projectTree",
            "icons",
            "wordCount"
        ]
    ],
    [
        1563066974743,
        "files.ipynb[10]",
        []
    ],
    [
        1563066974790,
        "files.ipynb[11]",
        [
            "mkdirpSync"
        ]
    ],
    [
        1563066974920,
        "files.ipynb[12]",
        [
            "mockTypescriptFs"
        ]
    ],
    [
        1563066974969,
        "files.ipynb[13]",
        []
    ],
    [
        1563066973444,
        "files.ipynb[2]",
        [
            "listInProject"
        ]
    ],
    [
        1563066973476,
        "files.ipynb[3]",
        [
            "ignores"
        ]
    ],
    [
        1563066973533,
        "files.ipynb[4]",
        [
            "listProjects"
        ]
    ],
    [
        1563066973619,
        "files.ipynb[5]",
        []
    ],
    [
        1563066973684,
        "files.ipynb[6]",
        [
            "findSimilarFile"
        ]
    ],
    [
        1563066974202,
        "files.ipynb[7]",
        [
            "projectRelatives",
            "fixRelative"
        ]
    ],
    [
        1563066974576,
        "files.ipynb[8]",
        [
            "findLongFunctions",
            "matchCurlyBraces"
        ]
    ],
    [
        1563066974613,
        "files.ipynb[9]",
        []
    ],
    [
        1563066787788,
        "fizz buzz.ipynb[0]",
        [
            "fizzBuzz"
        ]
    ],
    [
        1563066787846,
        "fizz buzz.ipynb[1]",
        [
            "isFib"
        ]
    ],
    [
        1563066787931,
        "fizz buzz.ipynb[2]",
        [
            "sortDesc"
        ]
    ],
    [
        1563066787935,
        "fizz buzz.ipynb[3]",
        []
    ],
    [
        1563066599340,
        "forms.ipynb[0]",
        [
            "importer",
            "import"
        ]
    ],
    [
        1563066599456,
        "forms.ipynb[1]",
        [
            "selectDropdown"
        ]
    ],
    [
        1563066599553,
        "forms.ipynb[2]",
        [
            "fillForm"
        ]
    ],
    [
        1563066599991,
        "forms.ipynb[3]",
        [
            "multiLogin",
            "fillAll"
        ]
    ],
    [
        1563066600193,
        "forms.ipynb[4]",
        [
            "testLogins"
        ]
    ],
    [
        1563066600200,
        "forms.ipynb[5]",
        []
    ],
    [
        1563066819712,
        "git commands.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066819712,
        "git commands.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066819713,
        "git commands.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066819713,
        "git commands.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066819713,
        "git commands.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066819713,
        "git commands.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746745,
        "git repos.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746746,
        "git repos.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746746,
        "git repos.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746747,
        "git repos.ipynb[3]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066746747,
        "git repos.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746747,
        "git repos.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746747,
        "git repos.ipynb[6]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066746747,
        "git repos.ipynb[7]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066813723,
        "git.ipynb[0]",
        [
            "renameUsingGit"
        ]
    ],
    [
        1563066813733,
        "git.ipynb[1]",
        []
    ],
    [
        1563066817111,
        "git.ipynb[10]",
        [
            "globBranch"
        ]
    ],
    [
        1563066817805,
        "git.ipynb[11]",
        [
            "gitHistory",
            "gitRemote",
            "gitBranch",
            "gitLog"
        ]
    ],
    [
        1563066814010,
        "git.ipynb[2]",
        [
            "gitTree"
        ]
    ],
    [
        1563066814590,
        "git.ipynb[3]",
        [
            "gitTipOfTree",
            "getResetCommit",
            "resetAllBranches"
        ]
    ],
    [
        1563066814731,
        "git.ipynb[4]",
        [
            "getBranchBoundaries"
        ]
    ],
    [
        1563066814776,
        "git.ipynb[5]",
        [
            "resetRebaseInstallEvent"
        ]
    ],
    [
        1563066815490,
        "git.ipynb[6]",
        [
            "getUpdate",
            "installBuildTestEvent",
            "updateHeartbeatEvent",
            "findBranchRemote"
        ]
    ],
    [
        1563066816113,
        "git.ipynb[7]",
        [
            "cloneProject",
            "deepCopy"
        ]
    ],
    [
        1563066816355,
        "git.ipynb[8]",
        [
            "gitEditor",
            "getGist",
            "replaceSelectors"
        ]
    ],
    [
        1563066816776,
        "git.ipynb[9]",
        [
            "loadScraped",
            "noop",
            "getAllLinks",
            "fixLink"
        ]
    ],
    [
        1563066597254,
        "github.ipynb[0]",
        [
            "getGithub"
        ]
    ],
    [
        1563066597260,
        "github.ipynb[1]",
        []
    ],
    [
        1563066597264,
        "github.ipynb[2]",
        []
    ],
    [
        1563066631590,
        "Good UX Intro.ipynb[0]",
        []
    ],
    [
        1563066631631,
        "Good UX Intro.ipynb[1]",
        []
    ],
    [
        1563066631672,
        "Good UX Intro.ipynb[2]",
        []
    ],
    [
        1563066631707,
        "Good UX Intro.ipynb[3]",
        []
    ],
    [
        1563066631715,
        "Good UX Intro.ipynb[4]",
        []
    ],
    [
        1563066594744,
        "google authorize.ipynb[0]",
        [
            "authorize",
            "storeToken",
            "receiveCode",
            "errorFallback",
            "renewToken"
        ]
    ],
    [
        1563066594846,
        "google authorize.ipynb[1]",
        [
            "authorizeSelenium"
        ]
    ],
    [
        1563066594963,
        "google authorize.ipynb[2]",
        []
    ],
    [
        1563066594967,
        "google authorize.ipynb[3]",
        []
    ],
    [
        1563066674640,
        "google calendar data.ipynb[0]",
        []
    ],
    [
        1563066674687,
        "google calendar data.ipynb[1]",
        [
            "chromeDtToDate"
        ]
    ],
    [
        1563066674915,
        "google calendar data.ipynb[2]",
        [
            "getBookmarksFromTakeout"
        ]
    ],
    [
        1563066675018,
        "google calendar data.ipynb[3]",
        [
            "getHistory"
        ]
    ],
    [
        1563066675050,
        "google calendar data.ipynb[4]",
        []
    ],
    [
        1563066675274,
        "google calendar data.ipynb[5]",
        [
            "syncChromeBookmarks"
        ]
    ],
    [
        1563066675814,
        "google calendar data.ipynb[6]",
        [
            "syncChromeHistory",
            "loadChromeHistory"
        ]
    ],
    [
        1563066676059,
        "google calendar data.ipynb[7]",
        [
            "syncChrome",
            "unzip"
        ]
    ],
    [
        1563066676086,
        "google calendar data.ipynb[8]",
        []
    ],
    [
        1563066671641,
        "google calendar graphs.ipynb[0]",
        []
    ],
    [
        1563066671855,
        "google calendar graphs.ipynb[1]",
        [
            "calendarSearchToHeatmap"
        ]
    ],
    [
        1563066672114,
        "google calendar graphs.ipynb[2]",
        [
            "calendarSearchToSwimlane"
        ]
    ],
    [
        1563066672264,
        "google calendar graphs.ipynb[3]",
        [
            "calendarSearchToPieChart"
        ]
    ],
    [
        1563066672268,
        "google calendar graphs.ipynb[4]",
        []
    ],
    [
        1563066667447,
        "google calendar.ipynb[0]",
        [
            "authorizeCalendar"
        ]
    ],
    [
        1563066667765,
        "google calendar.ipynb[1]",
        [
            "listEvents",
            "processResult"
        ]
    ],
    [
        1563066668057,
        "google calendar.ipynb[2]",
        [
            "runTodaysEvents"
        ]
    ],
    [
        1563066668119,
        "google calendar.ipynb[3]",
        []
    ],
    [
        1563066668168,
        "google calendar.ipynb[4]",
        [
            "correctTimeLimits"
        ]
    ],
    [
        1563066668426,
        "google calendar.ipynb[5]",
        [
            "correctCalendarId",
            "filterCalendar"
        ]
    ],
    [
        1563066668688,
        "google calendar.ipynb[6]",
        [
            "getDaysEvents"
        ]
    ],
    [
        1563066669183,
        "google calendar.ipynb[7]",
        [
            "updateEvent"
        ]
    ],
    [
        1563066669293,
        "google calendar.ipynb[8]",
        [
            "createNewEvent"
        ]
    ],
    [
        1563066944445,
        "google cloud api.ipynb[0]",
        [
            "listUrlMaps",
            "listTargetProxies",
            "listGlobalForwards",
            "listBackendBuckets",
            "listSslCertificates"
        ]
    ],
    [
        1563066944665,
        "google cloud api.ipynb[1]",
        []
    ],
    [
        1563066944735,
        "google cloud api.ipynb[2]",
        [
            "waitForOperation"
        ]
    ],
    [
        1563066946279,
        "google cloud api.ipynb[3]",
        [
            "insertBackendBucket",
            "insertGlobalForward",
            "updateUrlMap",
            "safeName",
            "addSslCertificate",
            "insertSslCertificate",
            "addTargetHttpsProxy",
            "insertTargetHttpsProxy",
            "addGlobalForward"
        ]
    ],
    [
        1563066946326,
        "google cloud api.ipynb[4]",
        []
    ],
    [
        1563066946467,
        "google cloud api.ipynb[5]",
        []
    ],
    [
        1563066947719,
        "google cloud api.ipynb[6]",
        [
            "addIP",
            "safeName",
            "createZone",
            "insertAddress"
        ]
    ],
    [
        1563066947761,
        "google cloud api.ipynb[7]",
        []
    ],
    [
        1563066947764,
        "google cloud api.ipynb[8]",
        []
    ],
    [
        1563066947835,
        "google cloud api.ipynb[9]",
        [
            "authorizeGoogle"
        ]
    ],
    [
        1563066941811,
        "google cloud commands.ipynb[0]",
        "Line 1: Unexpected token ["
    ],
    [
        1563066941811,
        "google cloud commands.ipynb[1]",
        "Line 1: Unexpected token ["
    ],
    [
        1563066941814,
        "google cloud commands.ipynb[2]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066941815,
        "google cloud commands.ipynb[3]",
        "Line 1: Unexpected string"
    ],
    [
        1563066941815,
        "google cloud commands.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066664818,
        "google contacts.ipynb[0]",
        [
            "getContacts",
            "findMatch",
            "googlePromise",
            "listContacts"
        ]
    ],
    [
        1563066665369,
        "google contacts.ipynb[1]",
        [
            "getSettings",
            "googlePromise",
            "mapMembership"
        ]
    ],
    [
        1563066665395,
        "google contacts.ipynb[2]",
        []
    ],
    [
        1563066665399,
        "google contacts.ipynb[3]",
        []
    ],
    [
        1563066662087,
        "google drive.ipynb[0]",
        [
            "authorizeDrive"
        ]
    ],
    [
        1563066662157,
        "google drive.ipynb[1]",
        []
    ],
    [
        1563066662210,
        "google drive.ipynb[2]",
        [
            "listDrive"
        ]
    ],
    [
        1563066662280,
        "google drive.ipynb[3]",
        [
            "insertPermission"
        ]
    ],
    [
        1563066662304,
        "google drive.ipynb[4]",
        []
    ],
    [
        1563066662389,
        "google drive.ipynb[5]",
        [
            "createSheet"
        ]
    ],
    [
        1563066662444,
        "google drive.ipynb[6]",
        [
            "copyFile"
        ]
    ],
    [
        1563066659566,
        "google maps.ipynb[0]",
        [
            "placesNearby"
        ]
    ],
    [
        1563066659615,
        "google maps.ipynb[1]",
        [
            "placeDetails"
        ]
    ],
    [
        1563066659677,
        "google maps.ipynb[2]",
        [
            "googleGeocodeAddress"
        ]
    ],
    [
        1563066659922,
        "google maps.ipynb[3]",
        [
            "getNearby"
        ]
    ],
    [
        1563066659926,
        "google maps.ipynb[4]",
        []
    ],
    [
        1563066939563,
        "google rpc.ipynb[0]",
        [
            "rpc",
            "functions",
            "https",
            "onRequest",
            "req",
            "res",
            "process",
            "env",
            "HOME",
            "HOMEPATH",
            "USERPROFILE",
            "config",
            "services",
            "Object",
            "keys",
            "reduce",
            "k",
            "forEach",
            "s",
            "toUpperCase",
            "cors",
            "Promise",
            "resolve",
            "then",
            "getResult",
            "command",
            "body",
            "query",
            "result",
            "importer",
            "interpret",
            "method",
            "circles",
            "r",
            "getOwnPropertyNames",
            "alt",
            "key",
            "console",
            "error",
            "status",
            "send",
            "catch",
            "e"
        ]
    ],
    [
        1563066939788,
        "google rpc.ipynb[1]",
        [
            "handler",
            "expressHandler"
        ]
    ],
    [
        1563066939912,
        "google rpc.ipynb[2]",
        []
    ],
    [
        1563066939915,
        "google rpc.ipynb[3]",
        []
    ],
    [
        1563066656366,
        "google scheduling.ipynb[0]",
        [
            "adjustPurpleEvents",
            "googlePromise",
            "batchPromises"
        ]
    ],
    [
        1563066656373,
        "google scheduling.ipynb[1]",
        []
    ],
    [
        1563066656378,
        "google scheduling.ipynb[2]",
        []
    ],
    [
        1563066656383,
        "google scheduling.ipynb[3]",
        []
    ],
    [
        1563066656391,
        "google scheduling.ipynb[4]",
        []
    ],
    [
        1563066656941,
        "google scheduling.ipynb[5]",
        [
            "calendarHeatmap"
        ]
    ],
    [
        1563066657087,
        "google scheduling.ipynb[6]",
        [
            "searchCalendar"
        ]
    ],
    [
        1563066657092,
        "google scheduling.ipynb[7]",
        []
    ],
    [
        1563066652519,
        "google search api.ipynb[0]",
        [
            "search"
        ]
    ],
    [
        1563066652586,
        "google search api.ipynb[1]",
        []
    ],
    [
        1563066652721,
        "google search api.ipynb[2]",
        [
            "authorizeSearch"
        ]
    ],
    [
        1563066648892,
        "google sheets.ipynb[0]",
        [
            "getTemplates"
        ]
    ],
    [
        1563066649048,
        "google sheets.ipynb[1]",
        []
    ],
    [
        1563066650150,
        "google sheets.ipynb[10]",
        [
            "authorizeSheets"
        ]
    ],
    [
        1563066649141,
        "google sheets.ipynb[2]",
        [
            "getInfo"
        ]
    ],
    [
        1563066649215,
        "google sheets.ipynb[3]",
        [
            "getDataSheet"
        ]
    ],
    [
        1563066649366,
        "google sheets.ipynb[4]",
        [
            "getRows"
        ]
    ],
    [
        1563066649487,
        "google sheets.ipynb[5]",
        []
    ],
    [
        1563066649546,
        "google sheets.ipynb[6]",
        [
            "addRow"
        ]
    ],
    [
        1563066649656,
        "google sheets.ipynb[7]",
        [
            "getSheetByName"
        ]
    ],
    [
        1563066650004,
        "google sheets.ipynb[8]",
        [
            "updateRow"
        ]
    ],
    [
        1563066650061,
        "google sheets.ipynb[9]",
        []
    ],
    [
        1563066936683,
        "google storage api.ipynb[0]",
        [
            "createBucket",
            "safeName",
            "setPublic",
            "listBuckets",
            "addBucket"
        ]
    ],
    [
        1563066936731,
        "google storage api.ipynb[1]",
        []
    ],
    [
        1563066936814,
        "google storage api.ipynb[2]",
        [
            "streamToOutput"
        ]
    ],
    [
        1563066936916,
        "google storage api.ipynb[3]",
        [
            "fetchOrStream"
        ]
    ],
    [
        1563066937049,
        "google storage api.ipynb[4]",
        [
            "streamToGoogle"
        ]
    ],
    [
        1563066937133,
        "google storage api.ipynb[5]",
        [
            "copyFileBucket"
        ]
    ],
    [
        1563066937173,
        "google storage api.ipynb[6]",
        []
    ],
    [
        1563066590951,
        "google takeout.ipynb[0]",
        [
            "downloadGoogleTakeout",
            "listTakeouts",
            "takeoutProducts"
        ]
    ],
    [
        1563066590992,
        "google takeout.ipynb[1]",
        [
            "googleTakeout"
        ]
    ],
    [
        1563066591027,
        "google takeout.ipynb[2]",
        []
    ],
    [
        1563066591558,
        "google takeout.ipynb[3]",
        [
            "convertUnicode",
            "addSite"
        ]
    ],
    [
        1563066591561,
        "google takeout.ipynb[4]",
        []
    ],
    [
        1563066584711,
        "google timeline.ipynb[0]",
        [
            "getGoogleTimeline",
            "selectDate"
        ]
    ],
    [
        1563066585376,
        "google timeline.ipynb[1]",
        [
            "readTimelinePage"
        ]
    ],
    [
        1563066585923,
        "google timeline.ipynb[2]",
        [
            "averageDestinations",
            "toRadians",
            "straightDistance"
        ]
    ],
    [
        1563066586815,
        "google timeline.ipynb[3]",
        [
            "reconcileTimelineLocations",
            "filterEvents"
        ]
    ],
    [
        1563066587033,
        "google timeline.ipynb[4]",
        [
            "loadLocations"
        ]
    ],
    [
        1563066587768,
        "google timeline.ipynb[5]",
        [
            "reconcileTimeline",
            "loadOnce",
            "daysInMonth"
        ]
    ],
    [
        1563066587897,
        "google timeline.ipynb[6]",
        [
            "timelineEvents"
        ]
    ],
    [
        1563066744017,
        "gulp.ipynb[0]",
        []
    ],
    [
        1563066744068,
        "gulp.ipynb[1]",
        [
            "tasksToPromise"
        ]
    ],
    [
        1563066744838,
        "gulp.ipynb[10]",
        []
    ],
    [
        1563066744072,
        "gulp.ipynb[2]",
        "Line 112: Unexpected string"
    ],
    [
        1563066744128,
        "gulp.ipynb[3]",
        []
    ],
    [
        1563066744220,
        "gulp.ipynb[4]",
        []
    ],
    [
        1563066744379,
        "gulp.ipynb[5]",
        [
            "searchNotebooks",
            "bufferToStream"
        ]
    ],
    [
        1563066744417,
        "gulp.ipynb[6]",
        []
    ],
    [
        1563066744776,
        "gulp.ipynb[7]",
        [
            "getRelativeImports"
        ]
    ],
    [
        1563066744826,
        "gulp.ipynb[8]",
        []
    ],
    [
        1563066744830,
        "gulp.ipynb[9]",
        []
    ],
    [
        1563066481222,
        "heartbeat.ipynb[0]",
        [
            "thump"
        ]
    ],
    [
        1563066481520,
        "heartbeat.ipynb[1]",
        [
            "runTodaysHeartbeat"
        ]
    ],
    [
        1563066481525,
        "heartbeat.ipynb[2]",
        []
    ],
    [
        1563066741900,
        "How code should look.ipynb[0]",
        []
    ],
    [
        1563066739714,
        "identity server.ipynb[0]",
        []
    ],
    [
        1563066739728,
        "identity server.ipynb[1]",
        []
    ],
    [
        1563066739847,
        "identity server.ipynb[2]",
        []
    ],
    [
        1563066739865,
        "identity server.ipynb[3]",
        []
    ],
    [
        1563066739870,
        "identity server.ipynb[4]",
        []
    ],
    [
        1563066739872,
        "identity server.ipynb[5]",
        []
    ],
    [
        1563066851929,
        "import.ipynb[0]",
        [
            "import",
            "importNotebook",
            "getCellPath"
        ]
    ],
    [
        1563066852052,
        "import.ipynb[1]",
        [
            "streamJson",
            "walkJson",
            "selectJson"
        ]
    ],
    [
        1563066852092,
        "import.ipynb[2]",
        [
            "regexToArray"
        ]
    ],
    [
        1563066852157,
        "import.ipynb[3]",
        [
            "runAllPromises"
        ]
    ],
    [
        1563066852292,
        "import.ipynb[4]",
        [
            "getCells"
        ]
    ],
    [
        1563066852578,
        "import.ipynb[5]",
        [
            "ctxGlobal",
            "result",
            "makeModule",
            "getCached"
        ]
    ],
    [
        1563066852917,
        "import.ipynb[6]",
        [
            "cacheCells",
            "clearNotebook",
            "getQuestions",
            "accumulateMarkdown"
        ]
    ],
    [
        1563066853071,
        "import.ipynb[7]",
        [
            "displayCell",
            "lookupCell",
            "resultMarkdown"
        ]
    ],
    [
        1563066853553,
        "import.ipynb[8]",
        [
            "interpret",
            "filterFilename",
            "queryDatabase",
            "searchFiles",
            "searchQueryFiles",
            "getAllCells"
        ]
    ],
    [
        1563066853625,
        "import.ipynb[9]",
        [
            "cacheAll"
        ]
    ],
    [
        1563066796290,
        "install Docker on Mac.ipynb[0]",
        []
    ],
    [
        1563066796368,
        "install Docker on Mac.ipynb[1]",
        []
    ],
    [
        1563066793985,
        "install Docker on Windows.ipynb[0]",
        []
    ],
    [
        1563066794134,
        "install Docker on Windows.ipynb[1]",
        []
    ],
    [
        1563066794197,
        "install Docker on Windows.ipynb[2]",
        []
    ],
    [
        1563066794199,
        "install Docker on Windows.ipynb[3]",
        "Line 16: Unexpected token ILLEGAL"
    ],
    [
        1563066794222,
        "install Docker on Windows.ipynb[4]",
        []
    ],
    [
        1563066849587,
        "intent.ipynb[0]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563066849588,
        "intent.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066849623,
        "intent.ipynb[2]",
        []
    ],
    [
        1563066849624,
        "intent.ipynb[3]",
        "Line 1: Invalid left-hand side in assignment"
    ],
    [
        1563066849624,
        "intent.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066849624,
        "intent.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066849624,
        "intent.ipynb[6]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066849631,
        "intent.ipynb[7]",
        []
    ],
    [
        1563066737522,
        "jupyter interaction.ipynb[0]",
        [
            "execute",
            "js_execute"
        ]
    ],
    [
        1563066735003,
        "Jupyter language kernels.ipynb[0]",
        []
    ],
    [
        1563066735030,
        "Jupyter language kernels.ipynb[1]",
        []
    ],
    [
        1563066735031,
        "Jupyter language kernels.ipynb[2]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066735031,
        "Jupyter language kernels.ipynb[3]",
        "Line 4: Unexpected token /"
    ],
    [
        1563066735031,
        "Jupyter language kernels.ipynb[4]",
        "Line 2: Unexpected token /"
    ],
    [
        1563066735032,
        "Jupyter language kernels.ipynb[5]",
        "Line 17: Invalid regular expression: missing /"
    ],
    [
        1563066735033,
        "Jupyter language kernels.ipynb[6]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066735033,
        "Jupyter language kernels.ipynb[7]",
        "Line 4: Unexpected end of input"
    ],
    [
        1563066735047,
        "Jupyter language kernels.ipynb[8]",
        []
    ],
    [
        1563066733050,
        "jupyter.ipynb[0]",
        "Line 3: Unexpected identifier"
    ],
    [
        1563066731087,
        "jwt in csharp.ipynb[0]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066731122,
        "jwt in csharp.ipynb[1]",
        []
    ],
    [
        1563066729188,
        "karma template.ipynb[0]",
        []
    ],
    [
        1563066729194,
        "karma template.ipynb[1]",
        []
    ],
    [
        1563066844029,
        "kernels.ipynb[0]",
        [
            "jsonInterface",
            "wireJson",
            "pathJson"
        ]
    ],
    [
        1563066844102,
        "kernels.ipynb[1]",
        []
    ],
    [
        1563066845434,
        "kernels.ipynb[10]",
        [
            "testProcessKernel"
        ]
    ],
    [
        1563066845509,
        "kernels.ipynb[11]",
        [
            "do_init",
            "do_respond",
            "replMetaKernal"
        ]
    ],
    [
        1563066845624,
        "kernels.ipynb[12]",
        [
            "testProcessKernel",
            "do_respond"
        ]
    ],
    [
        1563066845625,
        "kernels.ipynb[13]",
        "Line 5: Unexpected token ..."
    ],
    [
        1563066845625,
        "kernels.ipynb[14]",
        "Line 4: Unexpected end of input"
    ],
    [
        1563066845626,
        "kernels.ipynb[15]",
        "Line 10: Unexpected token :"
    ],
    [
        1563066845853,
        "kernels.ipynb[16]",
        [
            "wireKernelInterface",
            "wireKernel",
            "addCB"
        ]
    ],
    [
        1563066846894,
        "kernels.ipynb[17]",
        [
            "wireMetaKernel",
            "do_execute",
            "do_display",
            "do_shutdown",
            "do_complete",
            "do_history",
            "do_is_complete",
            "do_inspect"
        ]
    ],
    [
        1563066847048,
        "kernels.ipynb[18]",
        [
            "nodeMetaKernel",
            "do_execute",
            "do_is_complete",
            "do_init"
        ]
    ],
    [
        1563066847105,
        "kernels.ipynb[19]",
        [
            "testNodeMetaKernel"
        ]
    ],
    [
        1563066844260,
        "kernels.ipynb[2]",
        [
            "pythonJson",
            "nodeJson",
            "notebookJson",
            "bashJson",
            "processingJson"
        ]
    ],
    [
        1563066847212,
        "kernels.ipynb[20]",
        [
            "nodeKernel",
            "getVersion"
        ]
    ],
    [
        1563066847215,
        "kernels.ipynb[21]",
        []
    ],
    [
        1563066847222,
        "kernels.ipynb[22]",
        []
    ],
    [
        1563066847266,
        "kernels.ipynb[23]",
        [
            "bashKernel",
            "bashLanguage"
        ]
    ],
    [
        1563066847541,
        "kernels.ipynb[24]",
        [
            "parseMessage",
            "collapseMessage",
            "hash",
            "json"
        ]
    ],
    [
        1563066847669,
        "kernels.ipynb[25]",
        [
            "setupSockets",
            "setupSocket"
        ]
    ],
    [
        1563066847673,
        "kernels.ipynb[26]",
        []
    ],
    [
        1563066847676,
        "kernels.ipynb[27]",
        []
    ],
    [
        1563066844294,
        "kernels.ipynb[3]",
        [
            "languageInterface"
        ]
    ],
    [
        1563066844418,
        "kernels.ipynb[4]",
        [
            "kernelInfoInterface",
            "nativeKernelInfo"
        ]
    ],
    [
        1563066844418,
        "kernels.ipynb[5]",
        "Line 6: Unexpected token ..."
    ],
    [
        1563066844617,
        "kernels.ipynb[6]",
        [
            "nativeMetaKernel",
            "reassignProperties"
        ]
    ],
    [
        1563066844817,
        "kernels.ipynb[7]",
        [
            "do_init",
            "do_is_complete",
            "do_message",
            "do_respond",
            "do_install"
        ]
    ],
    [
        1563066844980,
        "kernels.ipynb[8]",
        [
            "socketMetaKernel",
            "do_respond",
            "do_init"
        ]
    ],
    [
        1563066845319,
        "kernels.ipynb[9]",
        [
            "do_init",
            "do_shutdown",
            "do_message",
            "do_execute",
            "do_complete",
            "do_inspect",
            "do_history",
            "do_is_complete",
            "processMetaKernel"
        ]
    ],
    [
        1563066628851,
        "Landing Pages.ipynb[0]",
        []
    ],
    [
        1563066628884,
        "Landing Pages.ipynb[1]",
        []
    ],
    [
        1563066629095,
        "Landing Pages.ipynb[10]",
        []
    ],
    [
        1563066629113,
        "Landing Pages.ipynb[11]",
        []
    ],
    [
        1563066629130,
        "Landing Pages.ipynb[12]",
        []
    ],
    [
        1563066629149,
        "Landing Pages.ipynb[13]",
        []
    ],
    [
        1563066629167,
        "Landing Pages.ipynb[14]",
        []
    ],
    [
        1563066629180,
        "Landing Pages.ipynb[15]",
        []
    ],
    [
        1563066629199,
        "Landing Pages.ipynb[16]",
        []
    ],
    [
        1563066629216,
        "Landing Pages.ipynb[17]",
        []
    ],
    [
        1563066629230,
        "Landing Pages.ipynb[18]",
        []
    ],
    [
        1563066629245,
        "Landing Pages.ipynb[19]",
        []
    ],
    [
        1563066628913,
        "Landing Pages.ipynb[2]",
        []
    ],
    [
        1563066629249,
        "Landing Pages.ipynb[20]",
        []
    ],
    [
        1563066628939,
        "Landing Pages.ipynb[3]",
        []
    ],
    [
        1563066628959,
        "Landing Pages.ipynb[4]",
        []
    ],
    [
        1563066628983,
        "Landing Pages.ipynb[5]",
        []
    ],
    [
        1563066629011,
        "Landing Pages.ipynb[6]",
        []
    ],
    [
        1563066629033,
        "Landing Pages.ipynb[7]",
        []
    ],
    [
        1563066629053,
        "Landing Pages.ipynb[8]",
        []
    ],
    [
        1563066629075,
        "Landing Pages.ipynb[9]",
        []
    ],
    [
        1563066839667,
        "languages.ipynb[0]",
        []
    ],
    [
        1563066839674,
        "languages.ipynb[1]",
        []
    ],
    [
        1563066840297,
        "languages.ipynb[10]",
        "Line 30: Unexpected quasi //*[contains(@type, \"Call\")"
    ],
    [
        1563066840301,
        "languages.ipynb[11]",
        []
    ],
    [
        1563066841752,
        "languages.ipynb[12]",
        []
    ],
    [
        1563066841756,
        "languages.ipynb[13]",
        []
    ],
    [
        1563066841759,
        "languages.ipynb[14]",
        []
    ],
    [
        1563066841761,
        "languages.ipynb[15]",
        []
    ],
    [
        1563066841832,
        "languages.ipynb[16]",
        [
            "toLogString",
            "toSafeString"
        ]
    ],
    [
        1563066841836,
        "languages.ipynb[17]",
        []
    ],
    [
        1563066841840,
        "languages.ipynb[18]",
        []
    ],
    [
        1563066841843,
        "languages.ipynb[19]",
        []
    ],
    [
        1563066839946,
        "languages.ipynb[2]",
        [
            "transpile",
            "remove",
            "replace"
        ]
    ],
    [
        1563066841847,
        "languages.ipynb[20]",
        []
    ],
    [
        1563066839980,
        "languages.ipynb[3]",
        []
    ],
    [
        1563066839985,
        "languages.ipynb[4]",
        []
    ],
    [
        1563066839989,
        "languages.ipynb[5]",
        []
    ],
    [
        1563066840005,
        "languages.ipynb[6]",
        []
    ],
    [
        1563066840095,
        "languages.ipynb[7]",
        []
    ],
    [
        1563066840264,
        "languages.ipynb[8]",
        [
            "babelTranspile",
            "transpileNotebook"
        ]
    ],
    [
        1563066840295,
        "languages.ipynb[9]",
        []
    ],
    [
        1563066478914,
        "levenshtein.ipynb[0]",
        [
            "levDist"
        ]
    ],
    [
        1563066478980,
        "levenshtein.ipynb[1]",
        [
            "levSort"
        ]
    ],
    [
        1563066478984,
        "levenshtein.ipynb[2]",
        []
    ],
    [
        1563066580442,
        "linkedin connections.ipynb[0]",
        [
            "syncLinkedInContacts",
            "escapeFilename"
        ]
    ],
    [
        1563066581014,
        "linkedin connections.ipynb[1]",
        [
            "scrapeEntireLinkedInProfile",
            "loadEntirePage"
        ]
    ],
    [
        1563066581189,
        "linkedin connections.ipynb[2]",
        [
            "listAllConnections"
        ]
    ],
    [
        1563066581223,
        "linkedin connections.ipynb[3]",
        []
    ],
    [
        1563066581598,
        "linkedin connections.ipynb[4]",
        [
            "listAllConnections",
            "connectLinkedin"
        ]
    ],
    [
        1563066581819,
        "linkedin connections.ipynb[5]",
        [
            "addLinkedinConnections"
        ]
    ],
    [
        1563066581855,
        "linkedin connections.ipynb[6]",
        []
    ],
    [
        1563066581859,
        "linkedin connections.ipynb[7]",
        []
    ],
    [
        1563066575852,
        "linkedin messages.ipynb[0]",
        [
            "loginLinkedIn",
            "enterLinkedIn"
        ]
    ],
    [
        1563066576117,
        "linkedin messages.ipynb[1]",
        [
            "readLinkedInProfileInfo",
            "visitMyProfile"
        ]
    ],
    [
        1563066576268,
        "linkedin messages.ipynb[2]",
        [
            "listLinkedInThreads",
            "scrollLinkedInThreads",
            "getLinkedInThreads"
        ]
    ],
    [
        1563066577003,
        "linkedin messages.ipynb[3]",
        [
            "readLinkedInThread",
            "getThreadParticipants",
            "scrollLinkedInMessages",
            "readLinkedInMessages"
        ]
    ],
    [
        1563066577474,
        "linkedin messages.ipynb[4]",
        [
            "scrapeLinkedInThreads",
            "readThread",
            "listThreads"
        ]
    ],
    [
        1563066577496,
        "linkedin messages.ipynb[5]",
        []
    ],
    [
        1563066577500,
        "linkedin messages.ipynb[6]",
        []
    ],
    [
        1563066626467,
        "marketing scripts.ipynb[0]",
        [
            "checkout"
        ]
    ],
    [
        1563066626601,
        "marketing scripts.ipynb[1]",
        [
            "contactUs"
        ]
    ],
    [
        1563066626602,
        "marketing scripts.ipynb[2]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066785717,
        "math.ipynb[0]",
        [
            "spliceClone",
            "permute",
            "solve"
        ]
    ],
    [
        1563066785723,
        "math.ipynb[1]",
        []
    ],
    [
        1563066476271,
        "mono.ipynb[0]",
        []
    ],
    [
        1563066476418,
        "mono.ipynb[1]",
        []
    ],
    [
        1563066476419,
        "mono.ipynb[2]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563066960852,
        "movie database.ipynb[0]",
        [
            "importSQL",
            "insertPrincipals",
            "insertTitles",
            "insertNames"
        ]
    ],
    [
        1563066961507,
        "movie database.ipynb[1]",
        [
            "getTitles",
            "getTitlesByName",
            "getActorsByTitles",
            "getActorTitlesIntersection",
            "getTitleTitleIntersection",
            "getExactName",
            "getExactTitle",
            "getTitle"
        ]
    ],
    [
        1563066961543,
        "movie database.ipynb[2]",
        [
            "dropTitles"
        ]
    ],
    [
        1563066961591,
        "movie database.ipynb[3]",
        []
    ],
    [
        1563066961711,
        "movie database.ipynb[4]",
        [
            "createTables"
        ]
    ],
    [
        1563066961714,
        "movie database.ipynb[5]",
        []
    ],
    [
        1563066726368,
        "ngx-translate.ipynb[0]",
        [
            "getTranslations"
        ]
    ],
    [
        1563066726965,
        "ngx-translate.ipynb[1]",
        [
            "findMisplaced",
            "flattenAllKeys",
            "getUnused",
            "getMissing"
        ]
    ],
    [
        1563066473940,
        "nmap.ipynb[0]",
        []
    ],
    [
        1563066473946,
        "nmap.ipynb[1]",
        []
    ],
    [
        1563066722889,
        "node express.ipynb[0]",
        []
    ],
    [
        1563066722988,
        "node express.ipynb[1]",
        []
    ],
    [
        1563066723015,
        "node express.ipynb[2]",
        []
    ],
    [
        1563066723140,
        "node express.ipynb[3]",
        []
    ],
    [
        1563066723340,
        "node express.ipynb[4]",
        []
    ],
    [
        1563066723349,
        "node express.ipynb[5]",
        []
    ],
    [
        1563066723719,
        "node express.ipynb[6]",
        []
    ],
    [
        1563066723727,
        "node express.ipynb[7]",
        []
    ],
    [
        1563066724131,
        "node express.ipynb[8]",
        [
            "app",
            "restart"
        ]
    ],
    [
        1563066791912,
        "node install.ipynb[0]",
        "Line 1: Unexpected token :"
    ],
    [
        1563066791913,
        "node install.ipynb[1]",
        "Line 1: Unexpected token var"
    ],
    [
        1563066791913,
        "node install.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066719699,
        "node simple-imap.ipynb[0]",
        [
            "imapClient",
            "authorizeGmail"
        ]
    ],
    [
        1563066720100,
        "node simple-imap.ipynb[1]",
        [
            "searchImap",
            "flatten"
        ]
    ],
    [
        1563066720124,
        "node simple-imap.ipynb[2]",
        []
    ],
    [
        1563066720368,
        "node simple-imap.ipynb[3]",
        [
            "scanCommandsEmail"
        ]
    ],
    [
        1563066720419,
        "node simple-imap.ipynb[4]",
        []
    ],
    [
        1563066720644,
        "node simple-imap.ipynb[5]",
        [
            "getAttachments",
            "saveAttachments"
        ]
    ],
    [
        1563066720745,
        "node simple-imap.ipynb[6]",
        [
            "sendEmail"
        ]
    ],
    [
        1563066720799,
        "node simple-imap.ipynb[7]",
        []
    ],
    [
        1563066702355,
        "notebook extensions.ipynb[0]",
        []
    ],
    [
        1563082223523,
        "notebook.ipynb[0]",
        []
    ],
    [
        1563082223561,
        "notebook.ipynb[1]",
        []
    ],
    [
        1563082224303,
        "notebook.ipynb[10]",
        [
            "exportAndDeploy",
            "outputExport"
        ]
    ],
    [
        1563082224633,
        "notebook.ipynb[11]",
        [
            "injectImports"
        ]
    ],
    [
        1563082224755,
        "notebook.ipynb[12]",
        [
            "getExtension"
        ]
    ],
    [
        1563082224892,
        "notebook.ipynb[13]",
        [
            "niceName"
        ]
    ],
    [
        1563082224942,
        "notebook.ipynb[14]",
        [
            "authorTemplate"
        ]
    ],
    [
        1563082223830,
        "notebook.ipynb[2]",
        []
    ],
    [
        1563082223834,
        "notebook.ipynb[3]",
        []
    ],
    [
        1563082223838,
        "notebook.ipynb[4]",
        [
            "replaceCore",
            "replaceProperty"
        ]
    ],
    [
        1563082223966,
        "notebook.ipynb[5]",
        [
            "replaceImports",
            "getImportTemplate",
            "replaceImport"
        ]
    ],
    [
        1563082224115,
        "notebook.ipynb[6]",
        []
    ],
    [
        1563082224139,
        "notebook.ipynb[7]",
        [
            "addImports",
            "addImport"
        ]
    ],
    [
        1563082224276,
        "notebook.ipynb[8]",
        []
    ],
    [
        1563082224299,
        "notebook.ipynb[9]",
        []
    ],
    [
        1563066811390,
        "npm.ipynb[0]",
        []
    ],
    [
        1563066811438,
        "npm.ipynb[1]",
        [
            "checkLocalNPM"
        ]
    ],
    [
        1563066811585,
        "npm.ipynb[2]",
        [
            "npm"
        ]
    ],
    [
        1563066811589,
        "npm.ipynb[3]",
        []
    ],
    [
        1563066471875,
        "openssl.ipynb[0]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471876,
        "openssl.ipynb[1]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471876,
        "openssl.ipynb[2]",
        "Line 3: Unexpected token ILLEGAL"
    ],
    [
        1563066471912,
        "openssl.ipynb[3]",
        []
    ],
    [
        1563066571372,
        "orchestration.ipynb[0]",
        [
            "resizeWindow",
            "getScreenSize"
        ]
    ],
    [
        1563066571650,
        "orchestration.ipynb[1]",
        [
            "onlyOneWindow",
            "closeAllTabs",
            "closeAllWindows"
        ]
    ],
    [
        1563066571927,
        "orchestration.ipynb[2]",
        [
            "tileWindows",
            "openUrl",
            "createNewWindows"
        ]
    ],
    [
        1563066572124,
        "orchestration.ipynb[3]",
        [
            "getAllSessionUrls",
            "repositionSession"
        ]
    ],
    [
        1563066572159,
        "orchestration.ipynb[4]",
        [
            "sendJoke"
        ]
    ],
    [
        1563066572217,
        "orchestration.ipynb[5]",
        [
            "sendFacebookThanks"
        ]
    ],
    [
        1563066572442,
        "orchestration.ipynb[6]",
        [
            "tellJokes"
        ]
    ],
    [
        1563066572664,
        "orchestration.ipynb[7]",
        [
            "cleanUpSessions",
            "closeAllSessions"
        ]
    ],
    [
        1563066573224,
        "orchestration.ipynb[8]",
        [
            "getScreenshots",
            "uploadS3"
        ]
    ],
    [
        1563066573227,
        "orchestration.ipynb[9]",
        []
    ],
    [
        1563066469861,
        "parse.ipynb[0]",
        [
            "escapeRegExp"
        ]
    ],
    [
        1563066832753,
        "patterns.ipynb[0]",
        [
            "walkTree"
        ]
    ],
    [
        1563066832758,
        "patterns.ipynb[1]",
        []
    ],
    [
        1563066834270,
        "patterns.ipynb[10]",
        []
    ],
    [
        1563066833336,
        "patterns.ipynb[2]",
        [
            "evaluateDom",
            "evaluateQuery",
            "selectTree",
            "selectDom"
        ]
    ],
    [
        1563066833499,
        "patterns.ipynb[3]",
        [
            "testProgram",
            "testSelect"
        ]
    ],
    [
        1563066833518,
        "patterns.ipynb[4]",
        []
    ],
    [
        1563066833551,
        "patterns.ipynb[5]",
        [
            "promiseOrResolve"
        ]
    ],
    [
        1563066833619,
        "patterns.ipynb[6]",
        [
            "childClass"
        ]
    ],
    [
        1563066833764,
        "patterns.ipynb[7]",
        [
            "extend",
            "override",
            "testClass",
            "childObj",
            "parentObj"
        ]
    ],
    [
        1563066834187,
        "patterns.ipynb[8]",
        [
            "interface",
            "typeErrorTemplate",
            "standardCompare",
            "arrayCompare",
            "objectCompare"
        ]
    ],
    [
        1563066834266,
        "patterns.ipynb[9]",
        [
            "myInterface",
            "print"
        ]
    ],
    [
        1563066467048,
        "polyfills.ipynb[0]",
        [
            "request"
        ]
    ],
    [
        1563066467056,
        "polyfills.ipynb[1]",
        []
    ],
    [
        1563066699271,
        "promisify automock socketio.ipynb[0]",
        [
            "promisifyMock"
        ]
    ],
    [
        1563066700107,
        "promisify automock socketio.ipynb[1]",
        [
            "sockifyRequire",
            "sockifyServer"
        ]
    ],
    [
        1563066700218,
        "promisify automock socketio.ipynb[2]",
        [
            "sockifyClient"
        ]
    ],
    [
        1563066700256,
        "promisify automock socketio.ipynb[3]",
        []
    ],
    [
        1563066700315,
        "promisify automock socketio.ipynb[4]",
        []
    ],
    [
        1563066700320,
        "promisify automock socketio.ipynb[5]",
        []
    ],
    [
        1563066783107,
        "python codekatas solutions.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783108,
        "python codekatas solutions.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783109,
        "python codekatas solutions.ipynb[10]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783110,
        "python codekatas solutions.ipynb[11]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783110,
        "python codekatas solutions.ipynb[12]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783110,
        "python codekatas solutions.ipynb[13]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783108,
        "python codekatas solutions.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783108,
        "python codekatas solutions.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783108,
        "python codekatas solutions.ipynb[4]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783108,
        "python codekatas solutions.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783109,
        "python codekatas solutions.ipynb[6]",
        "Line 1: Unexpected token ("
    ],
    [
        1563066783109,
        "python codekatas solutions.ipynb[7]",
        "Line 1: Unexpected token ("
    ],
    [
        1563066783109,
        "python codekatas solutions.ipynb[8]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066783109,
        "python codekatas solutions.ipynb[9]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066464153,
        "qr codes.ipynb[0]",
        []
    ],
    [
        1563066464440,
        "qr codes.ipynb[1]",
        [
            "httpRequest"
        ]
    ],
    [
        1563066464447,
        "qr codes.ipynb[2]",
        []
    ],
    [
        1563066958470,
        "reconcile timeline data.ipynb[0]",
        []
    ],
    [
        1563066697027,
        "register jupter kernels.ipynb[0]",
        "Line 1: Unexpected token"
    ],
    [
        1563066809176,
        "registry.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066956492,
        "resume.ipynb[0]",
        [
            "getCourseSVG"
        ]
    ],
    [
        1563066956524,
        "resume.ipynb[1]",
        []
    ],
    [
        1563066956528,
        "resume.ipynb[2]",
        []
    ],
    [
        1563066956532,
        "resume.ipynb[3]",
        []
    ],
    [
        1563066828573,
        "rpc.ipynb[0]",
        [
            "FUNCTION_GROUPS",
            "SELENIUM_GROUPS",
            "UNITTEST_GROUPS",
            "DEFAULT_GROUPS",
            "PUBLIC"
        ]
    ],
    [
        1563066828802,
        "rpc.ipynb[1]",
        [
            "getCellGroups",
            "getUnmatched"
        ]
    ],
    [
        1563066830571,
        "rpc.ipynb[10]",
        []
    ],
    [
        1563066829044,
        "rpc.ipynb[2]",
        [
            "getPermissions"
        ]
    ],
    [
        1563066829173,
        "rpc.ipynb[3]",
        [
            "groupPermissions"
        ]
    ],
    [
        1563066829343,
        "rpc.ipynb[4]",
        [
            "filterCommand"
        ]
    ],
    [
        1563066829675,
        "rpc.ipynb[5]",
        [
            "storeResult",
            "updateResultEvent"
        ]
    ],
    [
        1563066829944,
        "rpc.ipynb[6]",
        [
            "getResult"
        ]
    ],
    [
        1563066830180,
        "rpc.ipynb[7]",
        [
            "getEnvironment"
        ]
    ],
    [
        1563066830528,
        "rpc.ipynb[8]",
        [
            "getRpcFromSpec",
            "assignAndRequest",
            "getResourceParameters"
        ]
    ],
    [
        1563066830566,
        "rpc.ipynb[9]",
        []
    ],
    [
        1563066460610,
        "scraping.ipynb[0]",
        [
            "getLocations",
            "getLocation"
        ]
    ],
    [
        1563066460954,
        "scraping.ipynb[1]",
        [
            "getNearbyJSON",
            "getResultsPage",
            "getAllResults"
        ]
    ],
    [
        1563066461892,
        "scraping.ipynb[2]",
        [
            "loadLocations",
            "getAllLocationsData"
        ]
    ],
    [
        1563066461895,
        "scraping.ipynb[3]",
        []
    ],
    [
        1563066569097,
        "selenium commands.ipynb[0]",
        "Line 1: Unexpected number"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[1]",
        "Line 1: Unexpected token"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[2]",
        "Line 1: Unexpected token"
    ],
    [
        1563066569098,
        "selenium commands.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066567168,
        "selenium demo.ipynb[0]",
        []
    ],
    [
        1563066567178,
        "selenium demo.ipynb[1]",
        []
    ],
    [
        1563066527255,
        "selenium server.ipynb[0]",
        [
            "seleniumServer",
            "promisifyChrome",
            "response"
        ]
    ],
    [
        1563066564161,
        "selenium server.ipynb[1]",
        [
            "router",
            "route"
        ]
    ],
    [
        1563066564458,
        "selenium server.ipynb[2]",
        [
            "go",
            "router",
            "waitForComplete"
        ]
    ],
    [
        1563066564568,
        "selenium server.ipynb[3]",
        [
            "findElements",
            "router"
        ]
    ],
    [
        1563066564767,
        "selenium server.ipynb[4]",
        [
            "deleteSession",
            "createSession",
            "router",
            "getSessionCapabilities"
        ]
    ],
    [
        1563066564917,
        "selenium server.ipynb[5]",
        [
            "getWindowHandles",
            "router"
        ]
    ],
    [
        1563066565050,
        "selenium server.ipynb[6]",
        [
            "clickElement",
            "router"
        ]
    ],
    [
        1563066524889,
        "send social message.ipynb[0]",
        []
    ],
    [
        1563066521853,
        "service auth.ipynb[0]",
        [
            "getCredentials",
            "decrypt"
        ]
    ],
    [
        1563066522199,
        "service auth.ipynb[1]",
        [
            "saveCredentials",
            "encrypt",
            "encryptSet"
        ]
    ],
    [
        1563066522220,
        "service auth.ipynb[2]",
        []
    ],
    [
        1563066522414,
        "service auth.ipynb[3]",
        [
            "readPasswordsHtm"
        ]
    ],
    [
        1563066522612,
        "service auth.ipynb[4]",
        [
            "downloadGooglePasswords",
            "waitForPasswordLoad",
            "copyPasswordRow",
            "copyPasswords"
        ]
    ],
    [
        1563066522637,
        "service auth.ipynb[5]",
        []
    ],
    [
        1563066522979,
        "service auth.ipynb[6]",
        [
            "loginGoogle",
            "enterGoogleUsername",
            "enterGooglePassword",
            "enterCredentials"
        ]
    ],
    [
        1563066522982,
        "service auth.ipynb[7]",
        []
    ],
    [
        1563066807275,
        "sql to json.ipynb[0]",
        [
            "unserialize",
            "importer",
            "runInNewContext",
            "r",
            "__filename",
            "mysqlQuery",
            "showTables",
            "describeTable",
            "selectObjects"
        ]
    ],
    [
        1563066807280,
        "sql to json.ipynb[1]",
        []
    ],
    [
        1563066807282,
        "sql to json.ipynb[2]",
        []
    ],
    [
        1563066622848,
        "study sauce.ipynb[0]",
        []
    ],
    [
        1563066623036,
        "study sauce.ipynb[1]",
        [
            "getSignedUrl",
            "req",
            "res",
            "method",
            "status",
            "end",
            "file",
            "storage",
            "bucket",
            "body",
            "filename",
            "expiresAtMs",
            "Date",
            "now",
            "config",
            "action",
            "expires",
            "contentType",
            "err",
            "url",
            "console",
            "error",
            "send"
        ]
    ],
    [
        1563066623160,
        "study sauce.ipynb[2]",
        [
            "copyStudy"
        ]
    ],
    [
        1563066623514,
        "study sauce.ipynb[3]",
        [
            "renderCards",
            "parseCards"
        ]
    ],
    [
        1563066623773,
        "study sauce.ipynb[4]",
        [
            "authorizeProfile"
        ]
    ],
    [
        1563066623992,
        "study sauce.ipynb[5]",
        [
            "receiveCode",
            "extractProfile",
            "safeName"
        ]
    ],
    [
        1563066624155,
        "study sauce.ipynb[6]",
        [
            "renderUser"
        ]
    ],
    [
        1563066624155,
        "study sauce.ipynb[7]",
        "Line 2: Unexpected token :"
    ],
    [
        1563066624189,
        "study sauce.ipynb[8]",
        [
            "createStudyPack",
            "createCards"
        ]
    ],
    [
        1563082228727,
        "syntax.ipynb[0]",
        [
            "getRequires"
        ]
    ],
    [
        1563082228897,
        "syntax.ipynb[1]",
        [
            "testGetRequires"
        ]
    ],
    [
        1563082229214,
        "syntax.ipynb[10]",
        []
    ],
    [
        1563082229217,
        "syntax.ipynb[11]",
        [
            "treeToHtml",
            "treeToStr"
        ]
    ],
    [
        1563082229427,
        "syntax.ipynb[12]",
        []
    ],
    [
        1563082229433,
        "syntax.ipynb[13]",
        [
            "htmlToTree",
            "accumulateChildNodes"
        ]
    ],
    [
        1563082229708,
        "syntax.ipynb[14]",
        [
            "testASTArray",
            "setupASTArray",
            "htmlEntities"
        ]
    ],
    [
        1563082229850,
        "syntax.ipynb[15]",
        [
            "testHtmlTree"
        ]
    ],
    [
        1563082229898,
        "syntax.ipynb[16]",
        [
            "selectAst",
            "makeExpr"
        ]
    ],
    [
        1563082230014,
        "syntax.ipynb[17]",
        [
            "testCodeToDom"
        ]
    ],
    [
        1563082230098,
        "syntax.ipynb[18]",
        [
            "selectAcorn"
        ]
    ],
    [
        1563082230178,
        "syntax.ipynb[19]",
        [
            "makeXpaths",
            "makeCombinations"
        ]
    ],
    [
        1563082228946,
        "syntax.ipynb[2]",
        [
            "getExports"
        ]
    ],
    [
        1563082230449,
        "syntax.ipynb[20]",
        [
            "testMakeXpaths"
        ]
    ],
    [
        1563082230548,
        "syntax.ipynb[21]",
        [
            "minXpath"
        ]
    ],
    [
        1563082230591,
        "syntax.ipynb[22]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563082230591,
        "syntax.ipynb[23]",
        [
            "exprToXpath"
        ]
    ],
    [
        1563082230703,
        "syntax.ipynb[24]",
        []
    ],
    [
        1563082230728,
        "syntax.ipynb[25]",
        [
            "testExpressions",
            "matchCell",
            "findImport"
        ]
    ],
    [
        1563082230951,
        "syntax.ipynb[26]",
        []
    ],
    [
        1563082230955,
        "syntax.ipynb[27]",
        []
    ],
    [
        1563082230959,
        "syntax.ipynb[28]",
        []
    ],
    [
        1563082230965,
        "syntax.ipynb[29]",
        []
    ],
    [
        1563082229026,
        "syntax.ipynb[3]",
        [
            "testGetExports"
        ]
    ],
    [
        1563082230971,
        "syntax.ipynb[30]",
        []
    ],
    [
        1563082230973,
        "syntax.ipynb[31]",
        []
    ],
    [
        1563082230979,
        "syntax.ipynb[32]",
        []
    ],
    [
        1563082230983,
        "syntax.ipynb[33]",
        []
    ],
    [
        1563082229054,
        "syntax.ipynb[4]",
        [
            "getParameters"
        ]
    ],
    [
        1563082229104,
        "syntax.ipynb[5]",
        [
            "testGetParameters"
        ]
    ],
    [
        1563082229130,
        "syntax.ipynb[6]",
        [
            "getImports"
        ]
    ],
    [
        1563082229180,
        "syntax.ipynb[7]",
        [
            "testGetImports"
        ]
    ],
    [
        1563082229210,
        "syntax.ipynb[8]",
        []
    ],
    [
        1563082229214,
        "syntax.ipynb[9]",
        "Line 3: Unexpected token ..."
    ],
    [
        1563066694892,
        "test runner.ipynb[0]",
        [
            "testCells",
            "resetTests"
        ]
    ],
    [
        1563066695013,
        "test runner.ipynb[1]",
        [
            "testWatcher"
        ]
    ],
    [
        1563066692035,
        "twilio.ipynb[0]",
        [
            "incomingTwilio"
        ]
    ],
    [
        1563066692277,
        "twilio.ipynb[1]",
        [
            "incomingTwilio",
            "setReminder"
        ]
    ],
    [
        1563066692375,
        "twilio.ipynb[2]",
        [
            "sendTwilio"
        ]
    ],
    [
        1563066692452,
        "twilio.ipynb[3]",
        [
            "callTwilio"
        ]
    ],
    [
        1563066507643,
        "Untitled.ipynb[0]",
        []
    ],
    [
        1563066507648,
        "Untitled.ipynb[1]",
        []
    ],
    [
        1563066519119,
        "utilities.ipynb[0]",
        [
            "scrollClient"
        ]
    ],
    [
        1563066519340,
        "utilities.ipynb[1]",
        [
            "getAllUntil"
        ]
    ],
    [
        1563066519495,
        "utilities.ipynb[2]",
        [
            "clickSpa"
        ]
    ],
    [
        1563066519500,
        "utilities.ipynb[3]",
        []
    ],
    [
        1563066458154,
        "uuid.ipynb[0]",
        [
            "v35",
            "sha1",
            "bytesToUuid",
            "uuidToBytes",
            "stringToBytes"
        ]
    ],
    [
        1563066954453,
        "virtual worlds.ipynb[0]",
        []
    ],
    [
        1563069662209,
        "vnc.ipynb[0]",
        "Line 1: Unexpected token :"
    ],
    [
        1563069662209,
        "vnc.ipynb[1]",
        "Line 1: Unexpected token ."
    ],
    [
        1563069662210,
        "vnc.ipynb[2]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563069662210,
        "vnc.ipynb[3]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563069662210,
        "vnc.ipynb[4]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563069662210,
        "vnc.ipynb[5]",
        "Line 1: Unexpected string"
    ],
    [
        1563069662211,
        "vnc.ipynb[6]",
        "Line 1: Unexpected token ILLEGAL"
    ],
    [
        1563069662211,
        "vnc.ipynb[7]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563069662211,
        "vnc.ipynb[8]",
        "Line 1: Unexpected token ~"
    ],
    [
        1563066515459,
        "webdriver.ipynb[0]",
        [
            "createWebdriverClient"
        ]
    ],
    [
        1563066515746,
        "webdriver.ipynb[1]",
        [
            "connectSession"
        ]
    ],
    [
        1563066515865,
        "webdriver.ipynb[2]",
        [
            "readSessions"
        ]
    ],
    [
        1563066516210,
        "webdriver.ipynb[3]",
        [
            "updateOrAddSession",
            "lockPromise"
        ]
    ],
    [
        1563066516433,
        "webdriver.ipynb[4]",
        [
            "getSessions",
            "lockPromise",
            "updateOrAddSession"
        ]
    ],
    [
        1563066516694,
        "webdriver.ipynb[5]",
        [
            "lockPromise",
            "verifySession",
            "updateOrAddSession",
            "scanning",
            "addPlugins"
        ]
    ],
    [
        1563066516715,
        "webdriver.ipynb[6]",
        []
    ],
    [
        1563066516739,
        "webdriver.ipynb[7]",
        []
    ],
    [
        1563066516747,
        "webdriver.ipynb[8]",
        []
    ],
    [
        1563066516750,
        "webdriver.ipynb[9]",
        []
    ],
    [
        1563066512574,
        "What is Selenium.ipynb[0]",
        [
            "getSeleniumServer"
        ]
    ],
    [
        1563066512742,
        "What is Selenium.ipynb[1]",
        [
            "vncIframe",
            "urlEncode"
        ]
    ],
    [
        1563066512902,
        "What is Selenium.ipynb[2]",
        [
            "runSeleniumCell"
        ]
    ],
    [
        1563066512921,
        "What is Selenium.ipynb[3]",
        [
            "testLive"
        ]
    ],
    [
        1563066512926,
        "What is Selenium.ipynb[4]",
        []
    ],
    [
        1563069658632,
        "windows.ipynb[0]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563069658633,
        "windows.ipynb[1]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563069658668,
        "windows.ipynb[2]",
        []
    ],
    [
        1563069658669,
        "windows.ipynb[3]",
        "Line 1: Unexpected token <"
    ],
    [
        1563069658675,
        "windows.ipynb[4]",
        []
    ],
    [
        1563069658676,
        "windows.ipynb[5]",
        "Line 2: Unexpected identifier"
    ],
    [
        1563069658680,
        "windows.ipynb[6]",
        []
    ],
    [
        1563069658681,
        "windows.ipynb[7]",
        "Line 1: Unexpected string"
    ],
    [
        1563066620094,
        "Wireframing.ipynb[0]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620154,
        "Wireframing.ipynb[1]",
        []
    ],
    [
        1563066620694,
        "Wireframing.ipynb[10]",
        []
    ],
    [
        1563066620843,
        "Wireframing.ipynb[11]",
        []
    ],
    [
        1563066620843,
        "Wireframing.ipynb[12]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620843,
        "Wireframing.ipynb[13]",
        "Line 1: Unexpected token %"
    ],
    [
        1563066620846,
        "Wireframing.ipynb[14]",
        []
    ],
    [
        1563066620167,
        "Wireframing.ipynb[2]",
        []
    ],
    [
        1563066620235,
        "Wireframing.ipynb[3]",
        [
            "npmls"
        ]
    ],
    [
        1563066620277,
        "Wireframing.ipynb[4]",
        []
    ],
    [
        1563066620361,
        "Wireframing.ipynb[5]",
        []
    ],
    [
        1563066620396,
        "Wireframing.ipynb[6]",
        []
    ],
    [
        1563066620403,
        "Wireframing.ipynb[7]",
        []
    ],
    [
        1563066620448,
        "Wireframing.ipynb[8]",
        []
    ],
    [
        1563066620542,
        "Wireframing.ipynb[9]",
        []
    ],
    [
        1563066509961,
        "you earned it.ipynb[0]",
        [
            "highFive",
            "loginYouEarnedIt",
            "doHighFive"
        ]
    ],
    [
        1563066509997,
        "you earned it.ipynb[1]",
        [
            "runHighFiver"
        ]
    ],
    [
        1563066510018,
        "you earned it.ipynb[2]",
        []
    ],
    [
        1563066510023,
        "you earned it.ipynb[3]",
        []
    ],
    [
        1563066678709,
        "zuora to eloqua.ipynb[0]",
        [
            "csvToJson",
            "createBulkExportJob",
            "getBulkExportFile",
            "getBulkExportJobStatus",
            "getAuthHeaders"
        ]
    ],
    [
        1563066678803,
        "zuora to eloqua.ipynb[1]",
        [
            "getCatalog"
        ]
    ],
    [
        1563066682511,
        "zuora to eloqua.ipynb[10]",
        [
            "bulkImportTemplate",
            "contentCreateTemplate",
            "temporaryImportTemplate"
        ]
    ],
    [
        1563066682553,
        "zuora to eloqua.ipynb[11]",
        []
    ],
    [
        1563066682667,
        "zuora to eloqua.ipynb[12]",
        [
            "getImportData",
            "getOauthToken",
            "getEloquaConfig"
        ]
    ],
    [
        1563066682812,
        "zuora to eloqua.ipynb[13]",
        [
            "getCustomDataObject",
            "getImportDefinitions"
        ]
    ],
    [
        1563066683031,
        "zuora to eloqua.ipynb[14]",
        []
    ],
    [
        1563066683190,
        "zuora to eloqua.ipynb[15]",
        [
            "handler"
        ]
    ],
    [
        1563066683468,
        "zuora to eloqua.ipynb[16]",
        []
    ],
    [
        1563066683685,
        "zuora to eloqua.ipynb[17]",
        [
            "handler"
        ]
    ],
    [
        1563066683950,
        "zuora to eloqua.ipynb[18]",
        []
    ],
    [
        1563066684156,
        "zuora to eloqua.ipynb[19]",
        [
            "getZuoraMonth"
        ]
    ],
    [
        1563066679111,
        "zuora to eloqua.ipynb[2]",
        []
    ],
    [
        1563066684619,
        "zuora to eloqua.ipynb[20]",
        []
    ],
    [
        1563066684833,
        "zuora to eloqua.ipynb[21]",
        [
            "getZuoraAccounts",
            "zuoraQuery",
            "getContact",
            "getAccountById",
            "getPaymentMethod",
            "getAccountLast4Digits"
        ]
    ],
    [
        1563066684942,
        "zuora to eloqua.ipynb[22]",
        []
    ],
    [
        1563066685130,
        "zuora to eloqua.ipynb[23]",
        [
            "bulkUploadEloqua"
        ]
    ],
    [
        1563066685917,
        "zuora to eloqua.ipynb[24]",
        []
    ],
    [
        1563066685941,
        "zuora to eloqua.ipynb[25]",
        []
    ],
    [
        1563066686274,
        "zuora to eloqua.ipynb[26]",
        [
            "calculatePrice"
        ]
    ],
    [
        1563066688867,
        "zuora to eloqua.ipynb[27]",
        [
            "filterROR",
            "rorsToAccounts",
            "totalFilteredRecords",
            "accountTotals",
            "verifyMissing",
            "validateWorksheet",
            "compareRecordsCatalog"
        ]
    ],
    [
        1563066688870,
        "zuora to eloqua.ipynb[28]",
        []
    ],
    [
        1563066689289,
        "zuora to eloqua.ipynb[29]",
        [
            "zuoraQuery",
            "getContact",
            "getContactByAccount",
            "getAccountById",
            "getAccount",
            "getSubscription",
            "getPaymentMethod",
            "getRatePlans"
        ]
    ],
    [
        1563066679240,
        "zuora to eloqua.ipynb[3]",
        [
            "getQuery"
        ]
    ],
    [
        1563066689375,
        "zuora to eloqua.ipynb[30]",
        []
    ],
    [
        1563066689733,
        "zuora to eloqua.ipynb[31]",
        []
    ],
    [
        1563066689737,
        "zuora to eloqua.ipynb[32]",
        []
    ],
    [
        1563066679325,
        "zuora to eloqua.ipynb[4]",
        []
    ],
    [
        1563066679326,
        "zuora to eloqua.ipynb[5]",
        "Line 1: Unexpected identifier"
    ],
    [
        1563066679654,
        "zuora to eloqua.ipynb[6]",
        []
    ],
    [
        1563066680626,
        "zuora to eloqua.ipynb[7]",
        [
            "mapDataToFields",
            "mapRatePlanToProduct"
        ]
    ],
    [
        1563066682265,
        "zuora to eloqua.ipynb[8]",
        []
    ]
]

module.exports = {
    exportsCache
}


#### create export cache?

create cell cache?

Done: sort and index in an Array instead of object with properties

Done: make this a demonstration of idempotence, it keeps loading files.ipynb

Done: generalize this so the database can be anywhere, not just a JSON notebook


In [ ]:
var importer = require('../Core')
var {getExports} = importer.import('get exports from source')
var {
    refreshCache, memorySafe, updateCache,
    updateCode, exportsCache, cellCache
} = importer.import('cache.ipynb')

function makeCamel(str) {
    return str
        .replace(/[^a-z0-9 ]/ig, '')
        .split(' ')
        .map((w, i) => i > 0
             ? (w[0].toUpperCase() + w.substr(1))
             : w)
        .join('')
}

function createCellCache(search, cache, cacheCell, callback) {
    var camelCase = makeCamel(cacheCell.questions[0])
    
    var allIds = cellCache
        .filter(c => c.code.length < 100000)
        .map(cell => cell.id)
    
    if(!search) {
        search = refreshCache(cache, allIds)
        if(search) {
            search = search.replace(/\.ipynb\[[0-9]+\]$/ig, '.ipynb')
        }
    }

    var allCells = [].concat.apply([], importer.interpret(Array.isArray(search)
                                                          ? search : [search]))
    var allCellIds = allCells.map(c => c.id)
    
    // get only first occurrence
    allCells = allCells
        .filter((c, i) => allCellIds.indexOf(c.id) == i)
        .filter(c => c.code.length < 10000)
    
    console.log(`caching ${search} - ${allCells.length} cells`)

    return Promise
        .all(allCells.map(cell => memorySafe(() => {
            try {
                console.log(cell.id)
                return [Date.now(), cell.id, callback(cell.code)]
            } catch (e) {
                return [Date.now(), cell.id, e.message]
            }
        })))
        .then(updates => {
            updateCache(updates, cache, allIds)
        
            var code = `
// ${cacheCell.questions[0]} automatically replaced
var ${camelCase} = ${JSON.stringify(cache, null, 4)}

module.exports = {
    ${camelCase}
}
`
            updateCode(cacheCell, code)
        })
}

function createExportCache(search) {
    var cacheCell = importer.interpret('exports cache')
    return createCellCache(search, exportsCache, cacheCell, getExports)
}

module.exports = {
    createExportCache,
    createCellCache,
}


#### test export cache?


In [ ]:
var importer = require('../Core')
var {createExportCache} = importer.import('create export cache')

module.exports = createExportCache

if(typeof $$ !== 'undefined') {
    createExportCache()
}


### notebook cache tools

Some tools for when to update the notebook cache.



#### update code cell?

Find the code cell following the specified lookup markdown and update the contents.


In [ ]:
var assert = require('assert')
var fs = require('fs')

function updateCode(cell, code) {
    var notebook = JSON.parse(fs.readFileSync(cell.filename))
    var match = (/\/\/.*/ig).exec(code)[0]
    
    assert(match && match.length > 2,
           `nothing to match, include a "//" comment`)
    // replace code cell with new code
    // make sure the cell if where the cache says
    assert(notebook.cells[cell.to].source
           .join('')
           .includes(match),
           `code cell "${match}" could not be located`)
    
    notebook.cells[cell.to].source = code.split('\n')
        .map(line => line + '\n')
    
    fs.writeFileSync(cell.filename, JSON.stringify(notebook, null, 4))
}

module.exports = {
    updateCode
}


#### update regular cache?



In [ ]:

function sortAlphaNumeric(a, b) {
    // convert to strings and force lowercase
    a = typeof a === 'string'
        ? a.toLowerCase()
        : a.toString()
    b = typeof b === 'string'
        ? b.toLowerCase()
        : b.toString()

    return a.localeCompare(b)
}

function cleanCache(cache, allIds) {
    // clean up ids that exist in the index but not in memory
    var extra = cache.filter(e => !allIds.includes(e[1]))
    extra.forEach(e => cache.splice(cache.indexOf(e), 1))
    
    cache.sort((a, b) => {
        return sortAlphaNumeric(a[1], b[1])
    })
}

function updateCache(updates, cache, allIds) {
    var cacheIds = cache.map(e => e[1])
    
    // update with results from search
    updates.forEach(e => {
        var i = cacheIds.indexOf(e[1])
        if(i === -1) {
            cache.push(e)
            cacheIds.push(e[1])
        } else {
            cache.splice(i, 1, e)
        }
    })
    
    cleanCache(cache, allIds)
    
    return cache
}

module.exports = {
    updateCache,
    cleanCache,
    sortAlphaNumeric,
}


#### memory safe async?



In [ ]:

function memorySafe(callback) {
    return new Promise(resolve => {
        setTimeout(() => {
            try {
                if (process.memoryUsage().heapUsed > 500000000) {
                    throw new Error('out of memory')
                }
                return resolve(callback())
            } catch (e) {
                return resolve(e.message)
            }
        }, 100)
    })
}

module.exports = {
    memorySafe
}


#### refresh regular cache?



In [ ]:

function refreshCache(cache, allIds) {
    var cacheIds = cache.map(e => e[1])
    
    // add files that don't exist
    var missing = allIds.filter(n => !cacheIds.includes(n))
    if(missing.length > 0) {
        return missing[0]
    }
    
    // TODO: update any changed notebooks first
    
    // update the oldest record
    cache.sort((a, b) => a[0] - b[0])
    return cache[0][1]
}


module.exports = {
    refreshCache
}


## TODO: project cache

Classifying every project and requirement might take time. This helps functions in the files notebook for finding project files by name.



## TODO: bookmark cache

Always wanted to write something like evernote to automatically pull in the content of pages I bookmark. Incognito mode on news sites to avoid payment. Ad block plugins installed.  Might as well make ad-blocked cached copies of every page I visit.




## TODO: documentation cache

Tools for searching and downloading documention as it related to this library.

